In [ ]:
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "provenance": [],
      "gpuType": "T4"
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    },
    "language_info": {
      "name": "python"
    },
    "accelerator": "GPU"
  },
  "cells": [
    {
      "cell_type": "code",
      "execution_count": 1,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "7oJYv68Cv8xg",
        "outputId": "e3587d2c-9da8-4a05-e7d4-e2cf8f63109f"
      },
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "cuda\n"
          ]
        }
      ],
      "source": [
        "# 1 import library\n",
        "import dis\n",
        "import imageio\n",
        "import matplotlib\n",
        "from tqdm import tqdm\n",
        "\n",
        "import torch\n",
        "import torch.nn as nn\n",
        "import torch.optim as optim\n",
        "from torch.utils.data import DataLoader\n",
        "import matplotlib.pyplot as plt\n",
        "\n",
        "from torchvision.utils import make_grid, save_image\n",
        "import torchvision.datasets as datasets\n",
        "import torchvision.transforms as transforms\n",
        "\n",
        "matplotlib.style.use('ggplot')\n",
        "\n",
        "device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')\n",
        "print(device)"
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "# 2 variable setting\n",
        "batch_size = 512\n",
        "epochs = 200\n",
        "sample_size = 64\n",
        "nz = 128\n",
        "k = 1"
      ],
      "metadata": {
        "id": "B0aveaQPwZNk"
      },
      "execution_count": 2,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "# 3 mnist download\n",
        "transform = transforms.Compose(\n",
        "    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]\n",
        ")\n",
        "train_dataset = datasets.MNIST(\n",
        "    root=\"data2\", train=True, transform=transform, download=True\n",
        ")\n",
        "train_loader = DataLoader(\n",
        "    train_dataset, batch_size=batch_size, shuffle=True, num_workers=4\n",
        ")"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "sBBRQfW7wj2C",
        "outputId": "5c2ff10e-0923-4cf8-f8d6-b65a7da3639d"
      },
      "execution_count": 3,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stderr",
          "text": [
            "100%|██████████| 9.91M/9.91M [00:00<00:00, 63.5MB/s]\n",
            "100%|██████████| 28.9k/28.9k [00:00<00:00, 1.75MB/s]\n",
            "100%|██████████| 1.65M/1.65M [00:00<00:00, 13.0MB/s]\n",
            "100%|██████████| 4.54k/4.54k [00:00<00:00, 8.07MB/s]\n",
            "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.\n",
            "  warnings.warn(\n"
          ]
        }
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "# 4 Generator\n",
        "class Generator(nn.Module):\n",
        "    def __init__(self, nz):\n",
        "        super().__init__()\n",
        "        self.nz = nz\n",
        "        self.main = nn.Sequential(\n",
        "            nn.Linear(self.nz, 256),\n",
        "            nn.LeakyReLU(0.2),\n",
        "            nn.Linear(256, 512),\n",
        "            nn.LeakyReLU(0.2),\n",
        "            nn.Linear(512, 1024),\n",
        "            nn.LeakyReLU(0.2),\n",
        "            nn.Linear(1024, 784),\n",
        "            nn.Tanh(),\n",
        "        )\n",
        "\n",
        "    def forward(self, x):\n",
        "        return self.main(x).view(-1, 1, 28, 28)"
      ],
      "metadata": {
        "id": "jX5pWVHVwpYP"
      },
      "execution_count": 4,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "# 5 Descriminator\n",
        "class Discriminator(nn.Module):\n",
        "    def __init__(self):\n",
        "        super().__init__()\n",
        "        self.n_input = 784\n",
        "        self.main = nn.Sequential(\n",
        "            nn.Linear(self.n_input, 1024),\n",
        "            nn.LeakyReLU(0.2),\n",
        "            nn.Dropout(0.3),\n",
        "            nn.Linear(1024, 512),\n",
        "            nn.LeakyReLU(0.2),\n",
        "            nn.Dropout(0.3),\n",
        "            nn.Linear(512, 256),\n",
        "            nn.LeakyReLU(0.2),\n",
        "            nn.Dropout(0.3),\n",
        "            nn.Linear(256, 1),\n",
        "            nn.Sigmoid(),\n",
        "        )\n",
        "\n",
        "    def forward(self, x):\n",
        "        x = x.view(-1, 784)\n",
        "        return self.main(x)"
      ],
      "metadata": {
        "id": "w9f2tLldw0xM"
      },
      "execution_count": 5,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "# 6 object setting\n",
        "generator = Generator(nz).to(device)\n",
        "discriminator = Discriminator().to(device)\n",
        "print(generator)\n",
        "print(discriminator)"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "93oIyRQxw2mW",
        "outputId": "527ee21a-965e-460b-a0f5-799b4c8bcbb0"
      },
      "execution_count": 6,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Generator(\n",
            "  (main): Sequential(\n",
            "    (0): Linear(in_features=128, out_features=256, bias=True)\n",
            "    (1): LeakyReLU(negative_slope=0.2)\n",
            "    (2): Linear(in_features=256, out_features=512, bias=True)\n",
            "    (3): LeakyReLU(negative_slope=0.2)\n",
            "    (4): Linear(in_features=512, out_features=1024, bias=True)\n",
            "    (5): LeakyReLU(negative_slope=0.2)\n",
            "    (6): Linear(in_features=1024, out_features=784, bias=True)\n",
            "    (7): Tanh()\n",
            "  )\n",
            ")\n",
            "Discriminator(\n",
            "  (main): Sequential(\n",
            "    (0): Linear(in_features=784, out_features=1024, bias=True)\n",
            "    (1): LeakyReLU(negative_slope=0.2)\n",
            "    (2): Dropout(p=0.3, inplace=False)\n",
            "    (3): Linear(in_features=1024, out_features=512, bias=True)\n",
            "    (4): LeakyReLU(negative_slope=0.2)\n",
            "    (5): Dropout(p=0.3, inplace=False)\n",
            "    (6): Linear(in_features=512, out_features=256, bias=True)\n",
            "    (7): LeakyReLU(negative_slope=0.2)\n",
            "    (8): Dropout(p=0.3, inplace=False)\n",
            "    (9): Linear(in_features=256, out_features=1, bias=True)\n",
            "    (10): Sigmoid()\n",
            "  )\n",
            ")\n"
          ]
        }
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "# 7 loss function, optimizer setting\n",
        "optim_g = optim.Adam(generator.parameters(), lr=0.0002)\n",
        "optim_d = optim.Adam(discriminator.parameters(), lr=0.0002)\n",
        "\n",
        "criterion = nn.BCELoss()\n",
        "\n",
        "losses_g = []\n",
        "losses_d = []\n",
        "images = []"
      ],
      "metadata": {
        "id": "5g7Y9YAyxenr"
      },
      "execution_count": 7,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "# 8 image save function\n",
        "def save_generator_image(image, path):\n",
        "    save_image(image, path)"
      ],
      "metadata": {
        "id": "cgsHfaEuxj29"
      },
      "execution_count": 8,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "# 9 descriminator train function\n",
        "def train_discriminator(optimizer, data_real, data_fake):\n",
        "    b_size = data_real.size(0)\n",
        "    real_label = torch.ones(b_size, 1).to(device)\n",
        "    fake_label = torch.zeros(b_size, 1).to(device)\n",
        "    optimizer.zero_grad()\n",
        "    output_real = discriminator(data_real)\n",
        "    loss_real = criterion(output_real, real_label)\n",
        "    output_fake = discriminator(data_fake)\n",
        "    loss_fake = criterion(output_fake, fake_label)\n",
        "    loss_real.backward()\n",
        "    loss_fake.backward()\n",
        "    optimizer.step()\n",
        "    return loss_real + loss_fake"
      ],
      "metadata": {
        "id": "_Yo-tp-QxmYY"
      },
      "execution_count": 9,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "# 10 generator train function\n",
        "def train_generator(optimizer, data_fake):\n",
        "    b_size = data_fake.size(0)\n",
        "    real_label = torch.ones(b_size, 1).to(device)\n",
        "    optimizer.zero_grad()\n",
        "    output = discriminator(data_fake)\n",
        "    loss = criterion(output, real_label)\n",
        "    loss.backward()\n",
        "    optimizer.step()\n",
        "    return loss"
      ],
      "metadata": {
        "id": "UAYWzqvxxo3S"
      },
      "execution_count": 10,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "# 11 model training\n",
        "generator.train()\n",
        "discriminator.train()\n",
        "\n",
        "for epoch in range(epochs):\n",
        "    loss_g = 0.0\n",
        "    loss_d = 0.0\n",
        "    idxh = 0\n",
        "    for idx, data in enumerate(train_loader):\n",
        "        image, _ = data\n",
        "        image = image.to(device)\n",
        "        b_size = len(image)\n",
        "        for step in range(k):\n",
        "            data_fake = generator(torch.randn(b_size, nz).to(device).detach())\n",
        "            data_real = image\n",
        "            loss_d += train_discriminator(optim_d, image, data_fake)\n",
        "        data_fake = generator(torch.randn(b_size, nz).to(device).detach())\n",
        "        loss_g += train_generator(optim_g, data_fake)\n",
        "        idxh = idx\n",
        "\n",
        "    generated_image = generator(torch.randn(sample_size, nz).to(device)).cpu().detach()\n",
        "    generated_image = make_grid(generated_image)\n",
        "    save_generator_image(generated_image, f\"data2/generated_images_{epoch}.png\")\n",
        "    images.append(generated_image)\n",
        "    epoch_loss_g = loss_g / idxh\n",
        "    epoch_loss_d = loss_d / idxh\n",
        "    losses_g.append(epoch_loss_g)\n",
        "    losses_d.append(epoch_loss_d)\n",
        "\n",
        "    print(f\"Epoch {epoch} of {epochs}\")\n",
        "    print(f\"Generator loss: {epoch_loss_g:.8f}, Discriminator loss: {epoch_loss_d:.8f}\")"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "9YeX2MGFxrAy",
        "outputId": "767c0bd7-d129-4f6a-961c-b61db903cc43"
      },
      "execution_count": 11,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Epoch 0 of 200\n",
            "Generator loss: 1.46736825, Discriminator loss: 0.88422835\n",
            "Epoch 1 of 200\n",
            "Generator loss: 3.42396879, Discriminator loss: 1.17165136\n",
            "Epoch 2 of 200\n",
            "Generator loss: 5.22172260, Discriminator loss: 0.43309471\n",
            "Epoch 3 of 200\n",
            "Generator loss: 6.23461008, Discriminator loss: 0.31615496\n",
            "Epoch 4 of 200\n",
            "Generator loss: 6.20954800, Discriminator loss: 0.70262796\n",
            "Epoch 5 of 200\n",
            "Generator loss: 2.37324500, Discriminator loss: 0.98759067\n",
            "Epoch 6 of 200\n",
            "Generator loss: 2.79856682, Discriminator loss: 0.92130983\n",
            "Epoch 7 of 200\n",
            "Generator loss: 2.83879447, Discriminator loss: 0.87121773\n",
            "Epoch 8 of 200\n",
            "Generator loss: 2.43568754, Discriminator loss: 0.75134254\n",
            "Epoch 9 of 200\n",
            "Generator loss: 1.91227782, Discriminator loss: 1.22361934\n",
            "Epoch 10 of 200\n",
            "Generator loss: 5.08863497, Discriminator loss: 0.88915777\n",
            "Epoch 11 of 200\n",
            "Generator loss: 2.89450598, Discriminator loss: 0.93247551\n",
            "Epoch 12 of 200\n",
            "Generator loss: 1.60990071, Discriminator loss: 1.06442881\n",
            "Epoch 13 of 200\n",
            "Generator loss: 1.04128575, Discriminator loss: 1.26612186\n",
            "Epoch 14 of 200\n",
            "Generator loss: 3.38353705, Discriminator loss: 0.44761530\n",
            "Epoch 15 of 200\n",
            "Generator loss: 1.63931584, Discriminator loss: 1.11377561\n",
            "Epoch 16 of 200\n",
            "Generator loss: 1.34334195, Discriminator loss: 1.16043890\n",
            "Epoch 17 of 200\n",
            "Generator loss: 1.02372718, Discriminator loss: 0.96192700\n",
            "Epoch 18 of 200\n",
            "Generator loss: 1.44233382, Discriminator loss: 0.75959712\n",
            "Epoch 19 of 200\n",
            "Generator loss: 3.50957847, Discriminator loss: 0.56345087\n",
            "Epoch 20 of 200\n",
            "Generator loss: 4.27257872, Discriminator loss: 0.66824359\n",
            "Epoch 21 of 200\n",
            "Generator loss: 3.34697771, Discriminator loss: 0.52970421\n",
            "Epoch 22 of 200\n",
            "Generator loss: 3.46989679, Discriminator loss: 0.43533558\n",
            "Epoch 23 of 200\n",
            "Generator loss: 3.56194973, Discriminator loss: 0.42530981\n",
            "Epoch 24 of 200\n",
            "Generator loss: 3.35795116, Discriminator loss: 0.52181554\n",
            "Epoch 25 of 200\n",
            "Generator loss: 3.08038712, Discriminator loss: 0.52920544\n",
            "Epoch 26 of 200\n",
            "Generator loss: 3.50398231, Discriminator loss: 0.44437769\n",
            "Epoch 27 of 200\n",
            "Generator loss: 3.84736085, Discriminator loss: 0.32889691\n",
            "Epoch 28 of 200\n",
            "Generator loss: 4.31762600, Discriminator loss: 0.25096947\n",
            "Epoch 29 of 200\n",
            "Generator loss: 4.04818821, Discriminator loss: 0.40949699\n",
            "Epoch 30 of 200\n",
            "Generator loss: 3.66458821, Discriminator loss: 0.44659749\n",
            "Epoch 31 of 200\n",
            "Generator loss: 3.10808778, Discriminator loss: 0.52750206\n",
            "Epoch 32 of 200\n",
            "Generator loss: 3.34449387, Discriminator loss: 0.41735402\n",
            "Epoch 33 of 200\n",
            "Generator loss: 4.01395702, Discriminator loss: 0.30955860\n",
            "Epoch 34 of 200\n",
            "Generator loss: 3.91020370, Discriminator loss: 0.35422483\n",
            "Epoch 35 of 200\n",
            "Generator loss: 4.05588341, Discriminator loss: 0.39714545\n",
            "Epoch 36 of 200\n",
            "Generator loss: 3.83572006, Discriminator loss: 0.34041199\n",
            "Epoch 37 of 200\n",
            "Generator loss: 3.57581878, Discriminator loss: 0.44948366\n",
            "Epoch 38 of 200\n",
            "Generator loss: 3.32348752, Discriminator loss: 0.44638452\n",
            "Epoch 39 of 200\n",
            "Generator loss: 3.20565939, Discriminator loss: 0.47731730\n",
            "Epoch 40 of 200\n",
            "Generator loss: 3.32272744, Discriminator loss: 0.40913790\n",
            "Epoch 41 of 200\n",
            "Generator loss: 3.61290121, Discriminator loss: 0.35912728\n",
            "Epoch 42 of 200\n",
            "Generator loss: 3.65815091, Discriminator loss: 0.40756634\n",
            "Epoch 43 of 200\n",
            "Generator loss: 3.61441469, Discriminator loss: 0.46538317\n",
            "Epoch 44 of 200\n",
            "Generator loss: 3.62633991, Discriminator loss: 0.36673370\n",
            "Epoch 45 of 200\n",
            "Generator loss: 3.31978297, Discriminator loss: 0.42945099\n",
            "Epoch 46 of 200\n",
            "Generator loss: 3.28798723, Discriminator loss: 0.47180954\n",
            "Epoch 47 of 200\n",
            "Generator loss: 2.89008641, Discriminator loss: 0.51735109\n",
            "Epoch 48 of 200\n",
            "Generator loss: 3.34951591, Discriminator loss: 0.44297865\n",
            "Epoch 49 of 200\n",
            "Generator loss: 3.33733940, Discriminator loss: 0.45816511\n",
            "Epoch 50 of 200\n",
            "Generator loss: 2.95925999, Discriminator loss: 0.55927318\n",
            "Epoch 51 of 200\n",
            "Generator loss: 2.93385530, Discriminator loss: 0.52639091\n",
            "Epoch 52 of 200\n",
            "Generator loss: 3.09516835, Discriminator loss: 0.49864322\n",
            "Epoch 53 of 200\n",
            "Generator loss: 3.02133393, Discriminator loss: 0.51448512\n",
            "Epoch 54 of 200\n",
            "Generator loss: 2.73211622, Discriminator loss: 0.59324473\n",
            "Epoch 55 of 200\n",
            "Generator loss: 2.69786930, Discriminator loss: 0.58185965\n",
            "Epoch 56 of 200\n",
            "Generator loss: 2.46219754, Discriminator loss: 0.61991340\n",
            "Epoch 57 of 200\n",
            "Generator loss: 2.89709663, Discriminator loss: 0.55240101\n",
            "Epoch 58 of 200\n",
            "Generator loss: 2.89937806, Discriminator loss: 0.51080483\n",
            "Epoch 59 of 200\n",
            "Generator loss: 2.81546783, Discriminator loss: 0.57428533\n",
            "Epoch 60 of 200\n",
            "Generator loss: 2.67663145, Discriminator loss: 0.58749473\n",
            "Epoch 61 of 200\n",
            "Generator loss: 2.79741693, Discriminator loss: 0.55714780\n",
            "Epoch 62 of 200\n",
            "Generator loss: 2.89365816, Discriminator loss: 0.52548957\n",
            "Epoch 63 of 200\n",
            "Generator loss: 2.68603992, Discriminator loss: 0.57343668\n",
            "Epoch 64 of 200\n",
            "Generator loss: 2.60098195, Discriminator loss: 0.61906374\n",
            "Epoch 65 of 200\n",
            "Generator loss: 2.66036129, Discriminator loss: 0.58239174\n",
            "Epoch 66 of 200\n",
            "Generator loss: 2.61315274, Discriminator loss: 0.58118606\n",
            "Epoch 67 of 200\n",
            "Generator loss: 2.66039586, Discriminator loss: 0.56382042\n",
            "Epoch 68 of 200\n",
            "Generator loss: 2.50875878, Discriminator loss: 0.62872499\n",
            "Epoch 69 of 200\n",
            "Generator loss: 2.55819321, Discriminator loss: 0.62414300\n",
            "Epoch 70 of 200\n",
            "Generator loss: 2.60519314, Discriminator loss: 0.58613718\n",
            "Epoch 71 of 200\n",
            "Generator loss: 2.59904027, Discriminator loss: 0.63089108\n",
            "Epoch 72 of 200\n",
            "Generator loss: 2.64939976, Discriminator loss: 0.60018486\n",
            "Epoch 73 of 200\n",
            "Generator loss: 2.48600268, Discriminator loss: 0.62091202\n",
            "Epoch 74 of 200\n",
            "Generator loss: 2.40186405, Discriminator loss: 0.66928697\n",
            "Epoch 75 of 200\n",
            "Generator loss: 2.35534334, Discriminator loss: 0.66514075\n",
            "Epoch 76 of 200\n",
            "Generator loss: 2.28242803, Discriminator loss: 0.70972103\n",
            "Epoch 77 of 200\n",
            "Generator loss: 2.25724196, Discriminator loss: 0.72048426\n",
            "Epoch 78 of 200\n",
            "Generator loss: 2.24899340, Discriminator loss: 0.70394552\n",
            "Epoch 79 of 200\n",
            "Generator loss: 2.29950786, Discriminator loss: 0.69236290\n",
            "Epoch 80 of 200\n",
            "Generator loss: 2.15622497, Discriminator loss: 0.73408240\n",
            "Epoch 81 of 200\n",
            "Generator loss: 2.40079427, Discriminator loss: 0.66740584\n",
            "Epoch 82 of 200\n",
            "Generator loss: 2.26062894, Discriminator loss: 0.73538715\n",
            "Epoch 83 of 200\n",
            "Generator loss: 2.33010674, Discriminator loss: 0.69381815\n",
            "Epoch 84 of 200\n",
            "Generator loss: 2.36293602, Discriminator loss: 0.64654064\n",
            "Epoch 85 of 200\n",
            "Generator loss: 2.38298988, Discriminator loss: 0.64725745\n",
            "Epoch 86 of 200\n",
            "Generator loss: 2.34767771, Discriminator loss: 0.68533683\n",
            "Epoch 87 of 200\n",
            "Generator loss: 2.35299158, Discriminator loss: 0.72772163\n",
            "Epoch 88 of 200\n",
            "Generator loss: 2.27350402, Discriminator loss: 0.70225614\n",
            "Epoch 89 of 200\n",
            "Generator loss: 2.32590246, Discriminator loss: 0.69099158\n",
            "Epoch 90 of 200\n",
            "Generator loss: 2.26729202, Discriminator loss: 0.71929646\n",
            "Epoch 91 of 200\n",
            "Generator loss: 2.23014927, Discriminator loss: 0.71019548\n",
            "Epoch 92 of 200\n",
            "Generator loss: 2.13816261, Discriminator loss: 0.74455726\n",
            "Epoch 93 of 200\n",
            "Generator loss: 2.17468739, Discriminator loss: 0.74129355\n",
            "Epoch 94 of 200\n",
            "Generator loss: 2.14808202, Discriminator loss: 0.74559045\n",
            "Epoch 95 of 200\n",
            "Generator loss: 2.04745817, Discriminator loss: 0.76155490\n",
            "Epoch 96 of 200\n",
            "Generator loss: 2.08814096, Discriminator loss: 0.76712543\n",
            "Epoch 97 of 200\n",
            "Generator loss: 2.07406068, Discriminator loss: 0.79190391\n",
            "Epoch 98 of 200\n",
            "Generator loss: 2.02938771, Discriminator loss: 0.79473919\n",
            "Epoch 99 of 200\n",
            "Generator loss: 2.01049471, Discriminator loss: 0.76939929\n",
            "Epoch 100 of 200\n",
            "Generator loss: 1.87322021, Discriminator loss: 0.80877769\n",
            "Epoch 101 of 200\n",
            "Generator loss: 1.94673371, Discriminator loss: 0.80384773\n",
            "Epoch 102 of 200\n",
            "Generator loss: 1.99132729, Discriminator loss: 0.80096698\n",
            "Epoch 103 of 200\n",
            "Generator loss: 1.97372556, Discriminator loss: 0.83542913\n",
            "Epoch 104 of 200\n",
            "Generator loss: 1.89538300, Discriminator loss: 0.81462067\n",
            "Epoch 105 of 200\n",
            "Generator loss: 1.94617307, Discriminator loss: 0.84668517\n",
            "Epoch 106 of 200\n",
            "Generator loss: 1.85475385, Discriminator loss: 0.82730645\n",
            "Epoch 107 of 200\n",
            "Generator loss: 1.80992746, Discriminator loss: 0.83209687\n",
            "Epoch 108 of 200\n",
            "Generator loss: 1.87426937, Discriminator loss: 0.82178587\n",
            "Epoch 109 of 200\n",
            "Generator loss: 1.87625384, Discriminator loss: 0.86985695\n",
            "Epoch 110 of 200\n",
            "Generator loss: 1.78478241, Discriminator loss: 0.88494283\n",
            "Epoch 111 of 200\n",
            "Generator loss: 1.84385645, Discriminator loss: 0.85187191\n",
            "Epoch 112 of 200\n",
            "Generator loss: 1.90833473, Discriminator loss: 0.83779973\n",
            "Epoch 113 of 200\n",
            "Generator loss: 1.89481413, Discriminator loss: 0.83231401\n",
            "Epoch 114 of 200\n",
            "Generator loss: 1.95740783, Discriminator loss: 0.79891676\n",
            "Epoch 115 of 200\n",
            "Generator loss: 1.88143444, Discriminator loss: 0.82928592\n",
            "Epoch 116 of 200\n",
            "Generator loss: 1.90542054, Discriminator loss: 0.84123540\n",
            "Epoch 117 of 200\n",
            "Generator loss: 1.85634768, Discriminator loss: 0.84619319\n",
            "Epoch 118 of 200\n",
            "Generator loss: 1.83065820, Discriminator loss: 0.83869416\n",
            "Epoch 119 of 200\n",
            "Generator loss: 1.77742767, Discriminator loss: 0.86958355\n",
            "Epoch 120 of 200\n",
            "Generator loss: 1.86520004, Discriminator loss: 0.86406267\n",
            "Epoch 121 of 200\n",
            "Generator loss: 1.77337122, Discriminator loss: 0.87602949\n",
            "Epoch 122 of 200\n",
            "Generator loss: 1.76132107, Discriminator loss: 0.88130128\n",
            "Epoch 123 of 200\n",
            "Generator loss: 1.87243414, Discriminator loss: 0.84398037\n",
            "Epoch 124 of 200\n",
            "Generator loss: 1.84851372, Discriminator loss: 0.87641889\n",
            "Epoch 125 of 200\n",
            "Generator loss: 1.79083788, Discriminator loss: 0.86882412\n",
            "Epoch 126 of 200\n",
            "Generator loss: 1.82064605, Discriminator loss: 0.84794754\n",
            "Epoch 127 of 200\n",
            "Generator loss: 1.77255750, Discriminator loss: 0.88318670\n",
            "Epoch 128 of 200\n",
            "Generator loss: 1.88537693, Discriminator loss: 0.86483938\n",
            "Epoch 129 of 200\n",
            "Generator loss: 1.78816879, Discriminator loss: 0.88914007\n",
            "Epoch 130 of 200\n",
            "Generator loss: 1.75035012, Discriminator loss: 0.87343025\n",
            "Epoch 131 of 200\n",
            "Generator loss: 1.72198594, Discriminator loss: 0.88041693\n",
            "Epoch 132 of 200\n",
            "Generator loss: 1.82353783, Discriminator loss: 0.84878397\n",
            "Epoch 133 of 200\n",
            "Generator loss: 1.77096450, Discriminator loss: 0.86673498\n",
            "Epoch 134 of 200\n",
            "Generator loss: 1.74484873, Discriminator loss: 0.90161806\n",
            "Epoch 135 of 200\n",
            "Generator loss: 1.74555242, Discriminator loss: 0.90666169\n",
            "Epoch 136 of 200\n",
            "Generator loss: 1.68860877, Discriminator loss: 0.90159059\n",
            "Epoch 137 of 200\n",
            "Generator loss: 1.62660134, Discriminator loss: 0.91403139\n",
            "Epoch 138 of 200\n",
            "Generator loss: 1.74168575, Discriminator loss: 0.91773164\n",
            "Epoch 139 of 200\n",
            "Generator loss: 1.59002507, Discriminator loss: 0.93726575\n",
            "Epoch 140 of 200\n",
            "Generator loss: 1.59673035, Discriminator loss: 0.96404517\n",
            "Epoch 141 of 200\n",
            "Generator loss: 1.61764765, Discriminator loss: 0.94992918\n",
            "Epoch 142 of 200\n",
            "Generator loss: 1.60431612, Discriminator loss: 0.93174356\n",
            "Epoch 143 of 200\n",
            "Generator loss: 1.62319481, Discriminator loss: 0.92941028\n",
            "Epoch 144 of 200\n",
            "Generator loss: 1.65011632, Discriminator loss: 0.92251033\n",
            "Epoch 145 of 200\n",
            "Generator loss: 1.68404937, Discriminator loss: 0.91449010\n",
            "Epoch 146 of 200\n",
            "Generator loss: 1.51860106, Discriminator loss: 0.98845571\n",
            "Epoch 147 of 200\n",
            "Generator loss: 1.53655863, Discriminator loss: 0.96282554\n",
            "Epoch 148 of 200\n",
            "Generator loss: 1.57002890, Discriminator loss: 0.98526454\n",
            "Epoch 149 of 200\n",
            "Generator loss: 1.51471400, Discriminator loss: 0.98472524\n",
            "Epoch 150 of 200\n",
            "Generator loss: 1.51715839, Discriminator loss: 0.97396266\n",
            "Epoch 151 of 200\n",
            "Generator loss: 1.59968162, Discriminator loss: 0.97327143\n",
            "Epoch 152 of 200\n",
            "Generator loss: 1.53299940, Discriminator loss: 0.96471560\n",
            "Epoch 153 of 200\n",
            "Generator loss: 1.58398807, Discriminator loss: 0.94829661\n",
            "Epoch 154 of 200\n",
            "Generator loss: 1.51346278, Discriminator loss: 1.00445032\n",
            "Epoch 155 of 200\n",
            "Generator loss: 1.50478888, Discriminator loss: 0.99687290\n",
            "Epoch 156 of 200\n",
            "Generator loss: 1.46262014, Discriminator loss: 1.01252127\n",
            "Epoch 157 of 200\n",
            "Generator loss: 1.49607241, Discriminator loss: 0.99655026\n",
            "Epoch 158 of 200\n",
            "Generator loss: 1.52781463, Discriminator loss: 0.98752195\n",
            "Epoch 159 of 200\n",
            "Generator loss: 1.47867489, Discriminator loss: 1.01471376\n",
            "Epoch 160 of 200\n",
            "Generator loss: 1.51467872, Discriminator loss: 1.00300515\n",
            "Epoch 161 of 200\n",
            "Generator loss: 1.54080939, Discriminator loss: 0.97816759\n",
            "Epoch 162 of 200\n",
            "Generator loss: 1.47811723, Discriminator loss: 0.99620426\n",
            "Epoch 163 of 200\n",
            "Generator loss: 1.53887701, Discriminator loss: 0.97564328\n",
            "Epoch 164 of 200\n",
            "Generator loss: 1.52927959, Discriminator loss: 1.00405896\n",
            "Epoch 165 of 200\n",
            "Generator loss: 1.45117557, Discriminator loss: 1.01706958\n",
            "Epoch 166 of 200\n",
            "Generator loss: 1.47548151, Discriminator loss: 1.00527322\n",
            "Epoch 167 of 200\n",
            "Generator loss: 1.51851475, Discriminator loss: 0.99193472\n",
            "Epoch 168 of 200\n",
            "Generator loss: 1.49912083, Discriminator loss: 1.00451767\n",
            "Epoch 169 of 200\n",
            "Generator loss: 1.40552342, Discriminator loss: 1.02425110\n",
            "Epoch 170 of 200\n",
            "Generator loss: 1.44233263, Discriminator loss: 0.99554175\n",
            "Epoch 171 of 200\n",
            "Generator loss: 1.46207106, Discriminator loss: 1.03391421\n",
            "Epoch 172 of 200\n",
            "Generator loss: 1.41890121, Discriminator loss: 1.03168583\n",
            "Epoch 173 of 200\n",
            "Generator loss: 1.46766448, Discriminator loss: 0.99336070\n",
            "Epoch 174 of 200\n",
            "Generator loss: 1.40702784, Discriminator loss: 1.02599645\n",
            "Epoch 175 of 200\n",
            "Generator loss: 1.41261339, Discriminator loss: 1.02036309\n",
            "Epoch 176 of 200\n",
            "Generator loss: 1.40844119, Discriminator loss: 1.02781987\n",
            "Epoch 177 of 200\n",
            "Generator loss: 1.40339780, Discriminator loss: 1.04130840\n",
            "Epoch 178 of 200\n",
            "Generator loss: 1.35657942, Discriminator loss: 1.04943955\n",
            "Epoch 179 of 200\n",
            "Generator loss: 1.35479486, Discriminator loss: 1.06971550\n",
            "Epoch 180 of 200\n",
            "Generator loss: 1.35018909, Discriminator loss: 1.05157566\n",
            "Epoch 181 of 200\n",
            "Generator loss: 1.37601984, Discriminator loss: 1.04648519\n",
            "Epoch 182 of 200\n",
            "Generator loss: 1.27074802, Discriminator loss: 1.08620787\n",
            "Epoch 183 of 200\n",
            "Generator loss: 1.41983616, Discriminator loss: 1.04845750\n",
            "Epoch 184 of 200\n",
            "Generator loss: 1.33599675, Discriminator loss: 1.07876766\n",
            "Epoch 185 of 200\n",
            "Generator loss: 1.39789939, Discriminator loss: 1.04706764\n",
            "Epoch 186 of 200\n",
            "Generator loss: 1.35524583, Discriminator loss: 1.05749059\n",
            "Epoch 187 of 200\n",
            "Generator loss: 1.33442926, Discriminator loss: 1.06150913\n",
            "Epoch 188 of 200\n",
            "Generator loss: 1.35934544, Discriminator loss: 1.05114436\n",
            "Epoch 189 of 200\n",
            "Generator loss: 1.31001341, Discriminator loss: 1.07464981\n",
            "Epoch 190 of 200\n",
            "Generator loss: 1.35568869, Discriminator loss: 1.06042707\n",
            "Epoch 191 of 200\n",
            "Generator loss: 1.38085687, Discriminator loss: 1.04377973\n",
            "Epoch 192 of 200\n",
            "Generator loss: 1.41878605, Discriminator loss: 1.02985096\n",
            "Epoch 193 of 200\n",
            "Generator loss: 1.36476052, Discriminator loss: 1.05779946\n",
            "Epoch 194 of 200\n",
            "Generator loss: 1.40310681, Discriminator loss: 1.02962255\n",
            "Epoch 195 of 200\n",
            "Generator loss: 1.36154938, Discriminator loss: 1.04670608\n",
            "Epoch 196 of 200\n",
            "Generator loss: 1.34819019, Discriminator loss: 1.04979885\n",
            "Epoch 197 of 200\n",
            "Generator loss: 1.39085746, Discriminator loss: 1.05156088\n",
            "Epoch 198 of 200\n",
            "Generator loss: 1.41184187, Discriminator loss: 1.05708361\n",
            "Epoch 199 of 200\n",
            "Generator loss: 1.31153858, Discriminator loss: 1.07210433\n"
          ]
        }
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "# 12 pltting\n",
        "plt.figure()\n",
        "losses_g = [fl.item() for fl in losses_g]\n",
        "plt.plot(losses_g, label=\"Generator loss\")\n",
        "losses_d = [fl.item() for fl in losses_d]\n",
        "plt.plot(losses_d, label=\"Discriminator Loss\")\n",
        "plt.legend()\n",
        "plt.show()"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 430
        },
        "id": "-q69L63hxtc4",
        "outputId": "d4453e40-429a-4e9b-d817-dc32238bc628"
      },
      "execution_count": 12,
      "outputs": [
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 640x480 with 1 Axes>"
            ],
            "image/png": "...\n"
          },
          "metadata": {}
        }
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "# 13 image gereration\n",
        "import numpy as np\n",
        "\n",
        "fake_images = generator(torch.randn(batch_size, nz).to(device))\n",
        "for i in range(10):\n",
        "    fake_images_img = np.reshape(fake_images.data.cpu().detach().numpy()[i], (28, 28))\n",
        "    plt.imshow(fake_images_img, cmap=\"gray\")\n",
        "    # plt.savefig(f\"pytorch/data2/fake_images_img_{i}.png\")\n",
        "    plt.show()"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 1000
        },
        "id": "0nDaXA2y9o-5",
        "outputId": "4fee83ff-76ed-4c19-cdc4-a9406286e081"
      },
      "execution_count": 14,
      "outputs": [
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 640x480 with 1 Axes>"
            ],
            "image/png": "iVBORw0KGgoAAAANSUhEUgAAAaAAAAGdCAYAAABU0qcqAAAAOnRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjEwLjAsIGh0dHBzOi8vbWF0cGxvdGxpYi5vcmcvlHJYcgAAAAlwSFlzAAAPYQAAD2EBqD+naQAALihJREFUeJzt3X10lNWBx/FfQl7nJSQQwzsJESKtQkREtlUrgufY08OeFe2K28Metgp1T/Bl26V0t2xRUHoOtlirtafdbjzKsbWmbGnVper6sgjVLrZQEeOBWogJoRGBhCQTEvK2f3Bmlsk8k+ReJ3MnyffzD8x9njv35uaZ+eWZ58590np7e3sFAECSpbvuAABgdCKAAABOEEAAACcIIACAEwQQAMAJAggA4AQBBABwggACADhBAAEAnCCAAABOZLjuQDwrVqzQoUOHosp8Pp927dql6667Tm1tbVHbMjMzjdtob2+36tuYMWOs6pnq7u72LPf7/dq9e7euvfZahUKhqG1ZWVnG7Zw7d86qf8mSlpbmWe73+/XGG2/oc5/7XMw42CooKDCuc/bs2aTUSU/3/nvR7/dHXhd9x6Gnp8e4HSn+mCeazWupv9dFvONhJK44NmHCBM9yn8+n//zP/9Qtt9wS8z4pSR999NFQd02zZ8/Wz372swH3G7IAevHFF/X888+rqalJxcXFuv322zVz5sxB1z906JD2798fVRYMBiVJBw4cUEtLS9Q2mzdemzcByX0AhcfhnXfeiRmH7Oxs43Y6OjrMO5dE8d4M+xsHWxdddJFxHZvws6kTL4D6G4dUD6CMDPO3oK6uLs/y/sZhJAbQ1KlTPcsDgYAkqbq6Wq2trTHbjx07NqT9MjEkH8G9+eab2rZtm774xS9qy5YtKi4u1ubNm3XmzJmhaA4AMAwNSQC98MILWrJkia6//npNnTpVq1evVlZWll5//fWhaA4AMAwl/CO4rq4uHTlyRDfddFOkLD09XXPmzNHhw4dj9u/s7FRnZ2fkcVpamnJzc+Xz+SKn1GHhx33LJbuP4GxO/6XU+QjOaxxsPoKzGbtkGugjOK9xsBX++MJEvI/GklWnv3EYjR/BeY3DSPwILt6x6vf7o/7tK5Gvl3jitd1XwgOoublZPT09ys/PjyrPz8/X8ePHY/bfsWOHtm/fHnk8Y8YMbdmyRbt27YrbRm1tbcL6O5zV19e77kJKSKXPtF2qq6tz3YWUwPFw3u9//3vXXRiQ81lwy5Yt09KlSyOPw395XXfddTpw4EDUvsFgULW1tZo+ffqon4RQX1+vKVOmjPpJCMeOHdPUqVMTNgmhsLDQuI7XTKOBJHoSQl1dnaZNmzbqJyHEOx5G4hnQlClTPMv9fr9+//vf68orr/Q8zpLxh2t5ebl279494H4JD6C8vDylp6erqakpqrypqSnmrEg6P33aawp1W1tb3DeVlpaWUR1AYV7jYDOlergGUJjXONjKyckxruN6FlyY1ziMpgAK8xqHkRhAXjPcLhQKhTz3SdRrZaC2ByPhkxAyMjJUWlqqgwcPRsp6enp08OBBlZWVJbo5AMAwNSQfwS1dulSPP/64SktLNXPmTO3cuVMdHR1atGjRUDQHABiGhiSAPvvZz6q5uVlVVVVqampSSUmJvvnNb3p+BAcAGJ2GbBLC5z//eX3+85+3rr9q1Sr95S9/iSoLX2Bft25dzHWLjRs3Grdhs+yKZPe59ccff2zVlqkLp7QPNZupxLm5ucZ14l2rC1+nSEtLi7lmYXvt48SJE1b1TJWUlBjXiXfxOHxNcsyYMTHXJ23HwYbNdRab4zXe9d7wtWSv68qpvtyUjXjXK8PlOTk5A14vGyzTa4KD3Z/FSAEAThBAAAAnCCAAgBMEEADACQIIAOAEAQQAcIIAAgA4QQABAJwggAAAThBAAAAnCCAAgBMEEADAibTeFL1T0xVXXKH9+/dHlQWDQTU3NysvLy/mpko+n8+4Ddsb0tncrMtmUch4dzcNBoP6+OOPddFFF8WMg83ijrYLVtrcmM+mrXiHaH/Hg9dNDgcjmYu5Jkp/4zCaDMU42Lyv2BxDA9180kt/d8g9ffq0xo0b5zkOiVqgtD/z5s3Tvn37BtyPMyAAgBMEEADACQIIAOAEAQQAcIIAAgA4QQABAJwggAAAThBAAAAnCCAAgBMEEADACQIIAOAEAQQAcIIAAgA4keG6A4ni9/uN67S1tVm1ZbOA+NSpU43rHDt2zLM8KytLktTR0aGOjo6obfFWyO2Pzerekt0KvuG+m4j3M+Xk5ET+7bsCcXt7u3E7/bXVH9vVxJH6bN4jbF9PpuK9/sLl3d3dVq9RL6Y/02D35wwIAOAEAQQAcIIAAgA4QQABAJwggAAAThBAAAAnCCAAgBMEEADACQIIAOAEAQQAcIIAAgA4QQABAJwYMYuRNjU1GdexXTTQpl68hUX7M27cOM/yYDAY2Z6ZmRm17fTp08bt2MrIMD98zp07Z1wn3gKh4ec6d+5czPPa/m5ZWBSflM1ixTbHa7x2wuW9vb2e+ySyrU+6P2dAAAAnCCAAgBMEEADACQIIAOAEAQQAcIIAAgA4QQABAJwggAAAThBAAAAnCCAAgBMEEADACQIIAODEiFmMtKury7iOzaKBkv1Cl6ZaWloG3D7QPkPJZsyzsrKS0o7t79aGzaKsY8eONa4za9Ysz3KfzydJWrBggdra2qK2Pffcc8btSNKZM2eM6/zd3/2dcZ0DBw4Y1+nu7vYsHzNmTOTf8P8HqjMU+rY9GDb9i3fchcszMjI897F5PQ0VzoAAAE4QQAAAJxL+EVxVVZW2b98eVTZ58mQ98sgjiW4KADCMDck1oGnTpulb3/pW5HG8G4oBAEavIQmg9PR05efnD8VTAwBGiCEJoIaGBt15553KzMxUWVmZvvSlL6mwsNBz387OTnV2dkYep6WlKTc3V36/P3Lr6bDw477l4XqmbGdK2ZzR2dzque/ttsP6G4cLxzIVJXIWXH/jkMxba9vMggsEAsZ1wrPd4pV7bU/mbef9fr9xHa/f3UDizRjr73gYTbPg+hsHKTmz4AZ7LKT1Jni+6v79+9Xe3q7JkyersbFR27dv1+nTp7V161bl5ubG7N/3mtGMGTO0ZcuWRHYJAJCCEh5AfYVCIVVUVGjlypVavHhxzPZ4Z0DXXnut3nnnnah9g8Gg6uvrNWXKlJjvv4y2M6CamhqVlJTEjMNoOwOqq6vTtGnTYsYh1c+A8vLyjOtcfPHFnuU+n0/PP/+8/vqv/zrme0DPPvuscTuS3feA7rjjDuM6Bw8eNK7T3xlQbW2tpk+fHnM8jLYzoA8//FDFxcWe3xNMxhlQeXm5du/ePeB+Q/5FVL/fr8mTJ6uhocFze2ZmpucbbSgUivslS68vYI6mAArzGofRFEBhXuOQ6gFkcwz1DRev7X33sT3GbeqFQiHjOjZfpB7ozdrreBhNARQW74vqyQigwR4LQz49rb29XQ0NDUxKAABESfgZ0LZt23TllVeqsLBQjY2NqqqqUnp6uq655ppENwUAGMYSHkCnT5/W97//fbW0tCgvL0+zZ8/W5s2brT7zBgCMXAkPoH/6p39K9FMOSjIXn7S5vmDzuXC86znh8r4TOIaDc+fOJey5wr+Hnp6epF7z6augoMC4zhtvvGFcZ/bs2f1uf/XVV42fM56LLrrIuM7bb79tXGeg61pennnmGc/y8DXT73//+zGvi9raWuN2JOmPf/yjcZ2XX37ZuI7N8TvQa//06dOe5V/96leN2/rBD35gtP9gr4uyRAEAwAkCCADgBAEEAHCCAAIAOEEAAQCcIIAAAE4QQAAAJwggAIATBBAAwAkCCADgBAEEAHCCAAIAODHkN6Sz5ff7Y+5p3t+9zm1ubJVMybwhFpLr0ksvNa4T7+6mqcJmcUybu6ja+MxnPuNZHr7J31VXXRXT/9LSUqu2/vZv/9a4zg033GBc57777jOuE+8mnMFgUM3NzcrLy/N8X7S5eafX3az7M3PmzEHtxxkQAMAJAggA4AQBBABwggACADhBAAEAnCCAAABOEEAAACcIIACAEwQQAMAJAggA4AQBBABwggACADhBAAEAnEjZ1bDLy8sVCASiynw+nyTpr/7qr9TW1ha17cUXX0xa3zByZWSYvyS+/vWvG9c5e/ascZ2Ojo642wKBgFpbW2PK/X6/cTuS1NXVZVznG9/4hnGd3bt3G9cZO3asZ7nP59Nrr72mioqKmPeHnTt3GrcjSfn5+cZ1BrsS9IX6+93GY3PcSXbH+Kuvvmq0/+nTpwe1H2dAAAAnCCAAgBMEEADACQIIAOAEAQQAcIIAAgA4QQABAJwggAAAThBAAAAnCCAAgBMEEADACQIIAOBEyi5G+r//+7/av39/VFkwGJQkvfbaa2ppaYnaNmbMGOM2uru77TuIlDZhwgSreseOHTOuY7O4o9fCoQP5yU9+4lmelZWlNWvW6KmnntK5c+eitqWn2/2NOXnyZOM6NguL1tTUGNe5/vrrPctzc3MlSQUFBcrJyYnaVlBQYNyOJKWlpRnXsVnItaKiwriO7WKkNv0zfX8d7P6cAQEAnCCAAABOEEAAACcIIACAEwQQAMAJAggA4AQBBABwggACADhBAAEAnCCAAABOEEAAACcIIACAEym7GKmpnp4e113AEAkvQhuv3Gt7XV2dVVs2C4va+NGPfmRcZ/369Z7lwWBQa9as0X333RezSO/EiROt+nf27FnjOk1NTcZ1ent7jev0/RnDwosLt7a2qq2tLWqb7fuDzSLHgUDAuM7ChQuN68RbaDZcnp6e7rlPKr1XcgYEAHCCAAIAOGH8eUN1dbWee+45HT16VI2NjVq7dq2uuuqqyPbe3l5VVVXp1VdfVSgU0uzZs7Vq1SpNmjQpoR0HAAxvxmdAHR0dKikp0R133OG5/de//rV+85vfaPXq1fr2t7+t7Oxsbd68OeZGWQCA0c04gObNm6fbbrst6qwnrLe3Vzt37tTNN9+sBQsWqLi4WHfddZcaGxv19ttvJ6TDAICRIaFTfk6cOKGmpibNnTs3Uubz+TRz5kwdPnxYV199dUydzs5OdXZ2Rh6npaUpNzdXPp8vZnZTf7OebGZ22MzASQX9jcNIFO/nDM82spl15FpWVpZxHZtxsB2bZN3i3uY16PP5+i2Ptz2V+f1+4zo2s0Mlu/dK0+NhsL+DhAZQeBrm2LFjo8rHjh0bd4rmjh07tH379sjjGTNmaMuWLdq1a1fcdmpraz9xX0eC+vp6111ICYcOHXLdBWP33HNPwuvU1NRY9mZk2bFjh+suGHvttdcS/py2X0VIJuffA1q2bJmWLl0aeZyWliZJuu6663TgwIGofYPBoGprazV9+vSY7wKMtjOg+vp6TZkyJe53IkaS/v7yP3TokC655BK1trZGbbN9M87MzLSqZ+rRRx81rrNp0ybP8kAgoJqaGpWUlMSMw4QJE6z6Z/M9oDNnzhjXsXkNLliwwLPc5/Npx44dWrZsWcz3gHbu3GncjmR3Jmhj8eLFxnX27dvnWR4MBlVXV6dp06Z5vj8k4wxo7ty5/Z5EhCU0gPLz8yWdPxALCgoi5WfOnFFJSYlnnczMTM8XfVtbW9w315aWllEdQGFe4zAatba2DrtxsJmUM9DP6DUONh/tSHYBZPM7sHkN9g0Xr+0D7ZNqQqGQcZ2Bxjve+0MyAmiw45/Q7wEVFRUpPz9f7777blRHPvjgA5WVlSWyKQDAMGd8BtTe3q6GhobI4xMnTqimpkaBQECFhYX6whe+oF/+8peaNGmSioqK9POf/1wFBQVxT5sBAKOTcQD9+c9/1saNGyOPt23bJun8NZs1a9bob/7mb9TR0aEf//jHamtr0+zZs/XNb37TatYPAGDkMg6gSy+9VFVVVXG3p6Wlafny5Vq+fPkn6lh3d3fMtM7wY69t8Rbm689wvwY0Wtxwww2e5bm5uZKkRYsWxVyzSNZkAslu+vGGDRuM68S7bhQuP3fuXMw+x48fN25Hspsm3tXVZVwnJyfHuE68C/bZ2dmSzv8x3NHREbXN5v0hmX73u98Z12lsbOx3+5EjRzzLi4qKjNsyPcYHu39q/1YAACMWAQQAcIIAAgA4QQABAJwggAAAThBAAAAnCCAAgBMEEADACQIIAOAEAQQAcIIAAgA4QQABAJwggAAATji/JXc8EydO1LRp06LKAoGAJGnq1Kkxtx62XfV3pAnf0txEMlcFt1ll+d///d89y8M/66OPPpqwn8HmbpF5eXnGdWzuOGrDZoVqWzbHns1K4h9//LFneXhl7ZMnT6q9vf0T9y2ZbMZh/PjxnuXBYFDNzc0qKSnxvCNqRob52/6Fd7gejMG+JjgDAgA4QQABAJwggAAAThBAAAAnCCAAgBMEEADACQIIAOAEAQQAcIIAAgA4QQABAJwggAAAThBAAAAnUnYx0oaGBtXV1UWVBYNBSdKxY8c8F9kzZbtAYTIX70wG23EwXaBQkp588knjOuFFJuPJzs6OKWtsbDRuR5JWrlxpXMdmIclUl6xFTM+dO5eUdlLdPffcY1wn3us2XJ6Wlua5j83x2nfx54G0tbUNaj/OgAAAThBAAAAnCCAAgBMEEADACQIIAOAEAQQAcIIAAgA4QQABAJwggAAAThBAAAAnCCAAgBMEEADAiZRdjNRrIb3+Ftlrbm42bmP8+PFWfUvlBRQnTpxoXOfee++1auvmm282rjNp0iTjOllZWcbbn332WeN2JOl//ud/jOt0dnZatQU7J0+e9CzPzc2VJJ06dUpnz56N2nb69GmrtsaNG2dVz9QLL7xgXCcvL8+zPLxoc15enudipDbvlabH+GAXs+UMCADgBAEEAHCCAAIAOEEAAQCcIIAAAE4QQAAAJwggAIATBBAAwAkCCADgBAEEAHCCAAIAOEEAAQCcSNnFSHt7e9Xb2xtTFm9bUVGRcRupvKioJM+FBC8s91qUde7cucbt3H333ead0/8v/mgi3s/Un4EWQvR6zvfff9+4HUm66KKLjOu0tbVZtYXkyciwe6trbGw0rhNeDHSoxTvuxowZE9me6scmZ0AAACcIIACAE8bnpdXV1Xruued09OhRNTY2au3atbrqqqsi2x9//HHt2rUrqk55ebnWr1//yXsLABgxjAOoo6NDJSUlWrx4sb773e967nP55ZeroqLi/xux/PwVADByGSfDvHnzNG/evP6fNCND+fn5tn0CAIwCQ3JqUl1drVWrVsnv9+uyyy7TbbfdFndmSGdnZ9Qsp7S0NOXm5srv98fUCT/2ei6bGVmpfmYWb8ZYf+Pg8/mGtE/DRXZ2tlW9QCBgXMdm1lN3d7dxnYHaT9bsK9fivdZzcnKi/h1OEnncDXQ8DPZ22RfqO+t4IH6/f1D7pfWaPvMFbr311phrQL/97W+VnZ2toqIiNTQ06JlnnlFOTo42b96s9PTYOQ9VVVXavn175PGMGTO0ZcsW2y4BAIaJhJ8CXH311ZH/T58+XcXFxbr77rv13nvvac6cOTH7L1u2TEuXLo08Dv/Vf+211+qdd96J2jcYDKq+vl5TpkxRS0tL1DabM6CzZ88a10mm/s6Ajh07pqlTp8aMw5IlS4zbefrpp636lwrfA8rMzPTcfv/99xu3I0kvvPCCcZ3a2lrjOok+A4r3uhiJli1b5lmek5OjH//4x7rzzjvV3t4ete2xxx6zasvm92RzJmrz/b2PPvoobvs1NTUqKSnxPB6ScQZUXl6u3bt3D7jfkH8GNWHCBAWDQTU0NHgGUGZmpjIzM2PKQ6FQ3BdTS0tLzDabQR2uARTmNQ6p/sWzZOno6LCq19raalzH5k0/kQF0YT9GQwAN9Lptb29P+dd2X0Nx3MU7HpIRQKFQaFD7Dfn3gE6dOqXW1lYVFBQMdVMAgGHE+Ayovb1dDQ0NkccnTpxQTU2NAoGAAoGAfvGLX2jhwoXKz8/XRx99pKeffloTJ05UeXl5QjsOABjejAPoz3/+szZu3Bh5vG3bNknSddddp9WrV6u2tla7du1SKBTSuHHjNHfuXC1fvtzzYzYAwOhlHECXXnqpqqqq4m53teKBzcKitlOWk3Wdpbi42LM8PGVz+vTpMZ8d/+M//qNxO++995555yRdeeWVxnVsPn8+cuSIZ3l6erpmzZqlmpoa9fT0RG374Q9/aNyOlNrXBW0Wpx0/frxVW6dOnbKqZ8pmIktzc7NnefjYamlpiXmNvvXWW+adk/TKK68Y11mxYoVxnTfffNO4zrRp0zzLw5Ny+n7F5ZMwPY4G+z1Q1oIDADhBAAEAnCCAAABOEEAAACcIIACAEwQQAMAJAggA4AQBBABwggACADhBAAEAnCCAAABOEEAAACcIIACAE0N+R9RksbnDZCqvfCydv8Wwl+zs7Mi/fVeXXrRokXE7fr/fuI6kmBWoB+P99983rhPvdsXBYFDNzc2aP3/+qLgTaLxbPYfLvbb/x3/8h1Vbhw8fNq5TWlpqXOfee+81rvP66697lod//jfeeCPmeNi1a5dxO9L5FedNPfzww8Z1bG5Vn57uff4QLk9PT/fcx/TuppLU2NhotH+8Fcv74gwIAOAEAQQAcIIAAgA4QQABAJwggAAAThBAAAAnCCAAgBMEEADACQIIAOAEAQQAcIIAAgA4QQABAJxI2cVIx4wZozFjxsSUxdtmsxipzaJ8tjIyzIf65Zdf9iwPL1z4q1/9KuZnOHXqVFL6JkkvvfSScZ3ly5dbtZXK+h6Lg7F48WLjOl/+8pc9y8O/v+9973sxi9Nec801xu1Idova3nLLLcZ1bBa07fsz9i3v6uqK2cemHcluUdaJEyca12loaDCuk5mZ6Vne3/ukZPdeaTp+g92fMyAAgBMEEADACQIIAOAEAQQAcIIAAgA4QQABAJwggAAAThBAAAAnCCAAgBMEEADACQIIAOAEAQQAcCJlFyPt7u6OWTQv/NhrW05OjnEb7e3t9h00FF5A1MTvfvc7z/KMjAxNnTpVb7/9dsyii2fPnjVu5+OPPzauI0mvvvqqcR3bRSFT2YQJE4zrPPHEE8Z16urqPMvDC04WFxfHfc2YslnU9q233jKu09nZaVwn3jEULu/p6XF6nNksLJpMNmMTb+HTeAa7wDFnQAAAJwggAIATBBAAwAkCCADgBAEEAHCCAAIAOEEAAQCcIIAAAE4QQAAAJwggAIATBBAAwAkCCADgRMouRmrKZmFR24Uaw4s/mrBZADDeIqHhhQFPnjwZs5jj7bffbtzOyZMnjetIUn19vXGdl19+2biOze/JZvFXSbrpppuM69x6663Gdfx+v3Gd8vLyfrdfdtllMWUtLS3G7UjS/Pnzjeu0tbVZtQU7paWlnuXhY6ukpEShUChm+6FDh4zbys7ONto/KytrUPtxBgQAcIIAAgA4YfQR3I4dO7R3717V19crKytLZWVlWrFihSZPnhzZ59y5c9q2bZvefPNNdXZ2qry8XKtWrVJ+fn6i+w4AGMaMzoCqq6t14403avPmzfq3f/s3dXd368EHH4y6/vLUU0/pD3/4g772ta9p48aNamxs1NatWxPecQDA8GYUQOvXr9eiRYs0bdo0lZSUaM2aNTp58qSOHDki6fxFyNdee00rV67UZZddptLSUlVUVOjQoUM6fPjwkPwAAIDh6RPNggvPegkEApKkI0eOqLu7W3PmzInsM2XKFBUWFurw4cMqKyuLeY7Ozs6omVxpaWnKzc2V3+9XMBiM2jf8uG95stm0bzNzLt5tcMPlprfJjcd2xpjpzBjJbuzizYLr73iw/Zlyc3ON6wz29sMu2I6Dze/Jtq1ESZX3h2SJN5MyXB5vu834hN/jB2uwszzTent7e417o/PTih966CGFQiE98MADkqQ9e/bohz/8oX72s59F7fuv//qvuvTSS7VixYqY56mqqtL27dsjj2fMmKEtW7bYdAkAMIxY/+lWWVmpuro6bdq06RN1YNmyZVq6dGnkcfivqGuvvVbvvPNO1L7BYFD19fWaMmWK9fcbLtTU1GRVz2ZChc0Z0EMPPeRZnpmZqX/4h3/Qk08+GfM9oL//+783bufUqVPGdSTpBz/4gXGdH/3oR8Z1+jsDinc82P41fuGxOFjLli0zrrNkyRLjOv2dcfp8Ps/v4bS2thq3I0mXX365cR3bthIl0e8PqW7WrFme5X6/X2+88YY+97nPeX4P6E9/+pNxW6ZnQHPmzNFLL7004H5WAVRZWal9+/Zp48aNGj9+fKQ8Pz9fXV1dCoVCUadgZ86cifumnZmZ6flRUigUinsQtbS0OD3AbNq2CaC+4eK1faB9BsPyJFgdHR3GdWzGbqAvonodD7YBdPbsWeM6XV1dVm0lg+3v1ub35DqAwly/PySLV7j03e61j83YmB5HA/UtzGgSQm9vryorK7V3715t2LBBRUVFUdtLS0s1ZswYvfvuu5Gy48eP6+TJk57XfwAAo5fRGVBlZaX27NmjdevWKTc3N/IRls/nU1ZWlnw+nxYvXqxt27YpEAjI5/PpiSeeUFlZGQEEAIhiFEDhdbzuv//+qPKKigotWrRIkrRy5UqlpaVp69at6urqinwRFQCAC1nPghtqV1xxhfbv3x9VFgwG1dzcrLy8vJjPMceOHWvchu1n1jaLY9pcA/rMZz7jWe7z+fTSSy/pxhtvjLnwfOGMwsGynbYa/v6XCZsLoOvWrfMsDwQC2r9/v+bNmxfzuywsLDRuR5L++7//27iOzcKiNuItIpmenq6ysjIdPnw4ZtHbefPmWbVls7iva/29P4xE8a5zBoNBnTlzRmPHjvUch2S85c+bN0/79u0bcD/WggMAOEEAAQCcIIAAAE4QQAAAJwggAIATBBAAwAkCCADgBAEEAHCCAAIAOEEAAQCcIIAAAE4QQAAAJwggAIAT1rfkTjVnzpxJWls2d9u0qRO+/UU8v/rVr2LK/vCHPxi389nPfta4jiRdeumlxnVs7jj69a9/3bM8fCfde+65J+bOsCtWrDBu58LnNGHzMz355JPGde666y7P8vDqxwsWLIhZ/ThFF7v/ROK9lsLlaWlpMfuk+jjYvD/E+5nC5b29vZ77XHzxxcZtffjhh0b7Z2QMLlo4AwIAOEEAAQCcIIAAAE4QQAAAJwggAIATBBAAwAkCCADgBAEEAHCCAAIAOEEAAQCcIIAAAE4QQAAAJ0bMYqSpLicnx7jOjBkzPMsDgYA++OADzZkzR62trVHbbrrpJuN2rrnmGuM6ktTW1mZcZ8GCBcZ15s+f3+/2lStXxpT96U9/Mm5Hkt5++23jOv/8z/9sXOfjjz82rmO7+ORIMxLHITc317hOvPeUYDAoSRo3bpzn4rpHjhwxbis7O9to/zFjxgxqP86AAABOEEAAACcIIACAEwQQAMAJAggA4AQBBABwggACADhBAAEAnCCAAABOEEAAACcIIACAEwQQAMCJUb0YqddCfYPR2dlpXOfs2bPGdfouNBoWXgT0xIkTamlpidr20ksvGbcTXrzQVE1NjXGdwS5SeKGsrKx+t6elpcWUjR8/3rgdSVq9erVxnfb2dqu2MDJNmjTJuM5f/vIX4zrnzp3zLA8vxNrc3Bzz/nDhdhMdHR0J6VtfnAEBAJwggAAAThBAAAAnCCAAgBMEEADACQIIAOAEAQQAcIIAAgA4QQABAJwggAAAThBAAAAnCCAAgBMpuxhpZmZmzCKU4cdZWVkx2wa7+N2FbBblk6SMDPNh6+npsWrLVF1dnXGd7u5uq7ZKS0uN6zz22GPGdf7lX/7FszwQCOjw4cO65JJLYhZujbeQ60BYWPQ8rwVeB2L7ehppbBYWTU83PxeI954SLu/p6fHcp6SkxLit2tpao/0He/xwBgQAcIIAAgA4YfRZ0o4dO7R3717V19crKytLZWVlWrFihSZPnhzZ5/7771d1dXVUvRtuuEFf+cpXEtNjAMCIYBRA1dXVuvHGG3XxxReru7tbzzzzjB588EE9/PDDysnJiey3ZMkSLV++PPJ4oBuKAQBGH6MAWr9+fdTjNWvWaNWqVTpy5Ig+/elPR8qzs7OVn5+fkA4CAEamTzQLLnxr6EAgEFW+e/du7d69W/n5+Zo/f75uueUWZWdnez5HZ2dn1C2u09LSlJubK7/fH3Or6HA7fduT7GbB2cxms2UzCy5enfC4eN1K2+aW17az4Gxu5W1zG3Sv37ck+f3+qH8vZDOLa7jq73iwNRxnwQ3FOCSLzSy4eAYah3ivp8E852B5vSa9pPVaHjU9PT166KGHFAqF9MADD0TKX3nlFRUWFmrcuHH68MMP9dOf/lQzZ87U2rVrPZ+nqqpK27dvjzyeMWOGtmzZYtMlAMAwYh1AP/nJT/THP/5RmzZt0vjx4+Pud/DgQW3atEmPPvqoJk6cGLM93hnQ4sWLdeDAgah9A4GAampqVFJSEvM9j9F2BlRfX68pU6aopaUlalsyz4Dy8vKM63znO98xrnPfffd5lvv9fu3fv1/z5s1TKBSK2tb38WD1Hc/hoL/jwdZwPQNK9DgkS6LPgOrq6jRt2jTPcZg2bZrxc5p+v7C8vFy7du0acD+rd+DKykrt27dPGzdu7Dd8JGnmzJmSpIaGBs8AyszM9PxYJhQKxT2IWltbY7aNpgAKa2lpcRpANm9SF/6xMVgDfak0FAol7Iuow+2N60Jex4Ot4RhAYYkch2RJZACFxRsHm9eG6XgO9g9Ao5+6t7dXlZWV2rt3rzZs2KCioqIB69TU1EiSCgoKTJoCAIxwRqcAlZWV2rNnj9atW6fc3Fw1NTVJknw+n7KystTQ0KA9e/boiiuuUCAQUG1trZ566il96lOfUnFx8VD0HwAwTBkF0Msvvyzp/JdNL1RRUaFFixYpIyND7777rnbu3KmOjg6NHz9eCxcu1M0335ywDgMARgajAKqqqup3e2FhoTZu3PiJOgQAGB1SdjXsrq6umAvW4cd9Z85JdhdNJ0yYYNW3+vp6q3qm4n1nJlzuNYHDdkKBjebmZuM6K1euTFj74e8mNDQ0DLuLzqkuVSYUjBaJXC1/oNWwjx8/bvycphO2BjsZisVIAQBOEEAAACcIIACAEwQQAMAJAggA4AQBBABwggACADhBAAEAnCCAAABOEEAAACcIIACAEwQQAMCJlF2MtLe3N2ZBxPBjr202krWoqK14dw/tb1FWnGd7h8lELgqZaPEW3A2Xp6WlxewzEhcVjbcwZrg8IyMjZh/bRXpH4vjZ3D3a5/MZ7T/YxUs5AwIAOEEAAQCcIIAAAE4QQAAAJwggAIATBBAAwAkCCADgBAEEAHCCAAIAOEEAAQCcIIAAAE6k7Fpws2fPjinz+/2SpPLycoVCoWR3KWUwDuf1Nw6jaS24/sZhJK5lFm+dsfB6ZXPnzlVbW1vUttG0FtxQvD/k5uYa7V9WVjao/dJ6h+MIAwCGvWH1EdzZs2f1jW98Q2fPnnXdFacYh/MYh/MYh/MYh/OG0zgMqwDq7e3V0aNHh+VpcSIxDucxDucxDucxDucNp3EYVgEEABg5CCAAgBPDKoAyMzP1xS9+UZmZma674hTjcB7jcB7jcB7jcN5wGgdmwQEAnBhWZ0AAgJGDAAIAOEEAAQCcIIAAAE6k7Fpwfb344ot6/vnn1dTUpOLiYt1+++2aOXOm624lVVVVlbZv3x5VNnnyZD3yyCNuOpQk1dXVeu6553T06FE1NjZq7dq1uuqqqyLbe3t7VVVVpVdffVWhUEizZ8/WqlWrNGnSJIe9TryBxuHxxx/Xrl27ouqUl5dr/fr1ye7qkNmxY4f27t2r+vp6ZWVlqaysTCtWrNDkyZMj+5w7d07btm3Tm2++qc7OTpWXl2vVqlXKz8931/EEG8w43H///aquro6qd8MNN+grX/lKsrsb17AIoDfffFPbtm3T6tWrNWvWLP3Xf/2XNm/erEceeURjx4513b2kmjZtmr71rW9FHtsuujmcdHR0qKSkRIsXL9Z3v/vdmO2//vWv9Zvf/EZr1qxRUVGRnn32WW3evFkPP/ywsrKyHPR4aAw0DpJ0+eWXq6KiIvI43sKdw1V1dbVuvPFGXXzxxeru7tYzzzyjBx98UA8//LBycnIkSU899ZT27dunr33ta/L5fKqsrNTWrVv1wAMPOO594gxmHCRpyZIlWr58eeRxqr0ehsXR+cILL2jJkiW6/vrrJUmrV6/Wvn379Prrr+umm25y27kkS09PH1F/yQ3GvHnzNG/ePM9tvb292rlzp26++WYtWLBAknTXXXdp9erVevvtt3X11Vcns6tDqr9xCMvIyBjRx0ffs7k1a9Zo1apVOnLkiD796U+rra1Nr732mu69915ddtllkqSKigp99atf1eHDhwe9SnOqG2gcwrKzs1P6eEj5AOrq6tKRI0eigiY9PV1z5szR4cOH3XXMkYaGBt15553KzMxUWVmZvvSlL6mwsNB1t5w5ceKEmpqaNHfu3EiZz+fTzJkzdfjw4REVQINRXV2tVatWye/367LLLtNtt92mYDDoultDJnzbhUAgIEk6cuSIuru7NWfOnMg+U6ZMUWFh4YgKoL76jkPY7t27tXv3buXn52v+/Pm65ZZblJ2d7aKLnlI+gJqbm9XT0xOT4vn5+Tp+/LibTjkya9YsVVRUaPLkyWpsbNT27du1YcMGbd261fh+HSNFU1OTJMV8FDt27NjIttHi8ssv18KFC1VUVKSGhgY988wz+va3v63NmzePyI9qe3p69OSTT+qSSy7R9OnTJZ0/HjIyMiL3xAkbyceD1zhI0jXXXKPCwkKNGzdOH374oX7605/q+PHjWrt2rcPeRkv5AML/u/Djl+Li4kggvfXWW1q8eLHDniEVXHi2N336dBUXF+vuu+/We++9F3VGMFJUVlaqrq5OmzZtct0Vp+KNww033BD5//Tp01VQUKBNmzapoaFBEydOTHY3PaX8n0V5eXlKT0+P+eulqakppT/bTAa/36/JkyeroaHBdVecCR8DZ86ciSo/c+bMqD8+JkyYoGAwOCKPj8rKSu3bt0/33Xefxo8fHynPz89XV1dXzJ1AR+rxEG8cvIRnDafS8ZDyAZSRkaHS0lIdPHgwUtbT06ODBw+O2M9zB6u9vV0NDQ0j8oU1WEVFRcrPz9e7774bKWtra9MHH3ww6o+PU6dOqbW1VQUFBa67kjC9vb2qrKzU3r17tWHDBhUVFUVtLy0t1ZgxY6KOh+PHj+vkyZMj6ngYaBy81NTUSFJKHQ/D4iO4pUuX6vHHH1dpaalmzpypnTt3qqOjQ4sWLXLdtaTatm2brrzyShUWFqqxsVFVVVVKT0/XNddc47prQyoctGEnTpxQTU2NAoGACgsL9YUvfEG//OUvNWnSJBUVFennP/+5CgoKIrPiRor+xiEQCOgXv/iFFi5cqPz8fH300Ud6+umnNXHiRJWXlzvsdWJVVlZqz549WrdunXJzcyOfjPh8PmVlZcnn82nx4sXatm2bAoGAfD6fnnjiCZWVlY2oABpoHBoaGrRnzx5dccUVCgQCqq2t1VNPPaVPfepTKi4udtv5Cwyb1bBffPFFPffcc2pqalJJSYm+/OUva9asWa67lVSPPPKI3n//fbW0tCgvL0+zZ8/WbbfdljKf5w6V9957Txs3bowpv+6667RmzZrIF1FfeeUVtbW1afbs2brjjjuivpQ3EvQ3DqtXr9Z3vvMdHT16VKFQSOPGjdPcuXO1fPnyEXWGfOutt3qWV1RURP4gDX8R9be//a26urpG5BdRBxqHkydP6rHHHlNdXZ06Ojo0fvx4XXXVVbr55pvl8/mS3Nv4hk0AAQBGlpS/BgQAGJkIIACAEwQQAMAJAggA4AQBBABwggACADhBAAEAnCCAAABOEEAAACcIIACAEwQQAMAJAggA4MT/AWTtDCWRBnLOAAAAAElFTkSuQmCC\n"
          },
          "metadata": {}
        },
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 640x480 with 1 Axes>"
            ],
            "image/png": "iVBORw0KGgoAAAANSUhEUgAAAaAAAAGdCAYAAABU0qcqAAAAOnRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjEwLjAsIGh0dHBzOi8vbWF0cGxvdGxpYi5vcmcvlHJYcgAAAAlwSFlzAAAPYQAAD2EBqD+naQAAMaNJREFUeJzt3Xt0VeWZx/Ff7pdzTkggoiZKEsQYKxDwxhqv3Bwvw3TEG67q6NQGuwRdrRZ1po4IaFxDLQ5jx652OnEpS8dOZGTqBW1HqgIyLVQEgahoIVxCI0WSkDu5nPmDdTKcnH2SvK8neXP5fv6B8+795H3znr3Pk73P3s+OCwaDQQEAMMDiXQ8AADAykYAAAE6QgAAATpCAAABOkIAAAE6QgAAATpCAAABOkIAAAE6QgAAATpCAAABOJLoeQDR/+7d/q88++yyszefz6d1339WMGTPU2NgYtqy9vd24j7i4OKuxJSUlGcd0dnYax3R0dHi2+3w+rV+/XldccUXEPNhUVkpISDCO6Wl8A8Xn82nDhg26/PLLI+bBls02kZKSYhzT0tJiHBNNT/OQmGi3i9vsTzZsxhdtbD3Ng+2+PlD7k00/0T5Tetsv4uPNjztMx1dUVKSXXnqp1/X6LQG9/fbbev3111VbW6u8vDzdddddmjBhQp/jP/vsM23bti2sLRAISJI+/vhj1dfXhy1ra2szHqPtRpmcnGwcY/NhHS0mNA/bt2+PmIeRlIB6mgdbNttEamqqcUxzc7NxTDQ9zcNgT0A2f8xF29d7moeRlIB62y8GIgH1Vb+cgtu0aZNWrVqlm266ScuXL1deXp5KS0tVV1fXH90BAIagfklAb7zxhmbNmqUZM2bojDPO0Pz585WcnKx33323P7oDAAxBMT8F197erj179uj666/vaouPj9ekSZO0e/fuiPXb2trCDqnj4uKUlpYmn8/XdSgZEnrdvT30c0wN9VNwXvMwEk/Bec2DrYE6BWd7asxLT/MwEk/Bec3DSDwFF22/GIhTcD6fr0/rxTwBHTt2TJ2dncrMzAxrz8zM1KFDhyLWX7NmjVavXt31uqCgQMuXL+/xaKmysjJWwx3SDh486HoIg0JVVZXrIQwKzMMJzMMJQ2EenF8FN3fuXM2ZM6frdegvlRkzZujjjz8OWzcQCKiyslL5+fkj/iKEgwcP6owzzhjxFyFUVVUpNzd3xF+EEG0eRtoRULR5GGlHQD3tFwNxBFRcXKz169f3ul7ME1BGRobi4+NVW1sb1l5bWxtxVCSd2AC9NsLGxsaoHyr19fUjOgGFeM3DSEpAIV7zYMtmm7DZ9mKZgEK85mEkJaAQr3kYSQkoJNp+MRAJqK+3RcT8IoTExESNHz9eO3fu7Grr7OzUzp07VVhYGOvuAABDVL+cgpszZ46effZZjR8/XhMmTNDatWvV2tqq6dOn90d3AIAhqF8S0CWXXKJjx46pvLxctbW1ys/P1w9/+EPPU3AAgJGp3y5CuOaaa3TNNddYxy9ZskRHjx4NawudL/7pT38acR544cKFxn0cO3bMamytra1WcbESOh8bDAb77Q7lvrA5l3zqqacax1RXV3u2h87rx8XFWZ/j785mPl1vDz3x+/1WcQN107hNiaqsrCzP9tBlx1lZWRHffdXU1JgPzpLN7zRt2jTjmN///vee7b3tFzbfUZl+39vX/ZFipAAAJ0hAAAAnSEAAACdIQAAAJ0hAAAAnSEAAACdIQAAAJ0hAAAAnSEAAACdIQAAAJ0hAAAAnSEAAACecPxE1mkcffVQfffRRWFsgENC8efN09913x+wBZDZiVfiyNzaFMW0KhObl5RnHSNLevXuNY7788kvjmGjzMFiKsvbHQ9W82BS5HMj95J/+6Z+MY1asWGEcYzN3ttv4vn37jGNstsUPP/zQOCbagzFD7cnJyZ7r2BTPNS1gSjFSAMCgRgICADhBAgIAOEECAgA4QQICADhBAgIAOEECAgA4QQICADhBAgIAOEECAgA4QQICADhBAgIAOEECAgA4MWirYZuyqUh8/Phxq75sqt2aVpOVolc/DlWajYuLi0ll7j179ljFpaSkGMfYzvlgZlNdeKB0dHRYxdm8tw8//LBVX7HS3t4uSaqpqYmoAn7s2LEBG0diovnHqk2F72hCFbBbW1tjtm2aVmLv6/ocAQEAnCABAQCcIAEBAJwgAQEAnCABAQCcIAEBAJwgAQEAnCABAQCcIAEBAJwgAQEAnCABAQCcIAEBAJwYtMVIg8FgRNHP0GuvZbEs5teb+HjzvG1TFNLn83m2p6end/3b/ec2NTUZ92NrIOd8oITm1sSTTz5pHPPpp58ax+zbt8+zPS0tTZJ01VVXqbm5OWzZ+vXrjfuRhl/RWNuirDZCRVFNLFmyxDjm3nvv9WwPFSiurKz0LJycnZ1t3FcgEDBaP9pnV3ccAQEAnCABAQCcIAEBAJwgAQEAnCABAQCcIAEBAJwgAQEAnCABAQCcIAEBAJwgAQEAnCABAQCcIAEBAJyIC3pVqxsErr32Wu3cuTOsze/365NPPtG5556rhoaGsGUHDx4csLGFiv2ZSEw0r/sardhnIBDQsWPHlJGRofr6euOfGys2RVlj2U8gENDRo0c1evToiHmwLT6ZmZlpHHPWWWcZxyxevNg45oorroi6bNSoUaqrq4to37Jli3E/klRSUmIcc+DAAeOYzs5O45ietofa2lplZmZGbA82/Uh2+7rrj9TePh9s9lvT+Zs6daq2bt3a63ocAQEAnCABAQCciPnzgMrLy7V69eqwtpycHK1cuTLWXQEAhrB+eSDdmWeeqUcffbTr9UB9VwAAGDr6JQHFx8dbfZkLABg5+iUBVVdX67vf/a6SkpJUWFiob33rW1EfA9vW1hZ2tVdcXJzS0tKUnp4uv98ftm7oMa9ej3s1fWTs1zEYroI7+V9XBsNVcCf/ezLbq+Bs5rSvjx8+mc32YCMhIcEqrvu+1xc2cxfrq+CijWOkXQV38r/dDcRVcH3dJ2J+GfZHH32klpYW5eTkqKamRqtXr9bRo0e1YsWKrmfXn6z7d0YFBQVavnx5LIcEABiE+v0+oMbGRi1YsEB33nmnZs6cGbE82hHQjTfeqIqKirB1fT6f/vCHP+jCCy9UY2Nj2LKqqqr++QU8DIYjoKqqKuXm5o74+4D27dunvLw8p/cBFRQUGMc8/PDDxjGXXHJJ1GXR7gPqy70YXu677z7jGJt78WJ9BHTgwAGdeeaZI/4+oJ4+HwbiCKi4uFgbNmzodb1+Pw/g8/mUk5Oj6upqz+VJSUlKSkqKaG9qaoq42TSksbExYtlAfhC7TkAh9fX1IzoBhXjNg20Csjll1f2Pob5ob283jrFhOw/R9r2e2GyLsUxAJ49jJCegkGifDwORgPq6T/T7J0hLS4uqq6u5KAEAECbmR0CrVq3ShRdeqOzsbNXU1Ki8vFzx8fG67LLLYt0VAGAIi3kCOnr0qP7lX/5F9fX1ysjIUFFRkUpLS5WRkRHrrgAAQ1jME9D3v//9mPycr776KuJ7o9BlhV9++aXT7z68vrPqje25eFNjxowxjrE9PTp37lzjmPLycuOYaF9uh85Ld3Z2Wp/j787mfbLZFhcuXGgcE+0sQlpamsrKyvTAAw+oubk5bJnN9iBJb731lnFMenq6ccwnn3xiHPPxxx97ticnJ0uSlixZouPHj4ct27Vrl3E/kvTqq68ax9h8J2gj9Pt2l5KS0vVv93mQev9u2UtqaqrR+qEx9IYSBQAAJ0hAAAAnSEAAACdIQAAAJ0hAAAAnSEAAACdIQAAAJ0hAAAAnSEAAACdIQAAAJ0hAAAAnSEAAACcG5sH0FoLBYMSDnUKvvZYNJK8Cf72xeSBdUVGRZ3voeeuFhYURhQ//4i/+wriff/u3fzOOsfWDH/zAOOaJJ57wbA8VSCwpKVFLS0vYso8++sh8cLIrpmtaqFGS/uEf/sE4ZtOmTZ7tfr9fkvT73/8+4kFyzz33nHE/knTOOecYx9g8vO2MM84wjrnmmmt6XO71Hh49etS4H+lE4WNT//M//2McY/N51tra6tkeKlLa2trquY7NA+m671+2Y4sYi/FIAACIARIQAMAJEhAAwAkSEADACRIQAMAJEhAAwAkSEADACRIQAMAJEhAAwAkSEADACRIQAMAJEhAAwAkSEADAibigy7LSPbj00ku1bdu2sLZAIKDq6mqddtppqq+vD1vW1NRk3MdZZ51lNbb9+/cbx7S1tRnH/Md//Idne2Jiom6++Wa98soram9vD1s2Z84c434CgYBxzEA6fPiwZ3tcXJxOOeUU/fnPf46oJrx582arvq666irjmISEBOOYCy64wDjm448/9mwPBAI6duyYMjIyIvaLKVOmGPcjSQUFBcYxf/mXf2kcc/XVVxvH5ObmRl2WnJzsWa3ethp298+gvrj22mut+oqVnrYHya5quWkF7alTp2rLli29/1zjkQAAEAMkIACAEyQgAIATJCAAgBMkIACAEyQgAIATJCAAgBMkIACAEyQgAIATJCAAgBMkIACAEyQgAIATia4HEE1zc3NEgdFQ0cempqaIZTYF9ryKFvaFTWHRvLw845hoBVaTkpIknZij7mNJS0sz7mcg2cxdtCKXPp9PH3zwga6//no1NjZGLLORnJxsHDNx4kTjmAMHDhjHRCsIGWqPj4+PWGf37t3G/UjSp59+ahyzdu1a45j//u//No6JVj85Li5OBQUFqqqqiljH7/cb9yNJ3/72t63iBjOb+tMdHR39sj5HQAAAJ0hAAAAnSEAAACdIQAAAJ0hAAAAnSEAAACdIQAAAJ0hAAAAnSEAAACdIQAAAJ0hAAAAnSEAAACcGbTFSUzYF9qqrq636sil8euTIEeOY3/3ud57tqampuuOOO7Rlyxa1tLSELbvrrruM+xlIqampxjGdnZ2e7YFAQJK0Y8cO1dfXf61xhVx33XXGMTbbQ3t7u3FMNKH56ezsjJir7ttHX51yyinGMbNnzzaOibaN9yRa8dfQ+5CcnBzxeWBTKFWSnn76aeOYv/u7vzOOsfn8sinsO9hwBAQAcIIEBABwwvgUXEVFhV577TXt3btXNTU1WrRokS6++OKu5cFgUOXl5Vq3bp0aGxtVVFSkkpISnX766TEdOABgaDM+AmptbVV+fr6+853veC7/1a9+pbfeekvz58/Xk08+qZSUFJWWllo//A0AMDwZJ6CpU6fq1ltvDTvqCQkGg1q7dq1uuOEGXXTRRcrLy9O9996rmpoabdmyJSYDBgAMDzG9Cu7w4cOqra3V5MmTu9rS09M1YcIE7d69W5deemlETFtbW9jVHHFxcUpLS5PP5+u6yikk9Lp7u63Qo61N2VzBlJ6ebhwT7YqxULvNFWWu2bx3vV0FF6vtQfr/x76bcH0VXE/zEO0x3r2xeYS1zePgU1JSjGOizXeo3Wu57b6emGj+EWmzPcbyKrj+2C9M+Xy+Pq0X0wRUW1srSRo1alRY+6hRo7qWdbdmzRqtXr2663VBQYGWL1+uDRs2RO2nqqrqa491OHjqqadcD8FYtO3g62B7OIF5OCEnJyei7bbbbhuw/m+++eYB66snQ2F7cH4f0Ny5czVnzpyu16G/Xi6//HJt3749bN1AIKCqqirl5ubG5L6PwX4EdNNNN3m2p6am6qmnntKDDz4YcZ/HM888Y9yPzV/wtjIzM41jejoCiuX2IA3dI6Bo82B7BJSdnW0cM2PGDOOY8ePHG8fccccdnu1xcXHKycnRoUOHIo4o3nvvPeN+pBP3FJm65557jGNifQQU6/3CVHFxcY8HESExTUChD5e6ujplZWV1tdfV1Sk/P98zJikpyTMRNDY2Rp28+vr6IZeAOjo6jGN6u4mwpaXF+kZDV2zet2gJ6OSfOZITUIjXPNgmIJvTu83NzcYxra2txjG9fVgHg8GIdWxv2rSZP5ttsT9uRI3lfmGqsbGxT+vF9D6gsWPHKjMzUzt27Ohqa2pq0hdffKHCwsJYdgUAGOKMj4BaWlrCStgcPnxYlZWV8vv9ys7O1nXXXadXX31Vp59+usaOHatf/vKXysrK0kUXXRTTgQMAhjbjBPTHP/5RS5cu7Xq9atUqSdKVV16phQsX6m/+5m/U2tqqn//852pqalJRUZF++MMfWp1LBQAMX8YJ6LzzzlN5eXnU5XFxcZo3b57mzZv3tQZmyuZcre15eJtLMy+44ALjmJ/97Gc9Ll+5cmVEW2/fl3ix+d5Dkr75zW8ax9iMbyDZfFc3mNl8tyBJr732mnHM+eefbxxjM77e3qMxY8ZEtNlcVCFJH374oXFMRkaGccxXX31lHGPL5rvv/ip8Si04AIATJCAAgBMkIACAEyQgAIATJCAAgBMkIACAEyQgAIATJCAAgBMkIACAEyQgAIATJCAAgBMkIACAEyQgAIATzh/JHU18fHxEhevQa69lA1ll2aZi8vHjx41j1qxZ49memJio66+/Xq+//npERe+5c+ca92NbiffNN9+0isPAmTZtmlXcpEmTjGNsKtI3NTUZx1x22WWe7T6fTx988IFmzZoV8UTOQ4cOGfcjSQsXLjSOWbRokXHMv/7rvxrHRPudevqclOwqW5u+t31dnyMgAIATJCAAgBMkIACAEyQgAIATJCAAgBMkIACAEyQgAIATJCAAgBMkIACAEyQgAIATJCAAgBMkIACAE4O2GGlCQoISE8OHF3qdmJgYsSwhIcG4j+bmZqux2RQ+vf/++41j8vPzPdtDhf7GjRsXMZa6ujrjfm688UbjGGlgC8DCzg9+8AOruOTkZOOYI0eOGMcUFRUZxzQ0NHi2BwIBSdKnn36q+vr6sGW22+q2bduMY2677TbjmL/6q78yjvn5z3/u2R76XTs7Oz1/77i4OOO+gsFgv6zPERAAwAkSEADACRIQAMAJEhAAwAkSEADACRIQAMAJEhAAwAkSEADACRIQAMAJEhAAwAkSEADACRIQAMCJQVuMtK2tTcePHw9rC70+fvx4xDKbAns2MZI0ZswY4xibAoU7d+70bA8VIw0EAhHFBm1+p48++sg4BgMv2nsbao+Li4tYJzU11aov0+KTkrRy5UrjmJqaGuOYaPPQ3t7e9W/o/1+XTVHWjRs3GseUlZUZx9iyeW9tPyt7wxEQAMAJEhAAwAkSEADACRIQAMAJEhAAwAkSEADACRIQAMAJEhAAwAkSEADACRIQAMAJEhAAwAkSEADAiUFbjNRUUlKScUxbW5tVX0ePHjWOyczMNI75xS9+4dmekpKi0tJSPf/882ptbQ1bZlMotXtBUwxOgUCgx3av5aNGjbLqq7Gx0Thm/fr1xjEpKSnGMd23+ZBQkc1gMGhVcNPL7NmzjWOuueYa45g333zTOGbv3r2e7QkJCV3/hv5/sm9+85vGfb3++utG63v164UjIACAEyQgAIATxqfgKioq9Nprr2nv3r2qqanRokWLdPHFF3ctf/bZZ/X++++HxRQXF+uRRx75+qMFAAwbxgmotbVV+fn5mjlzpn784x97rjNlyhQtWLDg/ztJHDZfNQEAYsQ4M0ydOlVTp07t+YcmJlp96Q4AGDn65dCkoqJCJSUl8vl8mjhxom699daoV/C0tbWFXY0WFxentLQ0+Xy+iJiervaxeXSu7VVwNo+njfb79yTaFUKhdq/lNvNgMzap/x7T21c9bQ/DUbTf0+/3h/17sr5ejRQL6enpxjE27120bbw/tgebq2tt9guv9643NldFSlJaWlrM+oqmr9tCXPBrXK94yy23RHwH9MEHHyglJUVjx45VdXW1Xn75ZaWmpqq0tFTx8ZHXPJSXl2v16tVdrwsKCrR8+XLbIQEAhoiYHwFdeumlXf8fN26c8vLydN9992nXrl2aNGlSxPpz587VnDlzul6H/nq4/PLLtX379rB1A4GAqqqqlJubq/r6+rBlg/0IyOaU5G233ebZnpKSosWLF2vZsmUR90SMHj3auJ8VK1YYx0hSQ0ODVVys9LQ9DEcZGRme7X6/X59++qmKiooi3pPy8nKrvs477zzjmJtvvtk45qOPPjKOiXYfUH9sD88884xxjM29Q3/9139tHFNZWenZHggEtH//fo0bN85zHq677jrjvt566y2j9SdPnqz33nuv1/X6/eqAU089VYFAQNXV1Z4JKCkpyfMwt7GxMepGVF9fP+QSkM2pkGg72snLu69z/Phx435sd1bXCSjEa3sYjnrb7hoaGiLmoaOjoz+HFKapqck4xuZ9622/iOX2YPMZYXNSyWZf6u13jDYPzc3NMe+ru75uC/1+H9BXX32lhoYGZWVl9XdXAIAhxPgIqKWlRdXV1V2vDx8+rMrKSvn9fvn9fr3yyiuaNm2aMjMz9eWXX+rFF1/UaaedpuLi4pgOHAAwtBknoD/+8Y9aunRp1+tVq1ZJkq688krNnz9f+/fv1/vvv6/GxkaNHj1akydP1rx586yuJgEADF/GCei8887r8YvNWFU8CAQCEYUUQ5cCjho1KuKKumPHjhn3YXsBoM0FBTbj+/Wvf+3Z7vP5VFpaqnXr1kUUjWxvbzfux+bcPb4em+8RJ06c6NkeuuT13HPPjXgvJ0yYYD44STU1NcYxV111lXHMtm3bjGOifS8T+kyIj4/3vOLWxsaNG41jSktLjWOqqqqMY6J9vxdq7+jo8FxnzZo1xn2Zbq99/e6RWnAAACdIQAAAJ0hAAAAnSEAAACdIQAAAJ0hAAAAnSEAAACdIQAAAJ0hAAAAnSEAAACdIQAAAJ0hAAAAnSEAAACf6/Ymoturr61VXVxfW1tnZKUmqq6tz+gTM7uPqC5vqx9EqW4fa29vbI9apra017mfhwoXGMZKUmGi++Tz77LPGMdHmoafqx2lpacb92MbZPHW0e6X3vpg1a5Zne0pKiiRp+vTpEU8Lff/99437kaQpU6YYx9g8ldimanVqamqP7ampqREVswsLC437kaI/9ronY8aMMY45dOiQcYwtm88i05i+rs8READACRIQAMAJEhAAwAkSEADACRIQAMAJEhAAwAkSEADACRIQAMAJEhAAwAkSEADACRIQAMAJEhAAwIlBW4w0IyNDWVlZYW2BQECSlJWVFVEI06YIp00hREmaNm2accymTZuMY3bv3u3ZHpqHL774IqIoa2iZCdtCje+9955xzL//+78bx3zyySee7aEinPfff39EEc5XX33VuB9Jqq6uNo7Jzc01jjl27JhxzK9//WvPdp/Pp0ceeUTr1q1TY2Nj2LK1a9ca9yNJBw4cMI4JvR8m7rzzTuOYgwcPeraHCsnOnDlTzc3NYcu+//3vG/cjSU8//bRxzPz5841j7rnnHuOYaNud3++XJOXl5amhoSFiuc17GwwGjWP6giMgAIATJCAAgBMkIACAEyQgAIATJCAAgBMkIACAEyQgAIATJCAAgBMkIACAEyQgAIATJCAAgBMkIACAE4O2GOmxY8dUU1MT1tbe3i5JqqmpiSjCaaOjo8MqzqawqI3Ozs4e2zs7OyPWmTNnjnE/+/btMx+cpGuuucY4pq2tzTjmww8/9GxPT0+XJG3btk1NTU1hy6IVrOxN9yK3fWFTCDcnJ8c4ZteuXZ7toQK0n3zyScR+ccEFFxj3I0l1dXXGMd2LB/fF5ZdfbhwzadIkz/ZQceHHHnssYr+wLUZqUyz1wQcfNI6ZPn26ccwbb7zh2R7aHvbt2+f5OZmZmWncl+lnZagwbG84AgIAOEECAgA4QQICADhBAgIAOEECAgA4QQICADhBAgIAOEECAgA4QQICADhBAgIAOEECAgA4QQICADgxaIuRxsXFKS4uLqIt2jIbwWDwa/+Mweall14yjhk7dqxVX5MnTzaOaWhoMI7ZsWOHZ3uo6OLvfve7iKKLo0ePNu5HkhISEgYkJiUlxTgmWvHJUP/l5eURRSMPHTpk3I+kiOKuffHCCy8Yx2RkZBjHHD9+3LM99JmQnJwcsW/bjE2S7rrrLuOYw4cPG8esXbvWOMaWzb6xZ88eo/UbGxv7tB5HQAAAJ0hAAAAnjE7BrVmzRps3b1ZVVZWSk5NVWFio22+/PezZJsePH9eqVau0adMmtbW1qbi4WCUlJVbPoAAADF9GR0AVFRW6+uqrVVpaqn/8x39UR0eHnnjiCbW0tHSt88ILL+jDDz/UAw88oKVLl6qmpkYrVqyI+cABAEObUQJ65JFHNH36dJ155pnKz8/XwoULdeTIka4vqJqamvTb3/5Wd955pyZOnKjx48drwYIF+uyzz7R79+5++QUAAEPT17oKLnSljN/vl3TiSomOjo6wR+bm5uYqOztbu3fvVmFhYcTPaGtrC3tMc1xcnNLS0uTz+bqucgoJve7ebmuoXgUX63kIvX+mQo/ENhHtMeM9ifZ7hsbtNX7b32mgroKzmbto/YTavZbbPGJcknw+n3FM6JHYJmyuZo0Wc/JVsrHoR+r7o6VPZrNf2uwX0WJ6+3yw2TdMf6e+bj/WCaizs1PPP/+8zjnnHI0bN06SVFtbq8TExIjOR40apdraWs+fs2bNGq1evbrrdUFBgZYvX67169dH7fvgwYO2wx5WqqqqXA9hUPj8889dD2FQuOSSS2L2s2655ZaY/ayBlp+fH7Of9V//9V8x+1kDbSh8PlgnoLKyMh04cEDLli37WgOYO3eu5syZ0/U69JfKFVdcoe3bt4etGwgEdPDgQZ1xxhkR933YGMpHQFVVVcrNzY3JPJxyyilWcRMnTjSOsbkPqKKiwrPd7/fr888/19lnnx3xc7Oysoz7kQbuCMjm3qtHH300av+XXHKJNm3aFHEf0J/+9CfjfiTppz/9qXHMz372M+MYm6OFnu4Dys/PV2VlZcS+bXPEKUkLFiwwjnn33XeNY2J9BNTT50NBQYFxX3v37jVav7i4WBs2bOh1PasEVFZWpq1bt2rp0qUaM2ZMV3tmZqba29vV2NgYdhRUV1cX9Sq4pKQkJSUlRbQ3NjZG/XCtr68f0QkoJFbzYHOaQbK7WdEmprffsaGhIWIdr22qLwYqAdmcBumeXLyWd1+nvb3duB+p7zcSnszmQ9RmH+wtJhgMRqxju683Nzcbx9jsk7FMQCePw2ssNn8Emv5O/XIjajAYVFlZmTZv3qzFixdH/BU3fvx4JSQkhN25fujQIR05csTz+x8AwMhldARUVlamjRs36qGHHlJaWlrX9zrp6elKTk5Wenq6Zs6cqVWrVsnv9ys9PV3PPfecCgsLSUAAgDBGCeg3v/mNJGnJkiVh7QsWLND06dMlSXfeeafi4uK0YsUKtbe3d92ICgDAyYwSUHl5ea/rJCcnq6Sk5GsnnZ7O43ots2F7aaZN3zaXqNqcF7bx5z//2Squ+0UifXHyd4Z9Fa14Yuh7lKysrIjvfPLy8oz7kaTvfe97xjEzZ840jrH58r2374AuuuiiiLYjR44Y9yOduDjIVGpqqnHMBx98YBwT7QKO0D7W0dERse/Y7ut9+SK9O9vv3QaKaWFR6cTnuom+fgdLLTgAgBMkIACAEyQgAIATJCAAgBMkIACAEyQgAIATJCAAgBMkIACAEyQgAIATJCAAgBMkIACAEyQgAIATJCAAgBPWj+QebGyr3Q6UgapsPZBWrlxpHGPzpNIpU6Z4toeqH7/++usR82tTfVySTj/9dOMY2yfKmkpM7Hl3TUlJiWizeQKtZFch3aYa9pNPPmkcM3v2bM/25ORk3XvvvXr77bcjHtv993//98b9SPaP8jZlM3ctLS1Wfdl8VkZ7DHo0bW1tfVqPIyAAgBMkIACAEyQgAIATJCAAgBMkIACAEyQgAIATJCAAgBMkIACAEyQgAIATJCAAgBMkIACAEyQgAIATg7YYaXJyckRxxdDrlJSUiOJ4gUDAuI8jR47YD3AARCsaGGqPi4uLWCcYDBr3Y1MgVJIef/xx45hrr73WOCZaocbExESdddZZ2r17t9rb28OWzZgxw7gfybugZ29sijvavE+99eO1fMmSJcb9SFJ1dbVxzGeffWYcU1BQYBzz2GOPebYHAgHde++9evLJJ1VfXx+2LDc317gfSfL7/cYxDQ0NxjE2hUWjFacNtScmJnqu01tRWy+mxUj7WgyYIyAAgBMkIACAEyQgAIATJCAAgBMkIACAEyQgAIATJCAAgBMkIACAEyQgAIATJCAAgBMkIACAEyQgAIATg7YYaUdHR0SBydDr9vb2iGU2hUX7WjCvu87OTuMYm4KfbW1tnu2hQpbBYNCqqGV3poUGQz7//HPjmNraWuOYV1991bPd7/fr2muv1YMPPhhRANK2CGdxcbFxjE2hy7q6OuOYO+64w7M9PT1d69at06xZs9TU1BS2bMuWLcb9SJLP5zOOsdmOohWa7Um0/TbUHh8fH7HO/v37jfuR7Pb1rKws45iamhrjGNtipPfcc49xX//8z/9stH5f540jIACAEyQgAIATJCAAgBMkIACAEyQgAIATJCAAgBMkIACAEyQgAIATJCAAgBMkIACAEyQgAIATJCAAgBODuhhpR0dHRFu0ZTZsCg3ashlvSkpKj+0pKSkRBSBtCkLaFjQdqPn7xje+4dmelpYmSTrnnHPU3Nwctuyuu+6y6uvWW281jrnooouMYx5//HHjmMbGRs/2QCAgSdq+fbvq6+uNf66XlpYW45gxY8YYxxw4cMA4JppQ4c3m5uaI7SEzM9PqZ9oUCbWJiYuLM46xKcoqST/5yU+M++ovHAEBAJwgAQEAnDA6BbdmzRpt3rxZVVVVSk5OVmFhoW6//Xbl5OR0rbNkyRJVVFSExc2ePVt33313bEYMABgWjBJQRUWFrr76ap111lnq6OjQyy+/rCeeeEJPP/102IOlZs2apXnz5nW9Tk5Ojt2IAQDDglECeuSRR8JeL1y4UCUlJdqzZ0/YF8UpKSnWX/oBAEaGr3UVXOjxv36/P6x9w4YN2rBhgzIzM3XBBRfoxhtvjHpFV1tbW9ijp+Pi4pSWliafz9d1dU9I6HX39qHA5vHf0R7jHZrv7vMuDexVcDZX7niNuTehq92itXstt91GovXVE5sjfJvxRduGetoebK8WTUhIMI6xeW9bW1uNY6Lp6fPBdntob283jrHZn2z2pWjbak/bg2T3GWE6D319pHtc0PLTp7OzUz/60Y/U2NgYdknpO++8o+zsbI0ePVr79u3TSy+9pAkTJmjRokWeP6e8vFyrV6/uel1QUKDly5fbDAkAMIRYJ6Bf/OIX2rZtm5YtW9bj9f87d+7UsmXL9Mwzz+i0006LWB7tCOjyyy/X9u3bw9YNBAKqqqpSbm5uzO53GCixPgLau3evCgoK1NDQELZssB8BjR071jhm8uTJnu1paWl68cUXdfvtt0fc9/HOO+8Y9yNJN910k3HM+eefbxzzox/9yDgm2n1Afr9flZWVys/Pj9geBvIIaPTo0cYxX331lXFMNIFAoGseun8+2B4B2dzTMxiOgL744gtNmDAhYnuQBuYIqLi4WBs2bOh1PatTcGVlZdq6dauWLl3a681nEyZMkCRVV1d7JqCkpCTPD9rGxsaoSaa+vn5EJ6CQhoaGiHkY7AkoPT3dOKZ7cvFa3n0d2+2jt7682My5zfiiJaAQr+1hIBOQzanI/tiPY/n5YPNzBioB9ZYUvLYHaWASUG/baojRp2IwGFRZWZk2b96sxYsX9+mv2crKSklSVlaWSVcAgGHO6AiorKxMGzdu1EMPPaS0tDTV1tZKOvFXbXJysqqrq7Vx40adf/758vv92r9/v1544QWde+65ysvL64/xAwCGKKME9Jvf/EbSiZtNT7ZgwQJNnz5diYmJ2rFjh9auXavW1laNGTNG06ZN0w033BCzAQMAhgejBFReXt7j8uzsbC1duvRrDQgAMDIM2mrYA8Hmng/J7ku8qVOnGsd0vwowJFSFurOzM6Iitc3FDhMnTjSOkaKPryfV1dXGMevWrfNsD13Z9P7770d82drX+xC627Fjh3HMybcR9NWpp55qHBPtiqzQ9nj8+PGIbfPkCiUmbLbxP/3pT8YxvV1o4yXaBRKh9oSEhIh1bC6qGEg2Fy5MmzbNsz10oc+FF17Yda/myd59913jvkwvMOnr+0oxUgCAEyQgAIATJCAAgBMkIACAEyQgAIATJCAAgBMkIACAEyQgAIATJCAAgBMkIACAEyQgAIATJCAAgBODthhpfHx8RGHN0GuvZd2LcvaFzdMvbf3hD3+I2c8KPcK8++PMJbsnK44fP95qHDbFSG2KpUYrjNlTEc7u89JXOTk5xjG7du0yjjl48KBxTEpKSo/tKSkpEfPQ2tpq3I9ktx3ZFPy0eZ+ixYQKYLa0tKilpcX457pkM9/RioqGivSuX78+Zk+GNX2f+voEVY6AAABOkIAAAE6QgAAATpCAAABOkIAAAE6QgAAATpCAAABOkIAAAE6QgAAATpCAAABOkIAAAE4M2lpw55xzTkSbz+eTJBUXF6uxsTFsmU0tuKGqp3kYyFpwU6dONY6xqQUX7b2N9TxI0oQJE4xjjhw5YhwTDAaNY5KTkz3bQ/MwefLkiHmwrYlnM382MR0dHcYx0fS0PUSro9ebaHUIe2Lz3trMXbR+epoHW6bjKyoq6tvPDdrMFgAAX9OQOgXX3Nyshx9+eECrWA9GzMMJzMMJzMMJzMMJQ2kehlQCCgaD2rt3r9Uh7nDCPJzAPJzAPJzAPJwwlOZhSCUgAMDwQQICADgxpBJQUlKSbrrppq4nH45UzMMJzMMJzMMJzMMJQ2keuAoOAODEkDoCAgAMHyQgAIATJCAAgBMkIACAE4O2Flx3b7/9tl5//XXV1tYqLy9Pd911l1XdrqGsvLxcq1evDmvLycnRypUr3QxogFRUVOi1117T3r17VVNTo0WLFuniiy/uWh4MBlVeXq5169apsbFRRUVFKikp0emnn+5w1LHX2zw8++yzev/998NiiouL9cgjjwz0UPvNmjVrtHnzZlVVVSk5OVmFhYW6/fbblZOT07XO8ePHtWrVKm3atEltbW0qLi5WSUmJMjMz3Q08xvoyD0uWLFFFRUVY3OzZs3X33XcP9HCjGhIJaNOmTVq1apXmz5+vs88+W2+++aZKS0u1cuVKjRo1yvXwBtSZZ56pRx99tOu1TXHPoaa1tVX5+fmaOXOmfvzjH0cs/9WvfqW33npLCxcu1NixY/Wf//mfKi0t1dNPPx21gOdQ1Ns8SNKUKVO0YMGCrteJiUNiF++ziooKXX311TrrrLPU0dGhl19+WU888YSefvpppaamSpJeeOEFbd26VQ888IDS09NVVlamFStW6PHHH3c8+tjpyzxI0qxZszRv3ryu14NtfxgSW+cbb7yhWbNmacaMGZKk+fPna+vWrXr33Xd1/fXXux3cAIuPjx9Wf8n1xdSpU6NW3g4Gg1q7dq1uuOEGXXTRRZKke++9V/Pnz9eWLVt06aWXDuRQ+1VP8xCSmJg4rLeP7kdzCxcuVElJifbs2aNvfOMbampq0m9/+1t973vf08SJEyVJCxYs0P3336/du3ersLDQxbBjrrd5CElJSRnU28OgT0Dt7e3as2dPWKKJj4/XpEmTtHv3bncDc6S6ulrf/e53lZSUpMLCQn3rW99Sdna262E5c/jwYdXW1mry5Mldbenp6ZowYYJ27949rBJQX1RUVKikpEQ+n08TJ07UrbfeqkAg4HpY/aapqUmS5Pf7JUl79uxRR0eHJk2a1LVObm6usrOzh1UC6q77PIRs2LBBGzZsUGZmpi644ALdeOON1o+m6A+DPgEdO3ZMnZ2dEVk8MzNThw4dcjMoR84++2wtWLBAOTk5qqmp0erVq7V48WKtWLFCaWlprofnRG1trSRFnIodNWpU17KRYsqUKZo2bZrGjh2r6upqvfzyy3ryySdVWlo6LE/VdnZ26vnnn9c555yjcePGSTqxPSQmJnY9EydkOG8PXvMgSZdddpmys7M1evRo7du3Ty+99JIOHTqkRYsWORxtuEGfgPD/Tj79kpeX15WQ/vd//1czZ850ODIMBicf7Y0bN055eXm67777tGvXrrAjguGirKxMBw4c0LJly1wPxalo8zB79uyu/48bN05ZWVlatmyZqqurddpppw30MD0N+j+LMjIyFB8fH/HXS21t7aA+tzkQfD6fcnJyVF1d7XoozoS2gbq6urD2urq6Eb99nHrqqQoEAsNy+ygrK9PWrVv12GOPacyYMV3tmZmZam9vj3gS6HDdHqLNg5fQVcODaXsY9AkoMTFR48eP186dO7vaOjs7tXPnzmF7PrevWlpaVF1dPSx3rL4aO3asMjMztWPHjq62pqYmffHFFyN++/jqq6/U0NCgrKws10OJmWAwqLKyMm3evFmLFy/W2LFjw5aPHz9eCQkJYdvDoUOHdOTIkWG1PfQ2D14qKyslaVBtD0PiFNycOXP07LPPavz48ZowYYLWrl2r1tZWTZ8+3fXQBtSqVat04YUXKjs7WzU1NSovL1d8fLwuu+wy10PrV6FEG3L48GFVVlbK7/crOztb1113nV599VWdfvrpGjt2rH75y18qKyur66q44aKnefD7/XrllVc0bdo0ZWZm6ssvv9SLL76o0047TcXFxQ5HHVtlZWXauHGjHnroIaWlpXWdGUlPT1dycrLS09M1c+ZMrVq1Sn6/X+np6XruuedUWFg4rBJQb/NQXV2tjRs36vzzz5ff79f+/fv1wgsv6Nxzz1VeXp7bwZ9kyFTDfvvtt/Xaa6+ptrZW+fn5+va3v62zzz7b9bAG1MqVK/XJJ5+ovr5eGRkZKioq0q233jpozuf2l127dmnp0qUR7VdeeaUWLlzYdSPqO++8o6amJhUVFek73/lO2E15w0FP8zB//nw99dRT2rt3rxobGzV69GhNnjxZ8+bNG1ZHyLfccotn+4IFC7r+IA3diPrBBx+ovb19WN6I2ts8HDlyRD/5yU904MABtba2asyYMbr44ot1ww03KD09fYBHG92QSUAAgOFl0H8HBAAYnkhAAAAnSEAAACdIQAAAJ0hAAAAnSEAAACdIQAAAJ0hAAAAnSEAAACdIQAAAJ0hAAAAnSEAAACf+DzkSdNYOVhTnAAAAAElFTkSuQmCC\n"
          },
          "metadata": {}
        },
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 640x480 with 1 Axes>"
            ],
            "image/png": "iVBORw0KGgoAAAANSUhEUgAAAaAAAAGdCAYAAABU0qcqAAAAOnRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjEwLjAsIGh0dHBzOi8vbWF0cGxvdGxpYi5vcmcvlHJYcgAAAAlwSFlzAAAPYQAAD2EBqD+naQAAKUpJREFUeJzt3X9w1PWdx/HXbn6QZLNhA2mURElAjFHBkKjgiZwUcLQenRbtFMZjRqrQH6G9znmMN1OrJ1Q6Q1v8cR3G6zjpaQaLjam0ain1qq0T/FG0QQqmlpOAP8KlHJKQsPlBfuz9kdk9w+4m+XzZ3c/u5vn4R/fz/b73++GT735f+9397ufrCgQCAQEAkGBu2x0AAExOBBAAwAoCCABgBQEEALCCAAIAWEEAAQCsIIAAAFYQQAAAKwggAIAVBBAAwIpM2x2I5o477tB77703qs3j8aipqUmLFy+W3++31DNncnJyjGv6+voitqfyOMRSPMbB5XIZ1ziZzSqW24nHOLjd5u9Nh4eHY7Jtp5LldWF77MYbh8xM88P+/fffb7T+1KlT9fd///fjrhe3ANqzZ49eeOEFdXZ2qqysTHfddZfmzJkz4fr33ntP+/fvH9Xm9XolSQcOHFB3d3dM+xtvubm5xjW9vb0R21N5HGIpHuOQigEUj3GwfRB1IlleF7bHbrxxyMrKMn7OU6dOnXe/IonLR3Cvv/666uvr9aUvfUlbt25VWVmZtmzZotOnT8djcwCAFBSXAHrxxRe1bNkyffazn9VFF12k9evXKzs7W7///e/jsTkAQAqK+Udwg4ODam1t1Re/+MVQm9vt1rx583T48OGw9QcGBjQwMBB67HK5lJubK4/HEzqVDAo+Prc9FTj5CC7aZ7WpPA6xFI9xSOWP4GI5DrY/RnIiWV4XtsduvHFw8hGcac1Ev2eKeQB1dXVpeHhYPp9vVLvP59Px48fD1t+1a5caGxtDj2fNmqWtW7eqqakp6jba2tpi1t9UxjiMYBxGMA4jGIcRqTAO1q+CW7lypVasWBF6HHxnuHjxYh04cGDUul6vV21tbSotLU25L99jfRFCqo5DLMVjHFL1DCjW42D7XbwTyfK6sD12442DkzOgH//4x0brFxYW6nOf+9y468U8gAoKCuR2u9XZ2TmqvbOzM+ysSBoZjEgD4vf7o+5E3d3dKXfgHRwcNK6JFkBBqTgO8RDLcUjFAAqK5TjYPoieD9uvi2QZu2jj4CSAPv01yURM9HgX84sQMjMzNXv2bB06dCjUNjw8rEOHDqmioiLWmwMApKi4fAS3YsUKbd++XbNnz9acOXO0e/du9ff3a8mSJfHYHAAgBcUlgK6//np1dXWpoaFBnZ2dKi8v13e+852IH8EBACanuF2EcMstt+iWW26J19PHhJPP4Z06e/ascU20/gXbXS5X2DpOvo/A/3MyfsnymX8sOelfor4/w/kx/T5HktauXWu0fnV1tT7/+c+Pux6TkQIArCCAAABWEEAAACsIIACAFQQQAMAKAggAYAUBBACwggACAFhBAAEArCCAAABWEEAAACsIIACAFdbviGpTIidCdDK5Y7T+BdsDgQCTOSaBZJ5YNCMjw1Hd0NCQcU2ibsyXn58fsd3r9Y7676edOXPGeDsSk6UGmd7ROScnZ0LrcQYEALCCAAIAWEEAAQCsIIAAAFYQQAAAKwggAIAVBBAAwAoCCABgBQEEALCCAAIAWEEAAQCsIIAAAFYQQAAAK5J2NmyXyxU2U27w8VjLTCRyFmMns+q+9dZbEdvd7pH3DX/4wx/C/g0LFixISN+ccvJ3YkZi55zMap1ITv623d3d4y4/d53ga8aUk/4l8+zokrPXYG9vr9H6fX19E1qPMyAAgBUEEADACgIIAGAFAQQAsIIAAgBYQQABAKwggAAAVhBAAAArCCAAgBUEEADACgIIAGAFAQQAsCJpJyMNBAJhEwEGH0dalo6uu+66iO1er1cdHR1atmxZ2KSLmZnmf9KBgQFH/XPCyd8t2uSJY01O65TX6zWucTL55AUXXGBcc+TIEeMajHA6QWhGRoZxzfHjx41rnOwPjz/+eMT2rKwsSdIjjzwS8bX9jW98w3hb8cIZEADACgIIAGAFAQQAsIIAAgBYQQABAKwggAAAVhBAAAArCCAAgBUEEADACgIIAGAFAQQAsIIAAgBYkbSTkZpK5OSkbrd5bjuZDHFoaGjM9qGhobB1YjnZ53icTHzqZFs5OTkR24MTh0aaQPT666833o4k/fKXvzSucfJvcjJ2f/7znyO2B/fHvXv3hu1n77//vvF2JOnpp582runt7TWu+cIXvmBc84Mf/CBie35+viSprKxMZ86cGbWsuLjYeDuS9PbbbxvXfP3rXzeucXJMqa2tjdju9Xp1991365577gmbrFhytr/G6/jKGRAAwAoCCABgRcw/gmtoaFBjY+OotpKSEj366KOx3hQAIIXF5Tugiy++WPfff3/osZPPNwEA6S0uAeR2u+Xz+eLx1ACANBGXAGpvb9fXvvY1ZWVlqaKiQnfccYeKiooirjswMDDqtrEul0u5ubnyeDxhVzeNddVTIiXqKrhoxhoHJ31zeoWL7avgglc9Bf/7aXl5ecbbSXbR/rbB9kjLnfyNJCk3N9e4xsnfNnj7aBOR/t6S5PF4Rv030jJTTo41TsbOyXaivW7jcZw0PUZMdLxdgRhfX7d//3719fWppKREHR0damxs1KlTp7Rt27aIf5hzvzOaNWuWtm7dGssuAQCSUMwD6Fx+v1+1tbW68847tXTp0rDl0c6AFi9erAMHDoxa1+v1qq2tTaWlpRGvb0+UZDgDijYOk+0M6L333lNlZWXY7z4WLlxovB1J+tnPfmZck6jfAR06dChiu9vt1ty5c3Xo0KGw/ay1tdV4O9LIG0NTfX19xjW33nqrcc1jjz0Wsd3j8ejNN9/UddddJ7/fP2rZZz7zGePtSFJzc7NxzT/8wz8Y1/z61782rhnrDOjjjz/WRRddFLPjpOkxoqqqSk1NTeOuF/cfono8HpWUlKi9vT3i8qysrIin4X6/P+rgdXd3T+oACoo0DukYQJ9+gxLJmTNnwsahp6fHeDvJbrx9aHh4OGydwcFBR9ty8qNSJzXj/W0jOffNxrn8fn/YOk4/knVynHEyDk62M97rNpbHSdNjxLlvAKKJ++VpfX19am9v56IEAMAoMT8Dqq+v1zXXXKOioiJ1dHSooaFBbrdbN9xwQ6w3BQBIYTEPoFOnTumxxx5Td3e3CgoKVFlZqS1btqigoCDWmwIApLC4X4TgVE1Njfbv3z+qzev1qqurSwUFBVa/A7It1uPg5FJYydl3R1VVVcY1Y32ZmZubG/Ez9+zsbOPtOOXkezenE8AmMyf7wwsvvGBcM3v27IjtbrdbV1xxhVpaWsK+C+vv7zfejiRt2rTJuGb37t3GNXV1dcY1a9eujdg+3vEhEd9hV1dXT+gCDqYoAABYQQABAKwggAAAVhBAAAArCCAAgBUEEADACgIIAGAFAQQAsIIAAgBYQQABAKwggAAAVhBAAAAr4n5DOiQ/J5MTOq174oknjGtefPHFiO2ZmZlauXKl9uzZE3bjtZUrVxpvR5L++7//27jmk08+Ma5ZtGiRcU0iJzB1MrGokwk/L774YuOaaDeky8jIkDRyM7ShoaFRy5xO2hvLO5WOZfPmzcY10V5/wXa32x1xnXjcHNMpzoAAAFYQQAAAKwggAIAVBBAAwAoCCABgBQEEALCCAAIAWEEAAQCsIIAAAFYQQAAAKwggAIAVBBAAwAoCCABgBbNhp5nMTPM/6dmzZx1tq6SkxLjmlltuMa45ePBgxPbg7NA33HBD2AzETmf4dvJvclLjZOboKVOmRF3mcrkizsLsdAbtxx57zLjmu9/9rnFNb2+vcU20fdzr9erkyZP63Oc+Fzb7tdN93Amfz2dc09raalyzbdu2iO3Z2dmSpIceeijiv/tf/uVfjLcVL5wBAQCsIIAAAFYQQAAAKwggAIAVBBAAwAoCCABgBQEEALCCAAIAWEEAAQCsIIAAAFYQQAAAKwggAIAVrkCkGQyTQE1Njfbv3z+qzev1qqurSwUFBWGTDU4mY41DYWGh8fPNnz/fUT+mTZtmXLNjxw7jmkROwjk8PGxc42Siy5ycHOMaJ5y+vLOysoxrhoaGjGuc/J2i/Zs4PoxIhnGorq5Wc3PzuOtxBgQAsIIAAgBYQQABAKwggAAAVhBAAAArCCAAgBUEEADACgIIAGAFAQQAsIIAAgBYQQABAKwggAAAVmTa7gCiizYhZLA9KysrbJ2uri7j7Xi9XvPOSfrP//xP45p4TMLpdOLRSNxu8/dk2dnZMdt+rL355pu2uzCmJJ0LOa3FcgLY88UZEADACgIIAGCF8UdwLS0tev7553X06FF1dHRo48aNWrBgQWh5IBBQQ0ODXn75Zfn9flVWVmrdunWaMWNGTDsOAEhtxmdA/f39Ki8v19133x1x+a9+9Sv95je/0fr16/X9739fU6ZM0ZYtWxzdtAsAkL6MA6i6ulqrV68eddYTFAgEtHv3bt1222269tprVVZWpm9+85vq6OjQW2+9FZMOAwDSQ0yvgjtx4oQ6Ozt11VVXhdry8vI0Z84cHT58WIsWLQqrGRgY0MDAQOixy+VSbm6uPB5P2NVZwcdOr9pKNdGughtrHJzcUjo3N9e4BqkhIyPDUZ2T15iTW3LH0mQ7PkQz3jgk4io4j8czofViGkCdnZ2SpKlTp45qnzp1amjZuXbt2qXGxsbQ41mzZmnr1q1qamqKup22trbz7ms6OHbsmO0uQM4u3U6USJ9UTERHR0eMe5I4HB9GpMI4WP8d0MqVK7VixYrQ42A6L168WAcOHBi1rtfrVVtbm0pLS9Xd3Z3Qftow1hnQsWPHVF5eHjYOTs6Abr75Zkf9e+KJJ4xr0vHdqZMxT1Ro7du3z1Gdk30iGc6AJtPxIZrxxiERZ0BVVVVjnkQExTSAfD6fJOn06dMqLCwMtZ8+fVrl5eURayL9mFKS/H5/1J2ou7t7Uuxg0QIoKNI4ODkY9vb2GtcgNTgNBSevL9sBFDRZjg/jiTYOiQggv98/ofVi+jasuLhYPp9PBw8eDLX19PTo/fffV0VFRSw3BQBIccZnQH19fWpvbw89PnHihI4dO6b8/HwVFRXp1ltv1XPPPacZM2aouLhYzzzzjAoLC3XttdfGtOMAgNRmHEBHjhzRpk2bQo/r6+slSTfeeKM2bNigL3zhC+rv79dPfvIT9fT0qLKyUt/5zneSer4sAEDiGQfQlVdeqYaGhqjLXS6XVq1apVWrVp1Xx6BRl6dHaj/3EnbJ2WSfzz33nHnnlNxXfzn5LsypRE2oOda/ye12R1y+du1aR9tKlu9zEHvJNAFs8h5BAABpjQACAFhBAAEArCCAAABWEEAAACsIIACAFQQQAMAKAggAYAUBBACwggACAFhBAAEArCCAAABWEEAAACus35LbpoyMDEd1iZopONqdC4PtLpcrbJ38/Hzj7bz88svmnZNUU1NjXDN9+nTjGid3cHTKyQzfJ0+eNK6ZOnWqcc2TTz4ZsT0rK0tr165VfX192Ozo//M//2O8HaQGJ8cHidmwAQAggAAAdhBAAAArCCAAgBUEEADACgIIAGAFAQQAsIIAAgBYQQABAKwggAAAVhBAAAArCCAAgBVpMxnp3LlzjWsOHToUh57ETrRJA4PtgUAgbJ3Ozk7j7Tz99NPGNZKziTvnz59vXDPWBKYulyviOPn9fuPtSNJNN91kXDNr1izjmh07dhjXrF27dszl//iP/xjWdvXVVxtvR5Luuusu45oDBw442hacvZaiYTJSAADGQQABAKwggAAAVhBAAAArCCAAgBUEEADACgIIAGAFAQQAsIIAAgBYQQABAKwggAAAVhBAAAAr0mYy0mSfWDRRBgcHjWvq6+sdbeu3v/2tcY2TSTiXL18+5vJIEy56vV7j7UjSG2+8YVzT1tZmXJORkRHzmqysrLC2mpoa4+1I0jvvvGNc42RS1mPHjhnXpKNI+/B4hoaGIrYPDw+H/hv8/2TFGRAAwAoCCABgBQEEALCCAAIAWEEAAQCsIIAAAFYQQAAAKwggAIAVBBAAwAoCCABgBQEEALCCAAIAWJE2k5Ei8U6cOGFc84tf/MK4ZrzJSG2bMWOGcY2TSWMzM5P75Xr06FHjmkgTqI7HydglOyeThkabnDbYnpGREXGdaJOY2sAZEADACgIIAGCF8Tl9S0uLnn/+eR09elQdHR3auHGjFixYEFq+fft2vfrqq6NqqqqqdN99951/bwEAacM4gPr7+1VeXq6lS5fqRz/6UcR15s+fr9ra2v/fSJJ/dg0ASDzjZKiurlZ1dfXYT5qZKZ/P57RPAIBJIC6nJi0tLVq3bp08Ho/mzp2r1atXR71F8sDAgAYGBkKPXS6XcnNz5fF4wmqCj53ebjldJMs4OLmNcE5OThx6glTlZB+OdhVcsrwunHDyWnK7I3+FP944JOIqOI/HM6H1XIFAIOB0I1/+8pfDvgN67bXXNGXKFBUXF6u9vV07d+5UTk6OtmzZEnHAGhoa1NjYGHo8a9Ysbd261WmXAAApIuZnQIsWLQr9/8yZM1VWVqZvfetbevfddzVv3ryw9VeuXKkVK1aEHgffCSxevFgHDhwYta7X61VbW5tKS0vV3d0d666njGQZByfv2u666y7jmkceecS4xikn78ec/IbDyXbS8bvUadOmGdeMdQaUDK8LJ2J9BvThhx9q5syZEcchEWdAVVVVampqGne9uO/RF1xwgbxer9rb2yMGUFZWVsQfo/n9/qg7UXd3d8rtYPFgexycvGj6+vri0BOkKif773g/RLX9unAilgEUFG0cEhFAfr9/QuvF/XdAn3zyic6cOaPCwsJ4bwoAkEKMz4D6+vrU3t4eenzixAkdO3ZM+fn5ys/P17PPPquFCxfK5/Ppb3/7m3bs2KELL7xQVVVVMe04ACC1GQfQkSNHtGnTptDj+vp6SdKNN96o9evX68MPP9Srr74qv9+vadOm6aqrrtKqVasczfkEAEhfxgF05ZVXqqGhIepyZjyYPJx8kf7Xv/7VuKanpyfqsry8vDGXm/rTn/5kXLNz507jmuuuu8645vrrr4/Y7nK5dMkll+jIkSNhf5PS0lLj7UhSbm6uozpTTr4TjNa34JvcSN8rf/qnHsno85//vHHNnj17IraPNxlpoi6amQjmggMAWEEAAQCsIIAAAFYQQAAAKwggAIAVBBAAwAoCCABgBQEEALCCAAIAWEEAAQCsIIAAAFYQQAAAKwggAIAVaXOPXye3e0j2GXKTnZO7OP7xj380rrnxxhsjtns8Hv3hD3/QrbfeGnYHxr/85S/G25Gk3t5e4xonsws//vjjxjXReL1edXV1qbq6OuwOmPPnz3f0nPv3749Bz8Y33l09I1m6dGnE9ry8PEkj+8u5M6RfccUV5p2TtGPHDuOaU6dOGdc4uYPr2bNnx2w/e/Zs1HWSBWdAAAArCCAAgBUEEADACgIIAGAFAQQAsIIAAgBYQQABAKwggAAAVhBAAAArCCAAgBUEEADACgIIAGBF2kxGmsiJRZ1MwhkIBOLQE7suv/xy45o33njDuCYnJ2fM5S+99FJY27e//W3j7UjSf/zHfziqS1bvv/++o7q6ujrjmq985SvGNZ2dncY19fX1EduDr8uf/OQnYa+34ESlplavXm1cs2TJEuOa1157zbgmHXAGBACwggACAFhBAAEArCCAAABWEEAAACsIIACAFQQQAMAKAggAYAUBBACwggACAFhBAAEArCCAAABWpM1kpE643c7yd3h4OMY9sSs7O9tR3caNG41rCgoKHG1rLJH6//jjjzt6rjfffNO45s9//rNxTaL2IaeT9P785z83rlm2bJlxzalTp4xrKioqxlyem5sb1rZ48WLj7UjSzp07jWu6urqMa7Kysoxrok2KHGx3uVwR10mmiZE5AwIAWEEAAQCsIIAAAFYQQAAAKwggAIAVBBAAwAoCCABgBQEEALCCAAIAWEEAAQCsIIAAAFYQQAAAKyb1ZKTpNqmoU2fPnnVUd+WVV8a4J/Y99thjxjW33367cU13d7dxzdDQUMT2zMzM0H+D/x90wQUXGG9Hkn74wx8a15SWlhrXzJgxw7gmIyNjzOU5OTlhbW+//bbxdiTpkUceMa5pbm52tC1T0SYVDbYHAoGI60SbxNTJts4XZ0AAACsIIACAFUYfwe3atUv79u1TW1ubsrOzVVFRoTVr1qikpCS0ztmzZ1VfX6/XX39dAwMDqqqq0rp16+Tz+WLddwBACjM6A2ppadHNN9+sLVu26Lvf/a6Ghob00EMPqa+vL7TOU089pT/96U+65557tGnTJnV0dGjbtm0x7zgAILUZBdB9992nJUuW6OKLL1Z5ebk2bNigkydPqrW1VZLU09OjV155RXfeeafmzp2r2bNnq7a2Vn/96191+PDhuPwDAACp6byuguvp6ZEk5efnS5JaW1s1NDSkefPmhdYpLS1VUVGRDh8+HPFWugMDA6NuG+xyuZSbmyuPxyOv1ztq3eDjc9snm2QZB6e3NE9m411hFUlw/4+3aFfBjbU/OO1bOv5tnXByu/pItwQfTyxfy+MdHxJxFZzH45nQeo4DaHh4WE8++aQuu+wyzZw5U5LU2dmpzMzMsI1PnTpVnZ2dEZ9n165damxsDD2eNWuWtm7dqqampqjbbmtrc9rttMI4xN6iRYuMa44ePRqHnpj74IMPbHchKZx7Kfr5+Kd/+qeYPddY6urqYv6cqXB8cPyXqqur00cffaTNmzefVwdWrlypFStWhB4H03nx4sU6cODAqHW9Xq/a2tpUWlrq6HcU6SJZxuGVV14xrrnmmmvi0JPYee2114xr1qxZY1xz5swZ45qxzoA++OADlZWVhe0PTn5nI0nPPPOMcU1lZaVxjZPf4o11lpqZmanBwcGwdifv+iVp+/btxjXnHrcm4tlnnzWuiWa840MizoCqqqrGPIkIchRAdXV1am5u1qZNmzR9+vRQu8/n0+DgoPx+/6izoNOnT0e9Ci4rK0tZWVlh7X6/P+rBtbu7e1IHUJDtcUjHH/JGO8iPxUmYxPKHqJ9+znOf1+lHO+n4t3XCyY+0e3t7jWvi8TqOdnxIRAD5/f4JrWf0QW8gEFBdXZ327dunBx54QMXFxaOWz549WxkZGTp48GCo7fjx4zp58mTE738AAJOX0RlQXV2d9u7dq3vvvVe5ubmh73Xy8vKUnZ2tvLw8LV26VPX19crPz1deXp5++tOfqqKiggACAIxiFEAvvfSSJOnBBx8c1V5bW6slS5ZIku688065XC5t27ZNg4ODoR+iAgDwaa5AvGaZO081NTXav3//qDav16uuri4VFBRM6u+AkmUcnFxQ8NZbb8WhJ7Hj5OXwxhtvGNf09/fHrCYjI0M33XST/uu//ivse6K5c+cab0fSqNlNJup///d/jWs+/R3yRL3zzjsR291ut2pqatTc3Bz2HdaJEyeMtyPJ0Qwuf/d3f2dcE8vL3pPh+FBdXT2hSVm52B8AYAUBBACwggACAFhBAAEArCCAAABWEEAAACsIIACAFQQQAMAKAggAYAUBBACwggACAFhBAAEArCCAAABWxO7m6ZZFuqvqeAYGBuLQk8nj7bffNq4pLS01rvn444+jLnO5XBFnsHZ6C2Yndddff71xjZM7jo43Y/JNN91k/JyxdOrUKeOaRx55xLjmZz/7WcT2/Px8tbS0aM2aNWF3qe3r6zPejjTxO3t+Wk9Pj6NtmYq2PwTb3W53xHWS6W63nAEBAKwggAAAVhBAAAArCCAAgBUEEADACgIIAGAFAQQAsIIAAgBYQQABAKwggAAAVhBAAAArCCAAgBVpMxnp4OCg7S6Myckkl5Em2Ux1x48fN64pKCiI2O71enX8+HGVlpaqu7t71LJXX33VUf9qamoc1Zkab2LRWHG6D7W1tRnXzJs3z7jGySTC0QQnHD158mTY/uB0MtJkFm1S0WD78PBwUk08GglnQAAAKwggAIAVBBAAwAoCCABgBQEEALCCAAIAWEEAAQCsIIAAAFYQQAAAKwggAIAVBBAAwAoCCABgRdpMRprsE3cme/+S2ZkzZyK2Byd4PXPmTNg61157raNtLV++3Limp6fHuKa8vNy45s0334zYnp+fr/3796u6ujpsHD7++GPj7UjS1KlTjWuc7OP9/f3GNdFkZ2eHnjOWz5sIk3WyYs6AAABWEEAAACsIIACAFQQQAMAKAggAYAUBBACwggACAFhBAAEArCCAAABWEEAAACsIIACAFQQQAMCKtJmMNJESNXFgtO0E210uV9g66TBBYSwMDw87qnvppZdi3JPI9u7dG7Pn8nq9kqQjR46ou7s7Js/Z19cXk+dJpOC+HwgEUu51kOz9De5jE+XxeCa0HmdAAAArCCAAgBVGH8Ht2rVL+/btU1tbm7Kzs1VRUaE1a9aopKQktM6DDz6olpaWUXXLly/XV7/61dj0GACQFowCqKWlRTfffLMuueQSDQ0NaefOnXrooYf08MMPKycnJ7TesmXLtGrVqtDj4I2iAAAIMgqg++67b9TjDRs2aN26dWptbdUVV1wRap8yZYp8Pl9MOggASE/ndRVc8FbE+fn5o9qbmprU1NQkn8+nq6++WrfffrumTJkS8TkGBgY0MDAQeuxyuZSbmyuPxxN25UXwsekVGbFm+yq4scYh2a+miaVk2R9sYxxGMA4j4jEOps+Vl5c3ofVcAYdHrOHhYf3gBz+Q3+/X9773vVD77373OxUVFWnatGn64IMP9PTTT2vOnDnauHFjxOdpaGhQY2Nj6PGsWbO0detWJ10CAKQQxwH0xBNP6J133tHmzZs1ffr0qOsdOnRImzdv1r//+7/rwgsvDFse7Qxo8eLFOnDgwKh1vV6v2traVFpaGrPfOziRDGdAH3/8sS666KKwcZhsZ0DJsD/YxjiMYBxGxGMcTM+A5s6dq9/+9rfjrufoI7i6ujo1Nzdr06ZNY4aPJM2ZM0eS1N7eHjGAsrKylJWVFdbu9/ujDl53d/ekDqCgSOMwmQIoyPb+kCwYhxGMwwib4xD8emY8Rr8DCgQCqqur0759+/TAAw+ouLh43Jpjx45JkgoLC002BQBIc0ZnQHV1ddq7d6/uvfde5ebmqrOzU9LIF07Z2dlqb2/X3r17VVNTo/z8fH344Yd66qmndPnll6usrCwe/QcApCijAArOk/Xggw+Oaq+trdWSJUuUmZmpgwcPavfu3erv79f06dO1cOFC3XbbbTHrMAAgPRgFUENDw5jLi4qKtGnTpvPqEABgcmA27ASJ5YULqTzrb6JkZjrbtQcHB2PcE8RaImeJT5YLjmK1HadML2bw+/0TWo/JSAEAVhBAAAArCCAAgBUEEADACgIIAGAFAQQAsIIAAgBYQQABAKwggAAAVhBAAAArCCAAgBUEEADAiqSdjDTShILxmGzQCSYAHZGXl2dcM9E7JZ4vp5OKut3m78mGh4cdbQuxHe94TNKbkZFhXONk30v2yUhN/04TXZ8zIACAFQQQAMAKAggAYAUBBACwggACAFhBAAEArCCAAABWEEAAACsIIACAFQQQAMAKAggAYEXSzgVXWVkZ1ubxeCRJVVVV8vv9o5ZNpvnZxhqHRMrNzTWu6e3tjdn24zEOqTgXXLLsD07EcrzjMQ6ZmeaHSCdzwSX7OJj277LLLpvQeq7AZDpyAwCSRkp9BNfb26t//dd/jem76FTEOIxgHEYwDiMYhxGpNA4pFUCBQEBHjx6dVB+3RcI4jGAcRjAOIxiHEak0DikVQACA9EEAAQCsSKkAysrK0pe+9CVlZWXZ7opVjMMIxmEE4zCCcRiRSuPAVXAAACtS6gwIAJA+CCAAgBUEEADACgIIAGBF0s4Fd649e/bohRdeUGdnp8rKynTXXXdpzpw5truVUA0NDWpsbBzVVlJSokcffdROhxKkpaVFzz//vI4ePaqOjg5t3LhRCxYsCC0PBAJqaGjQyy+/LL/fr8rKSq1bt04zZsyw2OvYG28ctm/frldffXVUTVVVle67775EdzVudu3apX379qmtrU3Z2dmqqKjQmjVrVFJSElrn7Nmzqq+v1+uvv66BgQFVVVVp3bp18vl89joeYxMZhwcffFAtLS2j6pYvX66vfvWrie5uVCkRQK+//rrq6+u1fv16XXrppfr1r3+tLVu26NFHH9XUqVNtdy+hLr74Yt1///2hx04mMUw1/f39Ki8v19KlS/WjH/0obPmvfvUr/eY3v9GGDRtUXFysn//859qyZYsefvhhZWdnW+hxfIw3DpI0f/581dbWhh47mUwzmbW0tOjmm2/WJZdcoqGhIe3cuVMPPfSQHn74YeXk5EiSnnrqKTU3N+uee+5RXl6e6urqtG3bNn3ve9+z3PvYmcg4SNKyZcu0atWq0ONkez2kxN754osvatmyZfrsZz8rSVq/fr2am5v1+9//Xl/84hftdi7B3G53Wr2Tm4jq6mpVV1dHXBYIBLR7927ddtttuvbaayVJ3/zmN7V+/Xq99dZbWrRoUSK7GldjjUNQZmZmWu8f557NbdiwQevWrVNra6uuuOIK9fT06JVXXtG3v/1tzZ07V5JUW1urf/7nf9bhw4dVUVFho9sxN944BE2ZMiWp94ekD6DBwUG1traOChq326158+bp8OHD9jpmSXt7u772ta8pKytLFRUVuuOOO1RUVGS7W9acOHFCnZ2duuqqq0JteXl5mjNnjg4fPpxWATQRLS0tWrdunTwej+bOnavVq1fL6/Xa7lbc9PT0SJLy8/MlSa2trRoaGtK8efNC65SWlqqoqCitAuhc545DUFNTk5qamuTz+XT11Vfr9ttv15QpU2x0MaKkD6Curi4NDw+HpbjP59Px48ftdMqSSy+9VLW1tSopKVFHR4caGxv1wAMPaNu2bY7uzZMOOjs7JSnso9ipU6eGlk0W8+fP18KFC1VcXKz29nbt3LlT3//+97Vly5a0/Kh2eHhYTz75pC677DLNnDlT0sj+kJmZGbonTlA67w+RxkGSbrjhBhUVFWnatGn64IMP9PTTT+v48ePauHGjxd6OlvQBhP/36Y9fysrKQoH0xhtvaOnSpRZ7hmTw6bO9mTNnqqysTN/61rf07rvvjjojSBd1dXX66KOPtHnzZttdsSraOCxfvjz0/zNnzlRhYaE2b96s9vZ2XXjhhYnuZkRJ/7aooKBAbrc77N1LZ2dnUn+2mQgej0clJSVqb2+33RVrgvvA6dOnR7WfPn160u8fF1xwgbxeb1ruH3V1dWpubta//du/afr06aF2n8+nwcHBsDuBpuv+EG0cIgleNZxM+0PSB1BmZqZmz56tQ4cOhdqGh4d16NChtP08d6L6+vrU3t6eli+siSouLpbP59PBgwdDbT09PXr//fcn/f7xySef6MyZMyosLLTdlZgJBAKqq6vTvn379MADD6i4uHjU8tmzZysjI2PU/nD8+HGdPHkyrfaH8cYhkmPHjklSUu0PKfER3IoVK7R9+3bNnj1bc+bM0e7du9Xf368lS5bY7lpC1dfX65prrlFRUZE6OjrU0NAgt9utG264wXbX4ioYtEEnTpzQsWPHlJ+fr6KiIt1666167rnnNGPGDBUXF+uZZ55RYWFh6Kq4dDHWOOTn5+vZZ5/VwoUL5fP59Le//U07duzQhRdeqKqqKou9jq26ujrt3btX9957r3Jzc0OfjOTl5Sk7O1t5eXlaunSp6uvrlZ+fr7y8PP30pz9VRUVFWgXQeOPQ3t6uvXv3qqamRvn5+frwww/11FNP6fLLL1dZWZndzn9KysyGvWfPHj3//PPq7OxUeXm5vvKVr+jSSy+13a2EevTRR/WXv/xF3d3dKigoUGVlpVavXp00n+fGy7vvvqtNmzaFtd94443asGFD6Ieov/vd79TT06PKykrdfffdo36Ulw7GGof169frhz/8oY4ePSq/369p06bpqquu0qpVq9LqDPnLX/5yxPba2trQG9LgD1Ffe+01DQ4OpuUPUccbh5MnT+rHP/6xPvroI/X392v69OlasGCBbrvtNuXl5SW4t9GlTAABANJL0n8HBABITwQQAMAKAggAYAUBBACwggACAFhBAAEArCCAAABWEEAAACsIIACAFQQQAMAKAggAYAUBBACw4v8AILTzLi9yx/QAAAAASUVORK5CYII=\n"
          },
          "metadata": {}
        },
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 640x480 with 1 Axes>"
            ],
            "image/png": "iVBORw0KGgoAAAANSUhEUgAAAaAAAAGdCAYAAABU0qcqAAAAOnRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjEwLjAsIGh0dHBzOi8vbWF0cGxvdGxpYi5vcmcvlHJYcgAAAAlwSFlzAAAPYQAAD2EBqD+naQAAI95JREFUeJzt3X9wVNX9//HXJtnNj93FDcSIQQjEEKMCMajQURkRmMFaZuqvEUudsdWg06D1x1Cc0YJCjS2tONbWP5w2KhkVjalUULSOipqIHRxDqZhWqgHEYGTQBJINCfmx3z/2u/tx2d0kd93k7G6ej390z70n5/DO3X3l3r171ubz+XwCAGCUpZmeAABgbCKAAABGEEAAACMIIACAEQQQAMAIAggAYAQBBAAwggACABhBAAEAjCCAAABGZJieQDTLli3Tf//735A2p9Op+vp6zZs3T16v19DMzKMOftTBjzr4UQe/RKhDaWmpnnvuuSH3s43UWnCvv/66tm7dqvb2dhUWFuqmm25ScXHxsPvPnj1bu3btCmlzu906duyYxo0bp46OjnhPOWlQBz/q4Ecd/KiDXyLUoby8XI2NjUPuNyKX4Hbs2KGamhpde+21Wr9+vQoLC1VVVaWjR4+OxHAAgCQ0IgH0yiuvaOHChbrssst0xhlnaPny5XI4HNq+fftIDAcASEJxfw+or69Pzc3NuvLKK4NtaWlpmjlzpvbu3Ru2f29vr3p7e4OPbTabsrOz5XQ65Xa7Q/YNPD65fayhDn7UwY86+FEHv0Sog9PpHNZ+cQ+gY8eOaWBgQB6PJ6Td4/Ho0KFDYftv3rxZdXV1wcfTpk3T+vXrVV9fH3WMlpaWuM03mVEHP+rgRx38qINfMtTB+F1wV111lZYsWRJ8bLPZJEnz5s3T7t27Q/Z1u91qaWnRpEmTxvybjNSBOgRQBz/q4JcIdSgrKxv0JCIg7gE0btw4paWlqb29PaS9vb097KxIkux2u+x2e1i71+uNWryOjo4xfYAFUAc/6uBHHfyog5/JOgz39u+434SQkZGhoqIi7dmzJ9g2MDCgPXv2qKSkJN7DAQCS1IhcgluyZIkef/xxFRUVqbi4WNu2bVNPT4/mz58/EsMBAJLQiATQRRddpGPHjqm2tlbt7e2aOnWq7r333oiX4AAAY9OI3YRw+eWX6/LLLx+pH48klZZm/arvwMDACMwEgGksRgoAMIIAAgAYQQABAIwggAAARhBAAAAjCCAAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIwggAAARhj/RlSMLSwsCiCAMyAAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIwggAAARhBAAAAjCCAAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIwggAAARhBAAAAjCCAAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIwggAAARhBAAAAjCCAAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIwggAAARhBAAAAjCCAAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIwggAAARmSYngAApBK73W65T29v7wjMJPFxBgQAMIIAAgAYEfdLcLW1taqrqwtpKygo0KOPPhrvoQAASWxE3gOaPHmyVq9eHXyclsaJFgAg1IgEUFpamjwez0j8aABAihiRAGptbdWtt94qu92ukpISLVu2THl5eRH37e3tDbkDxGazKTs7W06nU263O2TfwOOT28ca6uBHHfyog1+i1MH0XXCJUAen0zms/Ww+n88Xz4F37dql7u5uFRQUqK2tTXV1dfr222+1YcMGZWdnh+1/8ntG06ZN0/r16+M5JQBAAop7AJ3M6/WqsrJSN954oxYsWBC2PdoZ0Lx587R79+6Qfd1ut1paWjRp0iR1dHSM5LQTGnXwow5+1MEvUeqQCGdAputQVlam+vr6Ifcb8Q+iOp1OFRQUqLW1NeJ2u90e8Rfm9XqjFq+jo2NMP9ECqIMfdfCjDn6m62A6gAJM1sHr9Q5rvxG/Pa27u1utra3clAAACBH3M6CamhpdcMEFysvLU1tbm2pra5WWlqZLLrkk3kMBAJJY3APo22+/1R//+Ed1dHRo3LhxKi0tVVVVlcaNGxfvoQAASSzuAXTnnXfG+0cihaSnp1vuE+0+mcAHnNPS0sI+7BzLONLYXRTyZBkZ1l8aTjvtNMt9jhw5YrlPtN/RYMfD5MmTLY8jSV999ZXlPidOnIhprLGIJQoAAEYQQAAAIwggAIARBBAAwAgCCABgBAEEADCCAAIAGEEAAQCMIIAAAEYQQAAAIwggAIARBBAAwIgR/0I6jC6Xy2W5T2dn5wjMJLJYvoDX4XAM2u5wOML2ycnJsT45SW1tbZb7jPCXCgfZbLZB2202W9g+sc5tx44dlvvMnj3bcp+TFw0djjVr1kRsz8zMlCStXLlSPT09IduqqqosjyPFdhyxGOnwcQYEADCCAAIAGEEAAQCMIIAAAEYQQAAAIwggAIARBBAAwAgCCABgBAEEADCCAAIAGEEAAQCMIIAAAEYQQAAAI1gNO8WM5srWGRnWD58f/ehHlvs89dRTg27/9NNPw9qKioosjyPF9m/q6+uz3Mdut1vuE21l68CcMzIywuYfy79HkpYtW2a5z//+97+YxrLq4MGDEduzsrIkSV9++aW6u7tDtsW6KrjX642pH4aHMyAAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIwggAAARhBAAAAjCCAAgBEEEADACAIIAGAEAQQAMILFSBGzWBb8fPHFFy33+fDDDyO2p6ena+7cudq7d6/6+/tDtv3whz+0PI4k/f3vf7fc5+SxhyOWBUxtNlvE9oGBgeB/A/8fcPz4ccvjSFJzc3NM/ayKZZHQd955J2K7y+WSJDU0NIzqoryIHWdAAAAjCCAAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIwggAAARhBAAAAjCCAAgBEEEADACAIIAGAEi5FCU6ZMianfv//9b8t9vvrqK8t9oi0+6XA4NHfuXNXX1+vEiRMh21pbWy2PI8W2eOell15quU9DQ4PlPtEWPQ209/f3x7QwaiS//e1v4/JzhhJtgdXBzJ49O2J7dna2JKmsrCzs93jgwAHrk8OI4wwIAGAEAQQAMMLyJbimpiZt2bJF+/btU1tbm1auXKk5c+YEt/t8PtXW1uqtt96S1+tVaWmpKioqdPrpp8d14gCA5Gb5DKinp0dTp07VzTffHHH7yy+/rNdee03Lly/XQw89pMzMTFVVVYVdowcAjG2WA6i8vFzXX399yFlPgM/n07Zt23T11VfrwgsvVGFhoW677Ta1tbVF/VZLAMDYFNe74A4fPqz29nbNmjUr2JaTk6Pi4mLt3btXF198cVif3t5e9fb2Bh/bbDZlZ2fL6XTK7XaH7Bt4fHL7WBPvOgS+yng0xHLXk8PhGLQ90vacnBzL40ix1TSWsWIZJ9odbiPxvIhW80QQuNstWnuk7WPpNSMRXiedTuew9rP5YvlS9v/vuuuuC3kP6NNPP9Xq1av1xBNPKDc3N7jfI488IpvNprvuuivsZ9TW1qquri74eNq0aVq/fn2sUwIAJAnjnwO66qqrtGTJkuDjwF/I8+bN0+7du0P2dbvdamlp0aRJk9TR0TGq80wk8a7D5MmTY+rX2Nhouc/hw4ct93nhhRcitjscDv3yl7/UY489FvYeY7TPDg3lvffes9znkksusdzngw8+sNxnsDOgeD8v1q1bZ7nPnXfeGZexh/LTn/40Ynt2drb++te/qqKiIuxzQFu3bh2NqSWERHidLCsrU319/ZD7xTWAPB6PJOno0aMhZ0BHjx7V1KlTI/ax2+2y2+1h7V6vN2rxOjo6xnQABcSrDp2dnXGYzfDEcsI91A0sJ06cCNunq6vL8jiSYqpnLGPFMs5QHzKN5/MikW8aGurDwsePHw/bZyy+Xph8nfR6vcPaL66fA8rPz5fH49HHH38cbOvq6tJnn32mkpKSeA4FAEhyls+Auru7Q5Y5OXz4sPbv3y+Xy6W8vDxdccUVeumll3T66acrPz9fzz//vHJzc3XhhRfGdeIAgORmOYA+//xzrV27Nvi4pqZGkn89rBUrVujHP/6xenp69MQTT6irq0ulpaW69957E/quGgDA6LMcQOeee65qa2ujbrfZbFq6dKmWLl36vSaWaobzhtzJ5s2bZ7nPr371K8t9KioqLPeRYns/57t3PA7XjBkzIrZnZPgP33POOUd9fX0h2/7xj39YHkeSfvKTn1ju8/rrr1vuE69FQ4eSnp4eU79Vq1bFeSbx8/LLL0dsD9x2/Oqrr47J93ySEWvBAQCMIIAAAEYQQAAAIwggAIARBBAAwAgCCABgBAEEADCCAAIAGEEAAQCMIIAAAEYQQAAAIwggAIARBBAAwAjjX8k9VsSysnUstm/fbrnP7373u5jGSkuz/vfLz372M8t9oq3MnJWVpSuuuELbtm1Td3d3yLaGhgbL40ixrfAdS5/RUllZaXoKg4pl1eqBgYFB2wcGBqLuk0oyMzMHbc/MzIz4zbY9PT0jOi8rOAMCABhBAAEAjCCAAABGEEAAACMIIACAEQQQAMAIAggAYAQBBAAwggACABhBAAEAjCCAAABGEEAAACNYjDSB2Wy2QdttNlvYPsePH7c8TiyLisbK4/FY7rNx48aI7W63W3/+85/13HPPhS1qORYWoxyOQ4cOmZ7CoF566SXTU0ha0RYVdTgcwe2JtPBoJJwBAQCMIIAAAEYQQAAAIwggAIARBBAAwAgCCABgBAEEADCCAAIAGEEAAQCMIIAAAEYQQAAAIwggAIARLEaawHw+36DtPp8vbJ/6+voRn9f30d7ebrlPVlbWoO1ZWVnq7e0N2dbZ2Wl5nFQ0btw401MY1Kmnnmp6CjCIMyAAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIwggAAARhBAAAAjCCAAgBEEEADACAIIAGAEAQQAMILFSBNYR0fHoNsPHToU1uZyuUZqOnHh8Xgs95kyZUrEdqfTKUk644wz5PV6Q7YdOHDA8jiS1NXVFVM/q4qKiiz3WbJkScT2zMxMSdIvfvEL9fT0hGx76KGHrE9uFC1atMhyn4yMyC9bgfaMjIywfQYGBqxP7nv0S2Q2m81yn2gLI39fnAEBAIwggAAARli+BNfU1KQtW7Zo3759amtr08qVKzVnzpzg9scff1zvvvtuSJ+ysjLdd99933+2AICUYTmAenp6NHXqVC1YsEAPP/xwxH3OO+88VVZW/t8gUa7ZAgDGLsvJUF5ervLy8sF/aEZGTG82AwDGjhE5NWlqalJFRYWcTqdmzJih66+/Xm63O+K+vb29IV+nbLPZlJ2dLafTGdYn8Djaz0JqCtztFq090vZYj5H09PSY+lkVy92KgbvdorVH255qov1uB3t9GEt3wQ31Ojkad8FFe86GzcX3Pe6vu+6668LeA3r//feVmZmp/Px8tba2atOmTcrKylJVVZXS0sLveaitrVVdXV3w8bRp07R+/fpYpwQASBJxPwO6+OKLg/8/ZcoUFRYW6vbbb9cnn3yimTNnhu1/1VVXhXy+IZDO8+bN0+7du0P2dbvdamlp0aRJk4b8jEwqiPQ5nwCXy6XOzs6I7Ykslr8of/CDH0Rsdzqd2r59uy677LKwzwEdPHgwpvmN1ueApk2bZrnP4sWLI7ZnZmZq3bp1WrNmTdjngO6///6Y5peTkxNTP6tOnDhhuc/EiRMjtrvdbh04cECFhYVhrw9j7QxosNfJ0TgDKisrU319/ZD7jfjdAaeddprcbrdaW1sjBpDdbpfdbg9r93q9UUOmo6NjTAQQ/E4Ol0jbT94n1uNjtAIo0h8PQzk5XCJtH2qfVDDU7zbS68NYCqCAaK+ToxFAQz1nA0b8c0DffPONOjs7lZubO9JDAQCSiOUzoO7ubrW2tgYfHz58WPv375fL5ZLL5dKLL76ouXPnyuPx6Ouvv9YzzzyjiRMnqqysLK4TBwAkN8sB9Pnnn2vt2rXBxzU1NZKkSy+9VMuXL9cXX3yhd999V16vV+PHj9esWbO0dOnSiJfZAABj1/e6C24kzZ49W7t27Qppc7vdOnbsmMaNGzcm3gN67733Iranp6froosu0o4dO9Tf3x+yrbS01PI4eXl5Mc0vlmvJsbxHMdgfL2lpaRGv08cyNym2a/6R7u4cSqzzQ/QbTGw2m8444wx9+eWXYe9ZTJ06NaaxRut4+O4f9cO1evXqiO2J8DpZXl6uxsbGIfdjLTgAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIwggAAARhBAAAAjCCAAgBEEEADACAIIAGAEAQQAMIIAAgAYMeLfiIrYrVq1KmJ7Tk6O3nrrLa1evTrsGzw/+ugjy+PEumKuw+Gw3CeWr+X44osvIrbbbDYVFhbq4MGDcVv9OD09PaZ+o2GwhettNlvE7bEudh/Lis6j5Y477ojYnp2drWeffVb33HOPjh8/HrJtNL/ZNJaxoq1sneoS9ygDAKQ0AggAYAQBBAAwggACABhBAAEAjCCAAABGEEAAACMIIACAEQQQAMAIAggAYAQBBAAwggACABjBYqQJ7J///GfEdrfbLUn68MMPwxYSPeWUUyyPk5eXZ31yks455xzLfebMmWO5T7Q6OJ1OvfPOO7rxxhvl9XpDtmVlZVkeR5Lq6uos9xk/frzlPn19fZb79Pf3R93mcrnCaiBJPT09lseRYvs32Ww2y31qa2st99m8eXPE9sDzYuvWrTEvsIvRxRkQAMAIAggAYAQBBAAwggACABhBAAEAjCCAAABGEEAAACMIIACAEQQQAMAIAggAYAQBBAAwggACABjBYqQp5ujRo6M21s6dO0elTzSBxScbGxvDFp+MZWFMSZo9e7blPueee67lPsePH7fcZ9euXRHb3W63vvrqK02fPj2sDu3t7ZbHkQZf+DSa2267zXKf1157zXIfpA7OgAAARhBAAAAjCCAAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIwggAAARhBAAAAjCCAAgBEEEADACBYjRUry+Xwx9Tt06NCo9IlFVlZWxPbAwqH9/f1hi4g2NDTENFYsi9o+9dRTlvucOHHCch+kDs6AAABGEEAAACMsXYLbvHmzdu7cqZaWFjkcDpWUlOiGG25QQUFBcJ8TJ06opqZGO3bsUG9vr8rKylRRUSGPxxPvuQMAkpilM6CmpiYtXrxYVVVV+vWvf63+/n49+OCD6u7uDu6zceNGffTRR7r77ru1du1atbW1acOGDXGfOAAguVkKoPvuu0/z58/X5MmTNXXqVK1YsUJHjhxRc3OzJKmrq0tvv/22brzxRs2YMUNFRUWqrKzUp59+qr17947IPwAAkJy+111wXV1dkiSXyyVJam5uVn9/v2bOnBncZ9KkScrLy9PevXtVUlIS9jN6e3vV29sbfGyz2ZSdnS2n0xn8yuWAwOOT28ca6uA31uoQ7S64wPMv8N/vSk9Pj2msjAzrLw2x/B7ieRfcWDseokmEOjidzmHtF3MADQwM6Omnn9ZZZ52lKVOmSPJ//3xGRkbY4KecckrU76bfvHmz6urqgo+nTZum9evXq76+PurYLS0tsU47pVAHP+rgF7gSYcqRI0eMjh/A8eCXDHWIOYCqq6t18OBBrVu37ntN4KqrrtKSJUuCj202myRp3rx52r17d8i+brdbLS0tmjRpkjo6Or7XuMmMOviNtToMdgbU3NysoqIidXZ2hmz729/+FtNYx44ds9znxhtvtNwn3mdAY+l4iCYR6lBWVjboSURATAFUXV2txsZGrV27VhMmTAi2ezwe9fX1yev1hpwFHT16NOpdcHa7XXa7Pazd6/VGLV5HR8eYPsACqIPfWKnDdy9VR9LZ2RlWh5M/mDpcfX19lvvE8jsYiQ+ijpXjYSgm6+D1eoe1n6WbEHw+n6qrq7Vz506tWbNG+fn5IduLioqUnp6ujz/+ONh26NAhHTlyJOL7PwCAscvSGVB1dbUaGhq0atUqZWdnB9/XycnJkcPhUE5OjhYsWKCamhq5XC7l5OToySefVElJCQEEAAhhKYDeeOMNSdIDDzwQ0l5ZWan58+dL8l8Httls2rBhg/r6+oIfRAUA4LssBVBtbe2Q+zgcDlVUVBA6QJx99wPf3xV4D7W7uztsn+XLl8c0Vnl5ueU+Q71HBZyMteAAAEYQQAAAIwggAIARBBAAwAgCCABgBAEEADCCAAIAGEEAAQCMIIAAAEYQQAAAIwggAIARBBAAwAgCCABgRMxfyQ0g8X3zzTcx9du8eXOcZwKE4wwIAGAEAQQAMIIAAgAYQQABAIwggAAARhBAAAAjCCAAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIxgMVIghXV1dY3aWNnZ2Zb7HD9+3HIfn89nuQ8SE2dAAAAjCCAAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIwggAAARhBAAAAjCCAAgBEEEADACAIIAGAEi5ECKWzy5Mkx9du/f7/lPrEsLJqbm2u5z6mnnhqx3el0SpKmT58ur9cbsu3AgQOWx5Gk7u7umPpheDgDAgAYQQABAIwggAAARhBAAAAjCCAAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIwggAAARhBAAAAjUmYxUpvNZrmPz+cbgZkAieOzzz4zPYVBtbe3W+7T1tYWsd3tdkvy/5s7OjpCtvFcT0ycAQEAjCCAAABGWLoEt3nzZu3cuVMtLS1yOBwqKSnRDTfcoIKCguA+DzzwgJqamkL6LVq0SLfcckt8ZgwASAmWAqipqUmLFy/WmWeeqf7+fm3atEkPPvigHnnkEWVlZQX3W7hwoZYuXRp87HA44jdjAEBKsBRA9913X8jjFStWqKKiQs3NzTrnnHOC7ZmZmfJ4PHGZIAAgNX2vu+C6urokSS6XK6S9vr5e9fX18ng8Ov/883XNNdcoMzMz4s/o7e1Vb29v8LHNZlN2dracTmfwrpaAwOOT2wP9rErWO2MGq8NYQh38krkOaWnW34aO9rwdrA7J+lyPRSIcD4GvRx+KzRfjb2ZgYEC///3v5fV69Zvf/CbY/uabbyovL0/jx4/XgQMH9Oyzz6q4uFgrV66M+HNqa2tVV1cXfDxt2jStX78+likBAJJIzAH0l7/8Rf/617+0bt06TZgwIep+e/bs0bp16/TYY49p4sSJYdujnQHNmzdPu3fvDtnX7XarpaVFkyZNCrvPf6ydAUWrw1hCHfySuQ7xPgP68ssvdcYZZ4zpzwElwvFQVlam+vr6IfeL6RJcdXW1GhsbtXbt2kHDR5KKi4slSa2trREDyG63y263h7V7vd6oxevo6BjTARQQqQ5jEXXwS8Y6xDOAAiLVIdmf67EweTx4vd5h7Wfpt+/z+VRdXa2dO3dqzZo1ys/PH7LP/v37JUm5ublWhgIApDhLZ0DV1dVqaGjQqlWrlJ2dHVxGIycnRw6HQ62trWpoaNDs2bPlcrn0xRdfaOPGjTr77LNVWFg4EvMHACQpSwH0xhtvSPJ/2PS7KisrNX/+fGVkZOjjjz/Wtm3b1NPTowkTJmju3Lm6+uqr4zZhAEBqsBRAtbW1g27Py8vT2rVrv9eEAABjQ8qshj0W32QEkt3AwEDcflbgNcDn8/F6kCRYjBQAYAQBBAAwggACABhBAAEAjCCAAABGEEAAACMIIACAEQQQAMAIAggAYAQBBAAwggACABhBAAEAjEiZxUgBxM8LL7xguc/SpUtHYCZIZZwBAQCMIIAAAEYQQAAAIwggAIARBBAAwAgCCABgBAEEADCCAAIAGEEAAQCMIIAAAEYQQAAAIxJ2LbjS0tKwNqfTKUkqKyuT1+sd7SklDOrgRx38RqIOHo/Hcp/y8vK4jB0rjge/RKhDpNfvSGw+n883wnMBACBMUl2CO378uO655x4dP37c9FSMog5+1MGPOvhRB79kqkNSBZDP59O+ffs01k/aqIMfdfCjDn7UwS+Z6pBUAQQASB0EEADAiKQKILvdrmuvvVZ2u930VIyiDn7UwY86+FEHv2SqA3fBAQCMSKozIABA6iCAAABGEEAAACMIIACAEQm7FtzJXn/9dW3dulXt7e0qLCzUTTfdpOLiYtPTGlW1tbWqq6sLaSsoKNCjjz5qZkKjpKmpSVu2bNG+ffvU1tamlStXas6cOcHtPp9PtbW1euutt+T1elVaWqqKigqdfvrpBmcdf0PV4fHHH9e7774b0qesrEz33XffaE91xGzevFk7d+5US0uLHA6HSkpKdMMNN6igoCC4z4kTJ1RTU6MdO3aot7dXZWVlqqioiGl9u0Q1nDo88MADampqCum3aNEi3XLLLaM93aiSIoB27NihmpoaLV++XNOnT9err76qqqoqPfroozrllFNMT29UTZ48WatXrw4+TktL/ZPYnp4eTZ06VQsWLNDDDz8ctv3ll1/Wa6+9phUrVig/P18vvPCCqqqq9Mgjj8jhcBiY8cgYqg6SdN5556mysjL4OCMjKZ7iw9bU1KTFixfrzDPPVH9/vzZt2qQHH3xQjzzyiLKysiRJGzduVGNjo+6++27l5OSourpaGzZs0G9+8xvDs4+f4dRBkhYuXKilS5cGHyfa8yEpjs5XXnlFCxcu1GWXXSZJWr58uRobG7V9+3ZdeeWVZic3ytLS0lLqL7nhKC8vj7rSss/n07Zt23T11VfrwgsvlCTddtttWr58uT788ENdfPHFoznVETVYHQIyMjJS+vg4+WxuxYoVqqioUHNzs8455xx1dXXp7bff1h133KEZM2ZIkiorK3XXXXdp7969KikpMTHtuBuqDgGZmZkJfTwkfAD19fWpubk5JGjS0tI0c+ZM7d2719zEDGltbdWtt94qu92ukpISLVu2THl5eaanZczhw4fV3t6uWbNmBdtycnJUXFysvXv3plQADUdTU5MqKirkdDo1Y8YMXX/99XK73aanNWK6urokSS6XS5LU3Nys/v5+zZw5M7jPpEmTlJeXl1IBdLKT6xBQX1+v+vp6eTwenX/++brmmmuUmZlpYooRJXwAHTt2TAMDA2Ep7vF4dOjQITOTMmT69OmqrKxUQUGB2traVFdXpzVr1mjDhg3Kzs42PT0j2tvbJSnsUuwpp5wS3DZWnHfeeZo7d67y8/PV2tqqTZs26aGHHlJVVVVKXqodGBjQ008/rbPOOktTpkyR5D8eMjIygt+JE5DKx0OkOkjSJZdcory8PI0fP14HDhzQs88+q0OHDmnlypUGZxsq4QMI/+e7l18KCwuDgfTBBx9owYIFBmeGRPDds70pU6aosLBQt99+uz755JOQM4JUUV1drYMHD2rdunWmp2JUtDosWrQo+P9TpkxRbm6u1q1bp9bWVk2cOHG0pxlRwv9ZNG7cOKWlpYX99dLe3p7Q1zZHg9PpVEFBgVpbW01PxZjAMXD06NGQ9qNHj4754+O0006T2+1OyeOjurpajY2Nuv/++zVhwoRgu8fjUV9fX9g3gabq8RCtDpEE7hpOpOMh4QMoIyNDRUVF2rNnT7BtYGBAe/bsSdnrucPV3d2t1tbWlHxiDVd+fr48Ho8+/vjjYFtXV5c+++yzMX98fPPNN+rs7FRubq7pqcSNz+dTdXW1du7cqTVr1ig/Pz9ke1FRkdLT00OOh0OHDunIkSMpdTwMVYdI9u/fL0kJdTwkxSW4JUuW6PHHH1dRUZGKi4u1bds29fT0aP78+aanNqpqamp0wQUXKC8vT21tbaqtrVVaWpouueQS01MbUYGgDTh8+LD2798vl8ulvLw8XXHFFXrppZd0+umnKz8/X88//7xyc3ODd8WlisHq4HK59OKLL2ru3LnyeDz6+uuv9cwzz2jixIkqKyszOOv4qq6uVkNDg1atWqXs7OzglZGcnBw5HA7l5ORowYIFqqmpkcvlUk5Ojp588kmVlJSkVAANVYfW1lY1NDRo9uzZcrlc+uKLL7Rx40adffbZKiwsNDv570ia1bBff/11bdmyRe3t7Zo6dap+/vOfa/r06aanNaoeffRR/ec//1FHR4fGjRun0tJSXX/99QlzPXekfPLJJ1q7dm1Y+6WXXqoVK1YEP4j65ptvqqurS6Wlpbr55ptDPpSXCgarw/Lly/WHP/xB+/btk9fr1fjx4zVr1iwtXbo0pc6Qr7vuuojtlZWVwT9IAx9Eff/999XX15eSH0Qdqg5HjhzRn/70Jx08eFA9PT2aMGGC5syZo6uvvlo5OTmjPNvokiaAAACpJeHfAwIApCYCCABgBAEEADCCAAIAGEEAAQCMIIAAAEYQQAAAIwggAIARBBAAwAgCCABgBAEEADCCAAIAGPH/ABN1y9Vk9nLQAAAAAElFTkSuQmCC\n"
          },
          "metadata": {}
        },
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 640x480 with 1 Axes>"
            ],
            "image/png": "iVBORw0KGgoAAAANSUhEUgAAAaAAAAGdCAYAAABU0qcqAAAAOnRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjEwLjAsIGh0dHBzOi8vbWF0cGxvdGxpYi5vcmcvlHJYcgAAAAlwSFlzAAAPYQAAD2EBqD+naQAAJH5JREFUeJzt3X1wVNX9x/HPJtmEZLNhgTRCogQwxKhAxAeYqSAQdLQO0wo6wDjMMGrQacDaOlQ7olRQnFLFoXX4o+PESorFxtSMaJH6xNggncEpyICxzSiPhokI5ImNCUl2f39kdn+E3U1yL7s5m8379U+4596Te/jm3nxyd++e6/D7/X4BADDIkkwPAAAwPBFAAAAjCCAAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIwggAAARhBAAAAjUkwPIJL7779f//3vf3u1uVwu1dTUaPbs2fJ6vYZGZh516EEdesRLHZKSrP896/P5orb/eKmDafFQh6KiIv31r3/tdztHrOaC27Vrl9599101NTUpPz9fDz74oAoKCgbc/8Ybb9SBAwd6tbndbrW0tCgrK0utra3RHvKQQR16UIce8VIH0wEUL3UwLR7qMH36dO3fv7/f7WLyEtzevXtVUVGh++67Txs3blR+fr42bNig5ubmWOwOADAExSSA3nvvPc2fP1/z5s3TlVdeqRUrVig1NVW7d++Oxe4AAENQ1N8D6urq0pEjR3TPPfcE25KSkjR16lTV1dWFbN/Z2anOzs7gssPhUHp6ulwul9xud69tA8uXtg831KEHdegRL3WIh5fgLv46XMVDHVwu14C2i3oAtbS0yOfzyePx9Gr3eDw6depUyPbV1dWqqqoKLk+cOFEbN25UTU1NxH3U19dHbbxDGXXoQR16UIce1KHHUKiD8bvgFi5cqAULFgSXHQ6HJGn27Nk6ePBgr23dbrfq6+uVl5c37N9kpA7UISBe6hAPV0DxUAfT4qEOxcXFfV5EBEQ9gLKyspSUlKSmpqZe7U1NTSFXRZLkdDrldDpD2r1eb8Titba2DusDLIA69KAOPUzXwXQABZiuQ7wwWYeB3v4d9ZsQUlJSNGnSJB0+fDjY5vP5dPjwYRUWFkZ7dwCAISomL8EtWLBAW7Zs0aRJk1RQUKCdO3eqo6NDc+fOjcXuAABDUEwC6Mc//rFaWlpUWVmppqYmTZgwQU899VTYl+AAAMNTzG5CuOuuu3TXXXfZ7u9wOII3JFzcFmldjCZ0wBCVlZVlq19LS0uURzJ8xOL9HMSH1NRUS9uHe18/HCYjBQAYQQABAIwggAAARhBAAAAjCCAAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIwggAAARhBAAAAjjD8RNRK/3x8ywWhgOdw64GJMKgpEz4ULFyxt39nZOaDtuAICABhBAAEAjCCAAABGEEAAACMIIACAEQQQAMAIAggAYAQBBAAwggACABhBAAEAjCCAAABGEEAAACMIIACAEXE7GzaAy+dwOGz1Y7Z5DAaugAAARhBAAAAjCCAAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIwggAAARhBAAAAjCCAAgBEEEADACCYjjWORJpIMtDscDtuTTV4sESeeTEqy97fViBEjLPdpb2+33Mftdlvuc+HChbDt6enpwa9dXV291jmdTsv7kaSWlhZb/ayyc/zaOV7tHg8+n89WPwwMV0AAACMIIACAEQQQAMAIAggAYAQBBAAwggACABhBAAEAjCCAAABGEEAAACMIIACAEQQQAMAIAggAYASTkcaxSJMuBtr9fn9CTiQaDXYnkezu7rbc58CBA5b7jBw50nKf2267LWx7ZmamJCk7OztkMtWTJ09a3o80eJOEDtZ+mFQ0PnEFBAAwggACABgR9ZfgKisrVVVV1astNzdXmzdvjvauAABDWEzeA7rqqqv0zDPPBJftPgwKAJC4YhJASUlJ8ng8sfjWAIAEEZMAamho0COPPCKn06nCwkLdf//9ys7ODrttZ2enOjs7g8sOh0Pp6elyuVwhjy0OLNt5nHEioQ49YlGHtLQ0y33sXOHbufsrcLfbpVwuV6+vFxvMY2SwHpUd6Y42zose8VCHcMdiOA5/lO/jPXDggNrb25Wbm6vGxkZVVVXp3Llz2rRpU/DZ9Re79D2jiRMnauPGjdEcEgAgDkU9gC7l9XpVVlam5cuXq6SkJGR9pCug2bNn6+DBg722dbvdqq+vV15enlpbW2M57LhGHXrEog52roB2795tuU9WVpblPj/5yU/CtrtcLu3bt08zZsyQ1+vtte7bb7+1vB+74uEKiPMiPupQXFysmpqafreL+QdRXS6XcnNz1dDQEHa90+mU0+kMafd6vRGL19raOqwPsADq0COadbhw4YLlPnY+5Gjnl/X58+f7XO/1ekO2Gczjw3QABXBe9DBZh0v/EIok5rentbe3q6GhgZsSAAC9RP0KqKKiQjfffLOys7PV2NioyspKJSUladasWdHeFQBgCIt6AJ07d05/+MMf1NraqqysLBUVFWnDhg22XvMGACSuqAfQL3/5y2h/S8SpwZpIMtL7BIH2pKSkkG1SU1Mt70eSnnzySct9Jk+ebLlPV1eX5T6rVq0K2x74vz788MMh72HZ+f9I9n5OdjBJqH2Rzr9Au8PhsHWOhhOr44EpCgAARhBAAAAjCCAAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIwggAAARhBAAAAjCCAAgBEEEADAiJg/kM6ucBPp9TXJ3mBNnoj/Z3rCykC7z+cL2cbOg+WknqdJWmVn4tOUFOun3po1a8K2u91uPfbYY1q/fn3IA8g4LxJXpJ9toN3v98f9z58rIACAEQQQAMAIAggAYAQBBAAwggACABhBAAEAjCCAAABGEEAAACMIIACAEQQQAMAIAggAYAQBBAAwggACABgRt7Nhh5vJdSjN8oqh6e2337bcZ/HixZb7HD9+3HKfjIyMPtszMjLU3d3da11zc7Pl/QCDhSsgAIARBBAAwAgCCABgBAEEADCCAAIAGEEAAQCMIIAAAEYQQAAAIwggAIARBBAAwAgCCABgBAEEADAibicjtcrhcFjuw4SmiWvcuHG2+j399NOW+7jdbst9pk6darmPz+frs93n80XcJl5x3g5vXAEBAIwggAAARhBAAAAjCCAAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIwggAAARhBAAAAjCCAAgBEJMxkpExQmrszMzD7bMzMzQ37+f/7zn23ta968eZb7tLS0WO5z8uRJy33Onz8ftj0woef58+cjbhOvOG+HN66AAABGEEAAACMsvwRXW1urHTt26OjRo2psbNTq1as1Y8aM4Hq/36/Kykp9/PHH8nq9KioqUmlpqe3nswAAEpPlK6COjg5NmDBBDz30UNj177zzjt5//32tWLFCL7zwgtLS0rRhwwZduHDhsgcLAEgclgNo+vTpWrp0aa+rngC/36+dO3dq0aJFuuWWW5Sfn69Vq1apsbFRn3/+eVQGDABIDFG9C+706dNqamrStGnTgm0ZGRkqKChQXV2dbr311pA+nZ2d6uzsDC47HA6lp6fL5XKFPOo4sGznEciJZLjVYSB3wV0qOTk5pmO6XElJ1t9+jfTz7ut4GE53mQ238yKSeKiDy+Ua0HZRDaCmpiZJ0siRI3u1jxw5MrjuUtXV1aqqqgouT5w4URs3blRNTU3E/dTX11/2WBMBdehRV1dndP+jR48elD7Nzc19rv/2228tf89ExHnRYyjUwfjngBYuXKgFCxYElwOfaZg9e7YOHjzYa1u32636+nrl5eWptbV1UMcZT4ZbHfq6Aqqrq1NhYWHI51+2bdtma1+33Xab5T52Pgdk55fDrFmzwra73W59++23uvLKK0OOh+F2BTSczotI4qEOxcXFfV5EBEQ1gDwej6Sev9RGjRoVbG9ubtaECRPC9nE6nXI6nSHtXq83YvFaW1uH9QEWMFzq0N8v0fPnz4fUobu7O5ZDumw+n89yn/5+1uGOh+EUQAHD5bzoj8k6eL3eAW0X1c8B5eTkyOPx6NChQ8G2trY2ff311yosLIzmrgAAQ5zlK6D29nY1NDQEl0+fPq1jx44pMzNT2dnZuvvuu/X2229r3LhxysnJ0ZtvvqlRo0bplltuierAAQBDm+UA+uabb7Ru3brgckVFhSRpzpw5WrlypX72s5+po6NDf/rTn9TW1qaioiI99dRTSk1Njd6oAQBDnuUAuv7661VZWRlxvcPh0JIlS7RkyZLLGhgS00Bvz7zYr3/967DtgT9qfvGLX4R80Pniq3Qrvv76a8t9It3h2ZfAH25WBG7QidTucDhCton394BSUqy/Dd3V1RWDkcAE5oIDABhBAAEAjCCAAABGEEAAACMIIACAEQQQAMAIAggAYAQBBAAwggACABhBAAEAjCCAAABGEEAAACMIIACAEcYfyY2hy85Mxvn5+Zb7VFdXh213uVz6zW9+o/feey/kCYx1dXWW9yNJP/rRjyz3+ctf/mK5z7Jlyyz32b17d9j2wAzjBQUFIXU4duyY5f1I6vVE44HKzc213Of48eOW+zQ2NoZtT05ODn4N/Dsg3p+QO1xxBQQAMIIAAgAYQQABAIwggAAARhBAAAAjCCAAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIwggAAARjAZKWxLS0uz3Ofo0aOW+/j9/rDtbrdbUs/Eo62trb3WdXR0WN6PJNXX11vuM2bMGMt9xo8fb7nPZ5991uf6Dz74IKRtzZo1lvcjSc8995zlPlVVVZb7vPTSS5b7nD17Nmy7z+cLfg38O5FFOv8C7Wlpabpw4ULIejvnhsPhiMn2XAEBAIwggAAARhBAAAAjCCAAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIwggAAARhBAAAAjCCAAgBEJMxlpSor1/0pXV1cMRjJ82KmfnT6RJpZMTU2V1DO54qUTLFqdPDFgxIgRlvssXrzYcp8DBw5Y7tPf2LKyskLa5syZY3k/kpScnGy5j52f7fHjxy33iTQ5baDd7/dH3CaRRJpUtK/zwi6r9Rzo9lwBAQCMIIAAAEYQQAAAIwggAIARBBAAwAgCCABgBAEEADCCAAIAGEEAAQCMIIAAAEYQQAAAIwggAIARCTMZKROLDr7Ozk7LfZKSrP/NE6lPoD0pKcnW9w2nvb3dch+Px2O5z7x58yz3mTBhQtj29PR0vfrqq/r5z3+uH374ode6srIyy/uRpBMnTlju8/3331vuw3k7vHEFBAAwggACABhh+SW42tpa7dixQ0ePHlVjY6NWr16tGTNmBNdv2bJFn376aa8+xcXFWrNmzeWPFgCQMCwHUEdHhyZMmKCSkhK99NJLYbe54YYber32bOdhcQCAxGY5GaZPn67p06f3/U1TUmy9MQsAGD5icmlSW1ur0tJSuVwuTZkyRUuXLpXb7Q67bWdnZ6+7qRwOh9LT0+VyuUL6BJYjfa/hIl7qEM072voS6fG+saiDnUd5Z2RkRG3/fUlPTw/bHnhUd7hHdtt5tLZk7+eUlpZmuU80f3bxcl6YFg91cLlcA9rO4b+Mh6cvXrw45D2gzz77TGlpacrJyVFDQ4O2b9+uESNGaMOGDWEP6srKSlVVVQWXJ06cqI0bN9odEgBgiIj6FdCtt94a/Pf48eOVn5+vRx99VF9++aWmTp0asv3ChQu1YMGC4HLgL9DZs2fr4MGDvbZ1u92qr69XXl6eWltboz30ISNe6hAPV0AnTpzQ+PHjo1YHO1dAN998c1T23Z/8/Pyw7SNGjNArr7yiRx99NORzTKWlpbb2Zeeq7t1337Xc53e/+53lPpHEy3lhWjzUobi4WDU1Nf1uF/O7A6644gq53W41NDSEDSCn0ymn0xnS7vV6IxavtbV1WB9gAabrYDqAAqJZBzsB1NbWFpV99+fSD5leqr29PWSb7u5uW/vy+XyW+3R0dFjuE4vj1/R5ES9M1sHr9Q5ou5h/Dujs2bM6f/68Ro0aFetdAQCGEMtXQO3t7WpoaAgunz59WseOHVNmZqYyMzP11ltvaebMmfJ4PPruu++0bds2jR07VsXFxVEdOABgaLMcQN98843WrVsXXK6oqJAkzZkzRytWrNCJEyf06aefyuv1avTo0Zo2bZqWLFkS9mU2AMDwZTmArr/+elVWVkZcb2rGAzuv3dt1GTcOJhQ77xPYeQ8o0q3Egfbk5OSQbey+92HnVuIvvvjCch8775dEErhh4Pjx4yHvR/3zn/+09T37OscjOX78uK19YfhiLjgAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIwggAAARhBAAAAjCCAAgBEEEADACAIIAGAEAQQAMIIAAgAYEfMnosYzZrUefHYeX/3NN9+Ebc/MzJQkZWVlhcyyfe7cOeuD08Cf5Hi58vLyLPd55ZVXwrYH/u8vvvhiyAzlgRpZNW7cOMt9Vq5caWtfiO5s/oHv5XA4wn7fePq9xxUQAMAIAggAYAQBBAAwggACABhBAAEAjCCAAABGEEAAACMIIACAEQQQAMAIAggAYAQBBAAwggACABiRMJORxtMEe4gs0sSifWlubg7bHph4s6WlRa2trb3WdXd3Wx+cTU6n03Kfr776ynIft9vd5/ri4uKQturqasv7kaRVq1ZZ7nPpRKgYODu/v6I5gakpXAEBAIwggAAARhBAAAAjCCAAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIwggAAARhBAAAAjCCAAgBEJMxkpBl9SkvW/X86dO2e5T1paWtj2lJSU4NfAvwMuXLhgeT92XXfddZb79DexaLQ88MADtvoN5mSukFwul+U+Xq83bHtgYlO/3x+1SZqtTnw60O25AgIAGEEAAQCMIIAAAEYQQAAAIwggAIARBBAAwAgCCABgBAEEADCCAAIAGEEAAQCMIIAAAEYQQAAAI5iMFLbZmejw0klDB6KtrS1se3JycnB9pG2sKiwstNznww8/jMq++7Nr166w7cnJybrjjjv04Ycfhkwi2tzcPBhDw0XsTNIbaWLRWBgxYoTlPh0dHTEYCVdAAABDCCAAgBGWXg+prq7Wvn37VF9fr9TUVBUWFmrZsmXKzc0NbnPhwgVVVFRo79696uzsVHFxsUpLS+XxeKI9dgDAEGbpCqi2tlZ33nmnNmzYoKefflrd3d16/vnn1d7eHtxm69at+s9//qPHH39c69atU2NjozZt2hT1gQMAhjZLAbRmzRrNnTtXV111lSZMmKCVK1fqzJkzOnLkiKSeN4M/+eQTLV++XFOmTNGkSZNUVlam//3vf6qrq4vJfwAAMDRd1l1wgTuPMjMzJUlHjhxRd3e3pk6dGtwmLy9P2dnZqqurC3uHUWdnpzo7O4PLDodD6enpcrlcIY8tDiwP1uOM41W81MHqY3olyel0Wu4T6fHasaiDnUcj26mDHYG7/iK1h1tv+hgZTPFyXti5C87n80Vt//3Vwc5dcKmpqZa2H+h5ZDuAfD6fXn/9dV1zzTUaP368JKmpqUkpKSkhOx85cqSamprCfp/q6mpVVVUFlydOnKiNGzeqpqYm4r7r6+vtDjuhUIcew6UOd9xxR5/rS0pKQtpaWlpiNZy4NVyOh/4MhTrYDqDy8nKdPHlS69evv6wBLFy4UAsWLAguB/6anD17tg4ePNhrW7fbrfr6euXl5am1tfWy9juUxUsd4uEKKNp1KCgosNzngw8+sNwnOzvbcp9InzdKTk5WSUmJPvnkk5DPAd17772W9zNUxct5EQ9XQH3VYTA+B1RcXKx//etf/W5nK4DKy8u1f/9+rVu3TmPGjAm2ezwedXV1yev19roKam5ujngXnNPpDPtLyev1RjyIWltbh3UABZiug+kACohmHex8INDOB3LtuDRcwq2/dJvheJ6YPi9MB1BApDpc/JbHQFkNoIGeR5Yq5ff7VV5ern379mnt2rXKycnptX7SpElKTk7WoUOHgm2nTp3SmTNnbH3CHACQuCxdAZWXl2vPnj164oknlJ6eHnxfJyMjQ6mpqcrIyFBJSYkqKiqUmZmpjIwMvfbaayosLCSAAAC9WAqgwGvdzz77bK/2srIyzZ07V5K0fPlyORwObdq0SV1dXcEPogIAcDFLAVRZWdnvNqmpqSotLSV0EJad158jTWAaaE9JSQnZxu77Mvfcc4/lPnZuKOjq6rLc56c//WnYdrfbrbNnz2rp0qXD8j2fWLLzPudgvSdo18UTB8TKQGvAXHAAACMIIACAEQQQAMAIAggAYAQBBAAwggACABhBAAEAjCCAAABGEEAAACMIIACAEQQQAMAIAggAYAQBBAAwwvYjuYHBmvU30szRgfaurq6QbXJzc23ta/HixZb72HmK6qpVqyz3iTSTeKC9s7PT1mzjiK7BOi8iPXk10J6UlBR2m1g8fdUuroAAAEYQQAAAIwggAIARBBAAwAgCCABgBAEEADCCAAIAGEEAAQCMIIAAAEYQQAAAIwggAIARBBAAwIi4nYw03ER6fU2yF08T7CG67Ey62NTUZGtfW7Zssdxn+fLllvts27bNch/0SEkJ/2sr0J6SkhKyTXd3t619DdbEonZE+p0XaPf5fHH/e5ErIACAEQQQAMAIAggAYAQBBAAwggACABhBAAEAjCCAAABGEEAAACMIIACAEQQQAMAIAggAYAQBBAAwIm4nIw03kd5QmmQP0WNn0sX29nZb+3r//fct9/n73/9uuY/dyTEhdXV19dne1dUVcRvEF66AAABGEEAAACMIIACAEQQQAMAIAggAYAQBBAAwggACABhBAAEAjCCAAABGEEAAACMIIACAEQQQAMCIuJ2MNJ45HA7Lffx+fwxGgkjs1vv777+33IeJRQF7uAICABhBAAEAjLD0Elx1dbX27dun+vp6paamqrCwUMuWLVNubm5wm2effVa1tbW9+t1+++16+OGHozNiAEBCsBRAtbW1uvPOO3X11Veru7tb27dv1/PPP6+XX35ZI0aMCG43f/58LVmyJLicmpoavREDABKCpQBas2ZNr+WVK1eqtLRUR44c0XXXXRdsT0tLk8fjicoAAQCJ6bLugmtra5MkZWZm9mqvqalRTU2NPB6PbrrpJt17771KS0sL+z06OzvV2dkZXHY4HEpPT5fL5ZLb7e61bWD50vbBZvouuHipg2l91cHOz0iSkpKsvy1q+i44joce1KFHPNTB5XINaDuH3+ZvRp/Pp9///vfyer167rnngu0fffSRsrOzNXr0aB0/flxvvPGGCgoKtHr16rDfp7KyUlVVVcHliRMnauPGjXaGBAAYQmwH0KuvvqovvvhC69ev15gxYyJud/jwYa1fv15//OMfNXbs2JD1ka6AZs+erYMHD/ba1u12q76+Xnl5eWptbbUz7KiIhyugeKiDaX3VYbhdAXE8UIeAeKhDcXGxampq+t3O1ktw5eXl2r9/v9atW9dn+EhSQUGBJKmhoSFsADmdTjmdzpB2r9cbsXitra3DOoACTNchXoSrw3AKoACOhx7UoYfJOni93gFtZ+ls8/v9Ki8v1759+7R27Vrl5OT02+fYsWOSpFGjRlnZFQAgwVm6AiovL9eePXv0xBNPKD09XU1NTZKkjIwMpaamqqGhQXv27NGNN96ozMxMnThxQlu3btW1116r/Pz8WIwfADBEWQqgDz74QFLPh00vVlZWprlz5yolJUWHDh3Szp071dHRoTFjxmjmzJlatGhR1AYMAEgMlgKosrKyz/XZ2dlat27dZQ0IADA8MBu2DYM1s3VycnKf7cnJySHbxMsb4tEU6YaCQLvD4QjZJj093da+In1erS+NjY229gUMd0xGCgAwggACABhBAAEAjCCAAABGEEAAACMIIACAEQQQAMAIAggAYAQBBAAwggACABhBAAEAjCCAAABGMBlpHIs0sWigvbu7OyEnH71UpMlfA+1+vz9km7a2Nlv7stsP9sTL04WHk0sfpxOrPgPBFRAAwAgCCABgBAEEADCCAAIAGEEAAQCMIIAAAEYQQAAAIwggAIARBBAAwAgCCABgBAEEADAibueCKyoqCmlzuVySpOLiYnm93sEeUtygDj2oQ4+hXIdozgU3lOsQTf3VYezYsZa/5/Tp0y1tH+73dzgOPzP7AQAMGFIvwf3www968skn9cMPP5geilHUoQd16EEdelCHHkOpDkMqgPx+v44ePTrsp2OnDj2oQw/q0IM69BhKdRhSAQQASBwEEADAiCEVQE6nU/fdd5+cTqfpoRhFHXpQhx7UoQd16DGU6sBdcAAAI4bUFRAAIHEQQAAAIwggAIARBBAAwIi4nQvuUrt27dK7776rpqYm5efn68EHH1RBQYHpYQ2qyspKVVVV9WrLzc3V5s2bzQxokNTW1mrHjh06evSoGhsbtXr1as2YMSO43u/3q7KyUh9//LG8Xq+KiopUWlqqcePGGRx19PVXhy1btujTTz/t1ae4uFhr1qwZ7KHGTHV1tfbt26f6+nqlpqaqsLBQy5YtU25ubnCbCxcuqKKiQnv37lVnZ6eKi4tVWloqj8djbuBRNpA6PPvss6qtre3V7/bbb9fDDz882MONaEgE0N69e1VRUaEVK1Zo8uTJ+sc//qENGzZo8+bNGjlypOnhDaqrrrpKzzzzTHA5KSnxL2I7Ojo0YcIElZSU6KWXXgpZ/8477+j999/XypUrlZOTo7/97W/asGGDXn75ZaWmphoYcWz0VwdJuuGGG1RWVhZcTkkZEqf4gNXW1urOO+/U1Vdfre7ubm3fvl3PP/+8Xn75ZY0YMUKStHXrVu3fv1+PP/64MjIyVF5erk2bNum5554zPProGUgdJGn+/PlasmRJcDnezochcXS+9957mj9/vubNmydJWrFihfbv36/du3frnnvuMTu4QZaUlJRQf8kNxPTp0yPOxuv3+7Vz504tWrRIt9xyiyRp1apVWrFihT7//HPdeuutgznUmOqrDgEpKSkJfXxcejW3cuVKlZaW6siRI7ruuuvU1tamTz75RI899pimTJkiSSorK9OvfvUr1dXVqbCw0MSwo66/OgSkpaXF9fEQ9wHU1dWlI0eO9AqapKQkTZ06VXV1deYGZkhDQ4MeeeQROZ1OFRYW6v7771d2drbpYRlz+vRpNTU1adq0acG2jIwMFRQUqK6uLqECaCBqa2tVWloql8ulKVOmaOnSpXK73aaHFTNtbW2SpMzMTEnSkSNH1N3dralTpwa3ycvLU3Z2dkIF0KUurUNATU2Nampq5PF4dNNNN+nee+9VWlqaiSGGFfcB1NLSIp/PF5LiHo9Hp06dMjMoQyZPnqyysjLl5uaqsbFRVVVVWrt2rTZt2qT09HTTwzOiqalJkkJeih05cmRw3XBxww03aObMmcrJyVFDQ4O2b9+uF154QRs2bEjIl2p9Pp9ef/11XXPNNRo/fryknuMhJSUl+EycgEQ+HsLVQZJmzZql7OxsjR49WsePH9cbb7yhU6dOafXq1QZH21vcBxD+38Uvv+Tn5wcD6d///rdKSkoMjgzx4OKrvfHjxys/P1+PPvqovvzyy15XBImivLxcJ0+e1Pr1600PxahIdbj99tuD/x4/frxGjRql9evXq6GhwdZD6WIh7v8sysrKUlJSUshfL01NTXH92uZgcLlcys3NVUNDg+mhGBM4Bpqbm3u1Nzc3D/vj44orrpDb7U7I46O8vFz79+/Xb3/7W40ZMybY7vF41NXVFfIk0EQ9HiLVIZzAXcPxdDzEfQClpKRo0qRJOnz4cLDN5/Pp8OHDCft67kC1t7eroaEhIU+sgcrJyZHH49GhQ4eCbW1tbfr666+H/fFx9uxZnT9/XqNGjTI9lKjx+/0qLy/Xvn37tHbtWuXk5PRaP2nSJCUnJ/c6Hk6dOqUzZ84k1PHQXx3COXbsmCTF1fEwJF6CW7BggbZs2aJJkyapoKBAO3fuVEdHh+bOnWt6aIOqoqJCN998s7Kzs9XY2KjKykolJSVp1qxZpocWU4GgDTh9+rSOHTumzMxMZWdn6+6779bbb7+tcePGKScnR2+++aZGjRoVvCsuUfRVh8zMTL311luaOXOmPB6PvvvuO23btk1jx45VcXGxwVFHV3l5ufbs2aMnnnhC6enpwVdGMjIylJqaqoyMDJWUlKiiokKZmZnKyMjQa6+9psLCwoQKoP7q0NDQoD179ujGG29UZmamTpw4oa1bt+raa69Vfn6+2cFfZMjMhr1r1y7t2LFDTU1NmjBhgh544AFNnjzZ9LAG1ebNm/XVV1+ptbVVWVlZKioq0tKlS+Pm9dxY+fLLL7Vu3bqQ9jlz5mjlypXBD6J+9NFHamtrU1FRkR566KFeH8pLBH3VYcWKFXrxxRd19OhReb1ejR49WtOmTdOSJUsS6gp58eLFYdvLysqCf5AGPoj62WefqaurKyE/iNpfHc6cOaNXXnlFJ0+eVEdHh8aMGaMZM2Zo0aJFysjIGOTRRjZkAggAkFji/j0gAEBiIoAAAEYQQAAAIwggAIARBBAAwAgCCABgBAEEADCCAAIAGEEAAQCMIIAAAEYQQAAAIwggAIAR/wfe5mDISwilJQAAAABJRU5ErkJggg==\n"
          },
          "metadata": {}
        },
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 640x480 with 1 Axes>"
            ],
            "image/png": "iVBORw0KGgoAAAANSUhEUgAAAaAAAAGdCAYAAABU0qcqAAAAOnRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjEwLjAsIGh0dHBzOi8vbWF0cGxvdGxpYi5vcmcvlHJYcgAAAAlwSFlzAAAPYQAAD2EBqD+naQAAI9dJREFUeJzt3X9sVfX9x/HXLb0t7b23XGhToRVaEEp1QCkobCoRQYdxLBN0QgyJmxbNimbTMExAURA0zGGYC1uWpUY6DVo6iejQEX+EFDGBCGNAVaKAYtkFkV5abmnpj/v94+ber5feW3ou9/Zzb/t8/APnc87nnk/fPfe+eu4993Nsfr/fLwAA+lia6QEAAAYmAggAYAQBBAAwggACABhBAAEAjCCAAABGEEAAACMIIACAEQQQAMAIAggAYES66QFEc9999+nzzz8Pa3M4HKqrq9OMGTPk8/kMjcw86hBAHQKoQ0Ai6mCz2Sz3MT27WSLqsG/fvrg8zqVsiZoL7r333tPbb78tr9eroqIiPfDAAxo7dmyv+0+ZMkX79+8Pa3O5XGpqalJOTo6am5vjPeSUQR0CqEMAdQhIRB1SMYASUYdE/UwJeQtu9+7dqq6u1j333KN169apqKhIa9eu1blz5xKxOwBACkpIAL3zzjuaPXu2br31Vl199dVavHixMjIy9NFHHyVidwCAFBT3z4A6Ojp09OhR3XXXXaG2tLQ0TZw4UUeOHOm2fXt7u9rb20PLNptNWVlZcjgccrlcYdsGly9tH2ioQwB1CKAOAYmoQ6q+BffDf5NZ3AOoqalJXV1dcrvdYe1ut1snT57stv3WrVtVW1sbWh49erTWrVunurq6qPtoaGiI23hTGXUIoA4B1CGAOgSkQh2MXwU3b948zZ07N7Qc/ItjxowZOnDgQNi2LpdLDQ0NKiwsHPAftlIH6hBEHQISUYdUPQOKdx2ampri8jiXinsA5eTkKC0tTV6vN6zd6/V2OyuSJLvdLrvd3q3d5/NFLV5zc/OAfqIFUYcA6hBAHQLiWYdUDKCgVDge4n4RQnp6usaMGaNDhw6F2rq6unTo0CGVlJTEe3cAgBSVkLfg5s6dq40bN2rMmDEaO3astm/frra2Ns2cOTMRuwMApKCEBNCNN96opqYm1dTUyOv1qri4WMuXL4/4FhwAYGBK2EUId9xxh+64445EPTwAJFyyfJ5jmtXPwsrLy3s1fQ+TkQIAjCCAAABGEEAAACMIIACAEQQQAMAIAggAYAQBBAAwggACABhBAAEAjCCAAABGEEAAACMIIACAEcbviAoMRD/96U8t99mxY4flPnv27LHcR5KmTZsWUz/ACs6AAABGEEAAACMIIACAEQQQAMAIAggAYAQBBAAwggACABhBAAEAjCCAAABGEEAAACMIIACAEQQQAMAIAggAYASzYSexrKysHtuzsrLU0dERtu7ChQsJHxeu3N69ey33yc3NjdjudDolScOGDVNGRkbYOma1RjLjDAgAYAQBBAAwggACABhBAAEAjCCAAABGEEAAACMIIACAEQQQAMAIAggAYAQBBAAwggACABhBAAEAjGAy0iQWbWLR9PT00PpUm3w0OHYrLp1wNZHS0qz/TdbV1WW5T2trq+U+0X7XFy9elCSdPXtWzc3Nlh8XMIUzIACAEQQQAMAIAggAYAQBBAAwggACABhBAAEAjCCAAABGEEAAACMIIACAEQQQAMAIAggAYAQBBAAwgslI0af6cmLRWPj9fst9iouLLfd59NFHLfd5/vnnI7Y7nU5JUm5urjIzM8PWnTlzxvJ+gL7CGRAAwAgCCABgRNzfgqupqVFtbW1YW0FBgTZs2BDvXQEAUlhCPgMaOXKknnrqqdByLDf5AgD0bwkJoLS0NLnd7kQ8NACgn0hIAHk8Hj388MOy2+0qKSnRfffdp7y8vIjbtre3q729PbRss9mUlZUlh8Mhl8sVtm1w+dL2gYY6BCSiDjabzXKf4FVoVmRkZMRtP8H2SOvb2tos7ydV8bwISIY6OByOXm1n88dy3WkP9u/fr9bWVhUUFKixsVG1tbU6e/as1q9fr6ysrG7bX/qZ0ejRo7Vu3bp4DgkAkITiHkCX8vl8qqys1P33369Zs2Z1Wx/tDGjGjBk6cOBA2LYul0sNDQ0qLCxUc3NzIoed1KhDQCLqEMsZ0KhRoyz3efjhhy33Wb9+fcR2p9OpgwcPauLEiTp//nzYuu+//97yflIVz4uAZKhDWVmZ6urqLrtdwr+I6nA4VFBQII/HE3G93W6X3W7v1u7z+aIWr7m5eUAfYEHUISCedYglgC590e+Nixcvxn0/58+f77bNQDw+eF4EmKyDz+fr1XYJvzyttbVVHo+HixIAAGHifgZUXV2t66+/Xnl5eWpsbFRNTY3S0tJ08803x3tXAIAUFvcAOnv2rP70pz+publZOTk5Ki0t1dq1a5WTkxPvXQEAUljcA+h3v/tdvB8S/Ugsb8V6vV7LfWbPnm25jySVlJRY7vOXv/zFcp9Yrv253HPrq6++ist+pNhq/u9//9tynx9+Yb23jh49arkPkhNTFAAAjCCAAABGEEAAACMIIACAEQQQAMAIAggAYAQBBAAwggACABhBAAEAjCCAAABGEEAAACMIIACAEQm/IR2S3y233BJTv++++85yn/r6est9RowYEbHd6XRKkoYPHx76f9CMGTMs70eKbTLdWG5Il5mZablPenrPT9dIN9OL5cZ3krRx40bLfdLSrP8925u7Zl4q2oSxDocj9G9XV1fYut7eIA19izMgAIARBBAAwAgCCABgBAEEADCCAAIAGEEAAQCMIIAAAEYQQAAAIwggAIARBBAAwAgCCABgBAEEADCCAAIAGMFs2NDOnTtj6vf8889b7vPkk09a7vP2229HbA/OvvzGG290m/146tSplvcjSYcOHbLcp7Gx0XKfV1991XKfnmaoTk9PV2dnZ7f2WGeBfvrppy33iWU27Fhn67bK7XbH1C/SDOOXE8vxMFBxBgQAMIIAAgAYQQABAIwggAAARhBAAAAjCCAAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIwggAAARjAZaT8Ty+SJfr8/pn2NGzfOcp9YJp+83M80efJky48ZzYQJE+L2WD358Y9/bLlPenrPT9dI63Nzcy3vR5J27Nhhuc/PfvYzy31eeukly33Onj3b4/qvv/66W9snn3xieT+SdPvtt8fUD73DGRAAwAgCCABgBAEEADCCAAIAGEEAAQCMIIAAAEYQQAAAIwggAIARBBAAwAgCCABgBAEEADCCAAIAGMFkpP3Mvffea7lPeXl5TPu6++67Y+oXb7FMwGqa3W43PYQe1dfXW+7zk5/8xHKfVatWWe6TlZUVsd3lcsnr9So/P1/Nzc1h686dO2d5P5LU1tZmuU8sx2MqHsPxwBkQAMAIAggAYITlt+Dq6+u1bds2HTt2TI2NjVq6dKmmTZsWWu/3+1VTU6MPPvhAPp9PpaWlqqio0IgRI+I6cABAarN8BtTW1qbi4mI9+OCDEde/9dZbevfdd7V48WI999xzyszM1Nq1a2O6ERkAoP+yHEDl5eVauHBh2FlPkN/v1/bt2zV//nzdcMMNKioq0iOPPKLGxkbt3bs3LgMGAPQPcb0K7vTp0/J6vZo0aVKoLTs7W2PHjtWRI0d00003devT3t6u9vb20LLNZlNWVpYcDodcLlfYtsHlS9sHmp7qEO0KoZ5kZGRc8ZjQv8RyTGRnZ1vuM2jQIMt9oj3/U/n1IZ5jToY6OByOXm0X1wDyer2SpCFDhoS1DxkyJLTuUlu3blVtbW1oefTo0Vq3bp3q6uqi7qehoeGKx9ofUAckym9+85s+6ROLaK8lQSdOnOiTccRTU1NT3B8zFV4fjH8PaN68eZo7d25oOXg9/IwZM3TgwIGwbV0ulxoaGlRYWNjtOv+BpKc6xPLdnB+esVrx2GOPxdQPye+vf/2r5T7btm2z3Gf58uWW+/z85z+P2O5yuXTixAmNHDmy2/Pi22+/tbwfqfd/yf9QLN/pycnJsdwnmmR4nSwrK+vxJCIorgHkdrslBb70NXTo0FD7uXPnVFxcHLGP3W6P+KU8n88XtXjNzc0DOoCCItXhwoULlh+HC0RwqViOiZaWFst9Ojs7Lfe53HM/FV8fEjFek3Xw+Xy92i6u3wPKz8+X2+3WwYMHQ20tLS368ssvVVJSEs9dAQBSnOUzoNbWVnk8ntDy6dOndfz4cTmdTuXl5enOO+/Um2++qREjRig/P1+vv/66hg4dqhtuuCGuAwcApDbLAfTVV1+Fzd9UXV0tSbrlllu0ZMkS/eIXv1BbW5v+9re/qaWlRaWlpVq+fDlXWgEAwtj8fr/f9CAimTJlivbv3x/W5nK51NTUpJycnJR7jzeWySfHjBkTsd3hcOjTTz/V1KlTu73X+o9//MPyfjg77b9680FwJJG+53c5mZmZlvvEMunp7bffHrHd6XTqiy++0Pjx43X+/PmwdYsWLbK8H0las2aN5T6xPNe3bNliuU+0iYeT4XWyvLxc+/btu+x2zAUHADCCAAIAGEEAAQCMIIAAAEYQQAAAIwggAIARBBAAwAgCCABgBAEEADCCAAIAGEEAAQCMIIAAAEYQQAAAI4zfknugaG9vt9znf//7X8R2l8slSTp16lS32W7748zW0WZMTktLU2lpqT7//HN1dXWFrUtPj+3Qbm1ttdznxhtvtNynt3eM/KFotzRxuVw6c+aM8vLyuh0Pv/zlLy3vR5KmT59uuU8sE+tPnDjRcp+0tMh/NwfvBvzdd991q8Pvf/97y/uRYpvZOhax/J6uvvrqiO1Op1OSVFhY2G1WcCn225MnAmdAAAAjCCAAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIwggAAARhBAAAAjCCAAgBEEEADACAIIAGAEk5EmsTlz5kRsz8rKkiTNnj07NAGjCZ2dnZb7RJtAsScejydiu8vlUlNTk6ZNm9Zt8sn+6OLFiz22X7x4sds2mzdvjmlfW7Zssdwnlgl3Y5nA9NKJZ4M6OjpC/wb/H5SXl2d5P8ku2qSiwcmKGxoakv55wRkQAMAIAggAYAQBBAAwggACABhBAAEAjCCAAABGEEAAACMIIACAEQQQAMAIAggAYAQBBAAwggACABjBZKRJbPTo0RHbMzMzJUlFRUVqa2vryyGFWb58ueU+0SYWRWJEm7jzcqJNfJoMcnNzI7Y7nU5J0rBhw5SRkRG2LpaJUiXJbrfH1M+qJ554ok/2k2w4AwIAGEEAAQCMIIAAAEYQQAAAIwggAIARBBAAwAgCCABgBAEEADCCAAIAGEEAAQCMIIAAAEYQQAAAI2x+v99vehCRTJkyRfv37w9rc7lcampqUk5Ojpqbmw2NrO8k6a8mZMKECZb7HD58OG77H2jHQzQDrQ6XTjQa5HK5dObMGeXl5XWrg8lJe3sjlkljo02U6nK55PV65Xa7Ix4PsU5Qa0V5ebn27dt32e04AwIAGEEAAQCMsHw/oPr6em3btk3Hjh1TY2Ojli5dqmnTpoXWb9y4UTt37gzrU1ZWphUrVlz5aAEA/YblAGpra1NxcbFmzZqlP/7xjxG3mTx5siorK/9/J+nc9w4AEM5yMpSXl6u8vLznB01Pl9vtjnVMAIABICGnJvX19aqoqJDD4dCECRO0cOFCuVyuiNu2t7eH3S7XZrMpKytLDoejW5/gcrTHQt9yOByW+8Tzd8fxEDDQ6hDtKrjgLbmD//Z30X7flzse+uIquN6+NlzRZdj33ntvt8+APv74Y2VmZio/P18ej0ebN2/W4MGDtXbtWqWldb/moaamRrW1taHl0aNHa926dbEOCQCQIuJ+BnTTTTeF/j9q1CgVFRXp0Ucf1eHDhzVx4sRu28+bN09z584NLdtsNknSjBkzdODAgbBtXS6XGhoaVFhYOCC+79DU1GR6CD2aPn265T6fffZZ3PY/0I6HaAZaHXo6Azp+/LiKi4t1/vz5sHVnzpzpi6HFLJazkmHDhkVsd7lcOnHihEaOHGnse0BlZWWqq6u77HYJvzrgqquuksvlksfjiRhAdrs94heqfD5f1CdTc3PzgHiiJTufz2e5TyJ+bxwPAQOlDtECKOj8+fMDog6X+xmjHQ99EUC9fW1I+PeAvv/+e50/f15Dhw5N9K4AACnE8hlQa2urPB5PaPn06dM6fvy4nE6nnE6ntmzZounTp8vtduvUqVN69dVXNXz4cJWVlcV14ACA1GY5gL766iutWrUqtFxdXS1JuuWWW7R48WJ988032rlzp3w+n4YNG6ZJkyZpwYIFUectAgAMTJYD6Ec/+pFqamqirmfGg8iCF1f0J5MmTbLcJ56TkWJgunDhQo/rT58+3UcjiZ9IVwhfTrTPcoLtXV1dffJ5z5VgLjgAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIwggAAARhBAAAAjCCAAgBEEEADACAIIAGAEAQQAMIIAAgAYkfA7oiLA7/ebHkLcBW/FYcXmzZsTMBKz4jmTcbJ44YUXLPdZs2aN5T5er9dyn8s9l1Jx5vlUHHM8cAYEADCCAAIAGEEAAQCMIIAAAEYQQAAAIwggAIARBBAAwAgCCABgBAEEADCCAAIAGEEAAQCMIIAAAEYwGWkS62nSRZvNFnF9X05qmJ5u/fA5ePCg5T6DBg2K2B6cBHTv3r3dJvcsLS21vB8ptkk4KyoqLPdpbm623GfUqFE9rj937ly3tr48HpYuXdon+7nczzRQJvYcPHhwj+2DBw9We3t7t/WdnZ2W9xXpceKBMyAAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIwggAAARhBAAAAjCCAAgBEEEADACAIIAGAEAQQAMILJSJPYt99+G7HdZrPp6quvVkNDQ7cJSQsLCy3vJzipZ1+YMGFC3B9z/PjxcXusZcuWxe2xejJs2LC4P+ZAmYSzL7W0tFjuk5ubm4CRdNfa2hqx3W63h9ZH2yZZcAYEADCCAAIAGEEAAQCMIIAAAEYQQAAAIwggAIARBBAAwAgCCABgBAEEADCCAAIAGEEAAQCMIIAAAEYM6MlIp06dGlO/Tz/9NM4jiWzUqFER210ul5qamnTdddepubk5bF0sE4t2dnbGND7gSnk8Hst9HA5H1HUul6vbc0KSFixYYHk/krRjxw7LfXg+9R5nQAAAIwggAIARlt6C27p1q/bs2aOGhgZlZGSopKREixYtUkFBQWibixcvqrq6Wrt371Z7e7vKyspUUVEht9sd77EDAFKYpTOg+vp6zZkzR2vXrtWTTz6pzs5OrVmzJuymR5s2bdKnn36qxx9/XKtWrVJjY6PWr18f94EDAFKbpQBasWKFZs6cqZEjR6q4uFhLlizRmTNndPToUUmBuwd++OGHuv/++zVhwgSNGTNGlZWV+uKLL3TkyJGE/AAAgNR0RVfBBW9X63Q6JUlHjx5VZ2enJk6cGNqmsLBQeXl5OnLkiEpKSro9Rnt7u9rb20PLNptNWVlZcjgccrlcYdsGly9tj1VPV9P0JF77j1VPdejL22sDV6qvbiOelZUVU79Ynuumr4KL9+tkLHr72hpzAHV1demVV17R+PHjQ5cLe71epaend9v5kCFD5PV6Iz7O1q1bVVtbG1oePXq01q1bp7q6uqj7bmhoiHXY/Qp1QKq76qqr4v6YkV54//nPf8Z9P8kuFV4fYg6gqqoqnThxQqtXr76iAcybN09z584NLQf/IpoxY4YOHDgQtq3L5VJDQ4MKCwsjXutvVXl5eUz99u/ff8X7vhI91SGWM6BofxwAiXbq1CnLfbKzs6Oui/Y9oF/96leW9yNJH374oeU+yXAGFM/XyViUlZX1eBIRFFMAVVVVad++fVq1apVyc3ND7W63Wx0dHfL5fGFnQefOnYt6FZzdbpfdbu/W7vP5ohavubk5LoX1+Xwx9TP1S71UpDrwFhxSid/v75P9XLhwIaZ+sTzXTQdQULxeJ2PR29dWS69Wfr9fVVVV2rNnj1auXKn8/Pyw9WPGjNGgQYN08ODBUNvJkyd15syZiJ//AAAGLktnQFVVVdq1a5eWLVumrKys0Fs32dnZysjIUHZ2tmbNmqXq6mo5nU5lZ2fr5ZdfVklJCQEEAAhjKYCC8yI988wzYe2VlZWaOXOmJOn++++XzWbT+vXr1dHREfoiKgAAP2QpgGpqai67TUZGhioqKlIidPpqUtG+1NXVZblPTx/q9uS///2v5T55eXmW+zCLRt+L5bMZ058/BifpNfnhO6zhE2sAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIwggAAARhBAAAAjCCAAgBEEEADACAIIAGAEAQQAMIIAAgAYEfMtudF/xHq3yHHjxsV5JNYEZz/OyckZ0LMf91SHSHcb7o329vZ4DA3oEWdAAAAjCCAAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIwggAAARhBAAAAjCCAAgBEEEADACAIIAGAEk5EC/RiTiqaGQYMGWe7T2dmZgJH0Lc6AAABGEEAAACMIIACAEQQQAMAIAggAYAQBBAAwggACABhBAAEAjCCAAABGEEAAACMIIACAEQQQAMAIJiMFAMNimVg0IyOjx/aMjIyI21y8eNHyvhKFMyAAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIwggAAARhBAAAAjCCAAgBEEEADACAIIAGAEAQQAMILJSAF0M2jQIMt9YplQE7GLNqlosP3ixYtJNfFoJJwBAQCMIIAAAEZYegtu69at2rNnjxoaGpSRkaGSkhItWrRIBQUFoW2eeeYZ1dfXh/W77bbb9NBDD8VnxACAfsFSANXX12vOnDm65ppr1NnZqc2bN2vNmjV68cUXNXjw4NB2s2fP1oIFC0LL0W6cBAAYuCwF0IoVK8KWlyxZooqKCh09elTXXXddqD0zM1NutzsuAwQA9E9XdBVcS0uLJMnpdIa119XVqa6uTm63W1OnTtXdd9+tzMzMiI/R3t6u9vb20LLNZlNWVpYcDodcLlfYtsHlS9sHGuoQQB0CElGHVLwKjuMhIBnq4HA4erWdze/3+2PZQVdXl/7whz/I5/Pp2WefDbW///77ysvL07Bhw/T111/rtdde09ixY7V06dKIj1NTU6Pa2trQ8ujRo7Vu3bpYhgQASCExB9Df//53/ec//9Hq1auVm5sbdbtDhw5p9erVeumllzR8+PBu66OdAc2YMUMHDhwI29blcqmhoUGFhYVqbm6OZdj9AnUIoA4BiahDqp4BcTwkRx3KyspUV1d32e1ieguuqqpK+/bt06pVq3oMH0kaO3asJMnj8UQMILvdLrvd3q3d5/NFLV5zc/OAPsCCqEMAdQiIZx1SMYCCOB4CTNbB5/P1ajtL3wPy+/2qqqrSnj17tHLlSuXn51+2z/HjxyVJQ4cOtbIrAEA/Z+kMqKqqSrt27dKyZcuUlZUlr9crScrOzlZGRoY8Ho927dqlKVOmyOl06ptvvtGmTZt07bXXqqioKBHjBwCkKEsBtGPHDkmBL5v+UGVlpWbOnKn09HQdPHhQ27dvV1tbm3JzczV9+nTNnz8/bgMGAPQPlgKopqamx/V5eXlatWrVFQ0IADAwMBs2gG6S5YIC9G9MRgoAMIIAAgAYQQABAIwggAAARhBAAAAjCCAAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIwggAAARjAZaT9TWlpquc/nn3+egJEg3mw2W4/tNput2zZ+vz/h4wJixRkQAMAIAggAYAQBBAAwggACABhBAAEAjCCAAABGEEAAACMIIACAEQQQAMAIAggAYAQBBAAwImnngos0p5nD4ZAklZWVyefz9fWQkkZPdRg9erTlx8vKyorLuPraQDseos0F11MdBtJccAPteIgmGerQ2zkpbf6BdIQCAJJGSr0Fd+HCBT3xxBO6cOGC6aEYRR0CqEMAdQigDgGpVIeUCiC/369jx44NqLcVIqEOAdQhgDoEUIeAVKpDSgUQAKD/IIAAAEakVADZ7Xbdc889stvtpodiFHUIoA4B1CGAOgSkUh24Cg4AYERKnQEBAPoPAggAYAQBBAAwggACABiRtHPBXeq9997T22+/La/Xq6KiIj3wwAMaO3as6WH1qZqaGtXW1oa1FRQUaMOGDWYG1Efq6+u1bds2HTt2TI2NjVq6dKmmTZsWWu/3+1VTU6MPPvhAPp9PpaWlqqio0IgRIwyOOv4uV4eNGzdq586dYX3Kysq0YsWKvh5qwmzdulV79uxRQ0ODMjIyVFJSokWLFqmgoCC0zcWLF1VdXa3du3ervb1dZWVlqqiokNvtNjfwOOtNHZ555hnV19eH9bvtttv00EMP9fVwo0qJANq9e7eqq6u1ePFijRs3Tv/617+0du1abdiwQUOGDDE9vD41cuRIPfXUU6HltLT+fxLb1tam4uJizZo1S3/84x+7rX/rrbf07rvvasmSJcrPz9cbb7yhtWvX6sUXX1RGRoaBESfG5eogSZMnT1ZlZWVoOT09JZ7ivVZfX685c+bommuuUWdnpzZv3qw1a9boxRdf1ODBgyVJmzZt0r59+/T4448rOztbVVVVWr9+vZ599lnDo4+f3tRBkmbPnq0FCxaElpPt+ZASR+c777yj2bNn69Zbb5UkLV68WPv27dNHH32ku+66y+zg+lhaWlq/+kuuN8rLy1VeXh5xnd/v1/bt2zV//nzdcMMNkqRHHnlEixcv1t69e3XTTTf15VATqqc6BKWnp/fr4+PSs7klS5aooqJCR48e1XXXXaeWlhZ9+OGH+u1vf6sJEyZIkiorK/XYY4/pyJEjKikpMTHsuLtcHYIyMzOT+nhI+gDq6OjQ0aNHw4ImLS1NEydO1JEjR8wNzBCPx6OHH35YdrtdJSUluu+++5SXl2d6WMacPn1aXq9XkyZNCrVlZ2dr7NixOnLkSL8KoN6or69XRUWFHA6HJkyYoIULF8rlcpkeVsK0tLRIkpxOpyTp6NGj6uzs1MSJE0PbFBYWKi8vr18F0KUurUNQXV2d6urq5Ha7NXXqVN19993KzMw0McSIkj6Ampqa1NXV1S3F3W63Tp48aWZQhowbN06VlZUqKChQY2OjamtrtXLlSq1fvz5l7+lzpbxeryR1eyt2yJAhoXUDxeTJkzV9+nTl5+fL4/Fo8+bNeu6557R27dp++VZtV1eXXnnlFY0fP16jRo2SFDge0tPTQ/fECerPx0OkOkjSzTffrLy8PA0bNkxff/21XnvtNZ08eVJLly41ONpwSR9A+H8/fPulqKgoFEiffPKJZs2aZXBkSAY/PNsbNWqUioqK9Oijj+rw4cNhZwT9RVVVlU6cOKHVq1ebHopR0epw2223hf4/atQoDR06VKtXr5bH49Hw4cP7epgRJf2fRTk5OUpLS+v214vX603q9zb7gsPhUEFBgTwej+mhGBM8Bs6dOxfWfu7cuQF/fFx11VVyuVz98vioqqrSvn379PTTTys3NzfU7na71dHR0e1OoP31eIhWh0iCVw0n0/GQ9AGUnp6uMWPG6NChQ6G2rq4uHTp0qN++n9tbra2t8ng8/fKJ1Vv5+flyu906ePBgqK2lpUVffvnlgD8+vv/+e50/f15Dhw41PZS48fv9qqqq0p49e7Ry5Url5+eHrR8zZowGDRoUdjycPHlSZ86c6VfHw+XqEMnx48clKamOh5R4C27u3LnauHGjxowZo7Fjx2r79u1qa2vTzJkzTQ+tT1VXV+v6669XXl6eGhsbVVNTo7S0NN18882mh5ZQwaANOn36tI4fPy6n06m8vDzdeeedevPNNzVixAjl5+fr9ddf19ChQ0NXxfUXPdXB6XRqy5Ytmj59utxut06dOqVXX31Vw4cPV1lZmcFRx1dVVZV27dqlZcuWKSsrK/TOSHZ2tjIyMpSdna1Zs2apurpaTqdT2dnZevnll1VSUtKvAuhydfB4PNq1a5emTJkip9Opb775Rps2bdK1116roqIis4P/gZSZDfu9997Ttm3b5PV6VVxcrF//+tcaN26c6WH1qQ0bNuizzz5Tc3OzcnJyVFpaqoULFybN+7mJcvjwYa1atapb+y233KIlS5aEvoj6/vvvq6WlRaWlpXrwwQfDvpTXH/RUh8WLF+uFF17QsWPH5PP5NGzYME2aNEkLFizoV2fI9957b8T2ysrK0B+kwS+ifvzxx+ro6OiXX0S9XB3OnDmjP//5zzpx4oTa2tqUm5uradOmaf78+crOzu7j0UaXMgEEAOhfkv4zIABA/0QAAQCMIIAAAEYQQAAAIwggAIARBBAAwAgCCABgBAEEADCCAAIAGEEAAQCMIIAAAEYQQAAAI/4P1VZou10QQKsAAAAASUVORK5CYII=\n"
          },
          "metadata": {}
        },
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 640x480 with 1 Axes>"
            ],
            "image/png": "iVBORw0KGgoAAAANSUhEUgAAAaAAAAGdCAYAAABU0qcqAAAAOnRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjEwLjAsIGh0dHBzOi8vbWF0cGxvdGxpYi5vcmcvlHJYcgAAAAlwSFlzAAAPYQAAD2EBqD+naQAAIHpJREFUeJzt3X9sVfX9x/HXLb2F9t4LF2gqtEoLlloVqKjAohIRzHSOZII/IM7Mqa0urbpJiEtkMkFqwiaE/SCL2Wqk0eFqJxMVCfFHSBGXmqEE6LZGC8pKKkFbKLe09Mf9/nHT++XSW+i53vZ9b/t8/CPnc87nno9vz/XF59xzP9cVDAaDAgBgiKVYDwAAMDIRQAAAEwQQAMAEAQQAMEEAAQBMEEAAABMEEADABAEEADBBAAEATBBAAAATqdYD6M99992n//znPxFtHo9HNTU1mj9/vgKBgNHI7FGHEOoQQh1CqENIItShsLBQf/3rXy96nGuw1oLbuXOn3nrrLbW0tCg3N1cPPfSQ8vPzB9z/2muv1aeffhrR5vP5dOrUKY0dO1atra3xHnLSoA4h1CGEOoRQh5BEqMPs2bO1b9++ix43KLfg9u7dq8rKSt19991av369cnNzVV5erpMnTw7G6QAASWhQAujtt9/WokWLdMstt+jSSy9VSUmJ0tLS9OGHHw7G6QAASSjunwF1dXWpoaFBd955Z7gtJSVFM2fOVH19fZ/jOzs71dnZGd52uVxKT0+Xx+ORz+eLOLZ3+/z2kYY6hFCHEOoQQh1CEqEOHo9nQMfFPYBOnTqlnp4e+f3+iHa/369jx471OX7btm2qrq4Ob0+dOlXr169XTU1Nv+dobGyM23iTGXUIoQ4h1CGEOoQkQx3Mn4JbsmSJFi9eHN52uVySpPnz52v//v0Rx/p8PjU2NionJ2fEf8hIHahDL+oQQh1CEqEORUVFF5xE9Ip7AI0dO1YpKSlqaWmJaG9paekzK5Ikt9stt9vdpz0QCPRbvNbW1hF9gfWiDiHUIYQ6hFCHEMs6DPTx77g/hJCamqpp06bp4MGD4baenh4dPHhQBQUF8T4dACBJDcotuMWLF2vz5s2aNm2a8vPztWPHDnV0dGjBggWDcToAQBIalAC64YYbdOrUKVVVVamlpUV5eXl6+umno96CAwCMTIP2EMLtt9+u22+/Peb+KSkpSklJ6dPW376enp6YzwUAGHosRgoAMEEAAQBMEEAAABMEEADABAEEADBBAAEATBBAAAATBBAAwAQBBAAwQQABAEwQQAAAEwQQAMCE+S+i9qenp6fPAqO929H2AQCSCzMgAIAJAggAYIIAAgCYIIAAACYIIACACQIIAGCCAAIAmCCAAAAmCCAAgAkCCABgggACAJgggAAAJgggAIAJAggAYIIAAgCYIIAAACYIIACACQIIAGCCAAIAmCCAAAAmCCAAgAkCCABgggACAJgggAAAJgggAIAJAggAYCLVegAAMNKNGjXKcZ/u7u5BGMnQYgYEADBBAAEATBBAAAATBBAAwAQBBAAwQQABAEwQQAAAEwQQAMAEAQQAMEEAAQBMEEAAABMEEADABIuRAoCx4bCwaCyYAQEATBBAAAATcb8FV1VVperq6oi27Oxsbdq0Kd6nAgAksUH5DOiyyy7TM888E95OSWGiBQCINCgBlJKSIr/fPxgvDQAYJgYlgJqamvToo4/K7XaroKBA9913nzIzM6Me29nZqc7OzvC2y+VSenq6PB6PfD5fxLG92+e3jzTUIYQ6hFCHEOoQkgh18Hg8AzrOFQwGg/E88aeffqr29nZlZ2erublZ1dXV+vbbb7Vhwwalp6f3Of78z4ymTp2q9evXx3NIAIAEFPcAOl8gEFBpaakeeOABLVy4sM/+/mZA8+fP1/79+yOO9fl8amxsVE5OjlpbWwdz2AmNOoRQhxDqEEIdQhKhDkVFRaqpqbnocYP+RVSPx6Ps7Gw1NTVF3e92u+V2u/u0BwKBfovX2to6oi+wXtQhhDqEUIcQ6hBiWYdAIDCg4wb98bT29nY1NTXxUAIAIELcZ0CVlZW6/vrrlZmZqebmZlVVVSklJUU33XRTvE8FAEhicQ+gb7/9Vr/73e/U2tqqsWPHqrCwUOXl5Ro7dmy8TwUASGJxD6Bf/OIX8X5JYNiJ5cvZ/fVJTU0N/7P3z716enqcD+479HPK5XI57jPIz01hCLFEAQDABAEEADBBAAEATBBAAAATBBAAwAQBBAAwQQABAEwQQAAAEwQQAMAEAQQAMEEAAQBMEEAAABOD/oN0QDIZyK84nu+bb75x3Ofyyy933OcHP/hB1Hav1ytJuuSSS+TxeCL25eXlOT6PJB04cMBxn1OnTjnuE8uirN3d3Y77IDExAwIAmCCAAAAmCCAAgAkCCABgggACAJgggAAAJgggAIAJAggAYIIAAgCYIIAAACYIIACACQIIAGCCAAIAmGA17GHG5XI57pOaGttlkJGR4bjPyZMnHffpb8Xk3vaUlJQ+x2zcuNHxeSTphhtucNwnlhWdOzs7Hff54x//GLW997/fCy+8oK6uroh9P/3pTx2fR4rtv1Msenp6huQ8SEzMgAAAJgggAIAJAggAYIIAAgCYIIAAACYIIACACQIIAGCCAAIAmCCAAAAmCCAAgAkCCABgggACAJhgMdJhJhgMOu4Ty8KY0tAtWBmLH//4xzH16+7udtwnlsVIY1k09s0334zaPmbMGP3whz/UO++8o/b29oh9p0+fdnyeoRTL9YrhgxkQAMAEAQQAMEEAAQBMEEAAABMEEADABAEEADBBAAEATBBAAAATBBAAwAQBBAAwQQABAEwQQAAAEyxGioQ3a9asqO0ej0eSNGPGDAUCgYh9O3fujOlc99xzj+M+DQ0Njvt8/vnnjvu8//77Udu9Xq8kaffu3X0WH/3e977n+DyStGfPnpj6AU4wAwIAmCCAAAAmHN+Cq6ur0/bt23X48GE1Nzdr5cqVmjt3bnh/MBhUVVWV3n//fQUCARUWFqq4uFiTJ0+O68ABAMnN8Qyoo6NDeXl5evjhh6Puf/PNN/Xuu++qpKREzz//vEaPHq3y8nKdPXv2Ow8WADB8OA6g2bNna/ny5RGznl7BYFA7duzQ0qVLNWfOHOXm5uqxxx5Tc3OzPvnkk7gMGAAwPMT1Kbjjx4+rpaUl4qmljIwM5efnq76+XjfeeGOfPp2dnRE/Ce1yuZSeni6PxyOfzxdxbO/2+e0jzUirQ+/TbufLyMiI+Oe53G73oI7pXLH8vPaoUaMc9+l92u18vfWJVqdotRmIZLy2Rtr7oj+JUIf+3rPni2sAtbS0SJLGjRsX0T5u3LjwvvNt27ZN1dXV4e2pU6dq/fr1qqmp6fc8jY2N33mswwF1CNm1a5fp+adOnTokfQ4dOnTB/bW1tY5fczjifRGSDHUw/x7QkiVLtHjx4vB2798m58+fr/3790cc6/P51NjYqJycHLW2tg7pOBPJSKtDf98DysjI0K5du/T9739fbW1tEfsef/zxmM515513Ou5z7Ngxx31i+e7QE088EbXd4/GotrZWc+fO7fN9qEsvvdTxeSTp448/jqmfpZH2vuhPItShqKjogpOIXnENIL/fL0k6efKkxo8fH24/efKk8vLyovZxu91Rb5cEAoF+i9fa2jqiL7BeI6UO5/9P9XxtbW19jjn3tu5gCwaDjvt0d3c77nP+l0zPFwgE+hxzfjAPVDJfVyPlfXExlnW42Hu2V1y/B5SVlSW/368DBw6E29ra2vT555+roKAgnqcCACQ5xzOg9vZ2NTU1hbePHz+uI0eOyOv1KjMzU3fccYfeeOMNTZ48WVlZWXrttdc0fvx4zZkzJ64DBwAkN8cB9MUXX2jNmjXh7crKSknSzTffrLKyMv3oRz9SR0eHXnzxRbW1tamwsFBPP/200tLS4jdqAEDScxxAV199taqqqvrd73K5tGzZMi1btuw7DQzodebMmajtKSkp4f3nH7NixYqYzvWTn/zEcZ/UVOcfpS5YsMBxn3fffTdqe28d/v73v6unpydi30cffeT4PJK0b98+x31i/bwJIxdrwQEATBBAAAATBBAAwAQBBAAwQQABAEwQQAAAEwQQAMAEAQQAMEEAAQBMEEAAABMEEADABAEEADBBAAEATJj/JDdwMV9++WXUdp/PJ0k6evRon19+bG9vH/Rx9ZowYYLjPv/4xz8c98nIyLjg/sLCwj5t06dPd3weSRo3bpzjPom8Ar7L5YqpXyy/douBYwYEADBBAAEATBBAAAATBBAAwAQBBAAwQQABAEwQQAAAEwQQAMAEAQQAMEEAAQBMEEAAABMEEADABIuRIuH1t7Co2+0O7x/KxUfj4WILi1o7fPiw9RDiikVFExMzIACACQIIAGCCAAIAmCCAAAAmCCAAgAkCCABgggACAJgggAAAJgggAIAJAggAYIIAAgCYIIAAACZYjBT4jr7++mvHfU6fPu24j9frddynra3NcR9J+uc//xlTP8AJZkAAABMEEADABAEEADBBAAEATBBAAAATBBAAwAQBBAAwQQABAEwQQAAAEwQQAMAEAQQAMEEAAQBMsBgpYODFF1903OeJJ57od5/b7VZnZ2ef9lGjRjk+jyS9/fbbMfUDnGAGBAAwQQABAEw4vgVXV1en7du36/Dhw2pubtbKlSs1d+7c8P7Nmzdr9+7dEX2Kioq0atWq7z5aAMCw4TiAOjo6lJeXp4ULF+qFF16Iesw111yj0tLS/z9JKh81AQAiOU6G2bNna/bs2Rd+0dRU+f3+WMcEABgBBmVqUldXp+LiYnk8Hs2YMUPLly+Xz+eLemxnZ2fE0zsul0vp6enyeDx9+vRu9/daIwV1CEnmOqSlpVkP4YJiqWlXV9cgjGTgkvl6iKdEqIPH4xnQca5gMBiM9ST33ntvn8+APvroI40ePVpZWVlqamrS1q1bNWbMGJWXlyslpe8zD1VVVaqurg5vT506VevXr491SACAJBH3GdCNN94Y/vOUKVOUm5urxx9/XIcOHdLMmTP7HL9kyRItXrw4vO1yuSRJ8+fP1/79+yOO9fl8amxsVE5OjlpbW+M99KRBHUKSuQ7l5eWO+/zsZz/rd19/3wPq6OhwfB4p9N51KhFmQMl6PcRTItShqKhINTU1Fz1u0J8OuOSSS+Tz+dTU1BQ1gNxut9xud5/2QCDQb/FaW1tH9AXWizqEJGMdzp49az2EC4qlntYB1CsZr4fBYFmHQCAwoOMG/XtA33zzjU6fPq3x48cP9qkAAEnE8Qyovb1dTU1N4e3jx4/ryJEj8nq98nq9ev311zVv3jz5/X59/fXXeuWVVzRp0iQVFRXFdeAAgOTmOIC++OILrVmzJrxdWVkpSbr55ptVUlKir776Srt371YgENCECRM0a9YsLVu2LOptNgDAyOU4gK6++mpVVVX1u58VDzDS9D4448S6desc93nyyScvuD/awqOxfgm8u7s7pn6AE6wFBwAwQQABAEwQQAAAEwQQAMAEAQQAMEEAAQBMEEAAABMEEADABAEEADBBAAEATBBAAAATBBAAwAQBBAAwMei/iAoMd8Fg0HGfaCtXD4aB/jLl+WJZRTvaT4IDF8IMCABgggACAJgggAAAJgggAIAJAggAYIIAAgCYIIAAACYIIACACQIIAGCCAAIAmCCAAAAmCCAAgAkWIx0isSw+2d3dPQgjQSL44osvHPdJSbnw3xej7T906JDj80gsLIqhwQwIAGCCAAIAmCCAAAAmCCAAgAkCCABgggACAJgggAAAJgggAIAJAggAYIIAAgCYIIAAACYIIACACRYjHSI9PT3WQ8AgSUtLc9xn3LhxgzCSvsrLy4fkPEAsmAEBAEwQQAAAEwQQAMAEAQQAMEEAAQBMEEAAABMEEADABAEEADBBAAEATBBAAAATBBAAwAQBBAAwwWKkMRg1apTjPt3d3YMwEiSCl19+2XoI/dq1a5f1EIB+MQMCAJgggAAAJhzdgtu2bZtqa2vV2NiotLQ0FRQU6P7771d2dnb4mLNnz6qyslJ79+5VZ2enioqKVFxcLL/fH++xAwCSmKMZUF1dnW677TaVl5frV7/6lbq7u7Vu3Tq1t7eHj9myZYv+9a9/acWKFVqzZo2am5u1YcOGuA8cAJDcHAXQqlWrtGDBAl122WXKy8tTWVmZTpw4oYaGBklSW1ubPvjgAz3wwAOaMWOGpk2bptLSUv33v/9VfX39oPwLAACS03d6Cq6trU2S5PV6JUkNDQ3q7u7WzJkzw8fk5OQoMzNT9fX1Kigo6PManZ2d6uzsDG+7XC6lp6fL4/HI5/NFHNu7fX77ULN+Ci5R6mAtUeqQmpq4D5Na12YoJcr1YC0R6uDxeAZ0XMzvnJ6eHr388su64oorNGXKFElSS0uLUlNT+5x83Lhxamlpifo627ZtU3V1dXh76tSpWr9+vWpqavo9d2NjY6zDHlaoQwh16N+pU6eshzDkuB5CkqEOMQdQRUWFjh49qrVr136nASxZskSLFy8Ob7tcLknS/PnztX///ohjfT6fGhsblZOTo9bW1u903u8iEWZAiVAHa4lSh4qKCsd97rnnnkEYSV9jx44dkvMkgkS5HqwlQh2KioouOInoFVMAVVRUaN++fVqzZo0mTpwYbvf7/erq6lIgEIiYBZ08ebLfp+Dcbrfcbnef9kAg0G/xWltbR3QA9bKuQ6KwrkNXV5fZuS9mJF4f1tdDorCsQyAQGNBxjh5CCAaDqqioUG1trVavXq2srKyI/dOmTdOoUaN04MCBcNuxY8d04sSJqJ//AABGLkczoIqKCu3Zs0dPPfWU0tPTw5/rZGRkKC0tTRkZGVq4cKEqKyvl9XqVkZGhl156SQUFBQQQACCCowDqXVfq2WefjWgvLS3VggULJEkPPPCAXC6XNmzYoK6urvAXUQEAOJejAKqqqrroMWlpaSouLh7WocPCojjXnDlzhuQ8F/qsKTU1NaE/iwKiYS04AIAJAggAYIIAAgCYIIAAACYIIACACQIIAGCCAAIAmCCAAAAmCCAAgAkCCABgggACAJgggAAAJgggAICJmH+SG0DImTNnhuQ8hYWFUdu9Xq8+++wzXX/99Tp9+vSQjAWIB2ZAAAATBBAAwAQBBAAwQQABAEwQQAAAEwQQAMAEAQQAMEEAAQBMEEAAABMEEADABAEEADBBAAEATAybxUhdLpfjPsFgcBBGgpHm6quvdtwnlmuvrKwsantaWpokqaSkRGfPno3Yt2LFCsfnAYYKMyAAgAkCCABgggACAJgggAAAJgggAIAJAggAYIIAAgCYIIAAACYIIACACQIIAGCCAAIAmCCAAAAmhs1ipCwsinhwu92O+wQCAcd9Nm/e7LjP2rVro7b7fD6VlZVp7dq1am1tdfy6gBVmQAAAEwQQAMAEAQQAMEEAAQBMEEAAABMEEADABAEEADBBAAEATBBAAAATBBAAwAQBBAAwQQABAEwMm8VIgXhIT0933Oe5555z3OdPf/qT4z7t7e1R23sXUG1vb+/3GCARMQMCAJgggAAAJhzdgtu2bZtqa2vV2NiotLQ0FRQU6P7771d2dnb4mGeffVZ1dXUR/W699VY98sgj8RkxAGBYcBRAdXV1uu2223T55Zeru7tbW7du1bp167Rx40aNGTMmfNyiRYu0bNmy8HZaWlr8RgwAGBYcBdCqVasitsvKylRcXKyGhgZdddVV4fbRo0fL7/fHZYAAgOHpOz0F19bWJknyer0R7TU1NaqpqZHf79d1112nu+66S6NHj476Gp2dners7Axvu1wupaeny+PxyOfzRRzbu31++0hDHUIGow6xvFZ/13a8z5OSEv0j2wvVYST9VD3vi5BEqIPH4xnQca5gjFdoT0+PfvOb3ygQCEQ8hvree+8pMzNTEyZM0JdffqlXX31V+fn5WrlyZdTXqaqqUnV1dXh76tSpWr9+fSxDAgAkkZgD6M9//rM+++wzrV27VhMnTuz3uIMHD2rt2rX6/e9/r0mTJvXZ398MaP78+dq/f3/EsT6fT42NjcrJyVFra2sswx4WqEPIYNRh7Nixjvs8+OCDjvtUVFQ47hMIBKK2+3w+/e9//9Oll17apw4jbQbE+yIx6lBUVKSampqLHhfTLbiKigrt27dPa9asuWD4SFJ+fr4kqampKWoAud3u8BfpzhUIBPotXmtr64i+wHpRh5B41sHlcjnu09HR4bhPLOPtL4DOfc2RHEC9eF+EWNbhYtdqL0ffAwoGg6qoqFBtba1Wr16trKysi/Y5cuSIJGn8+PFOTgUAGOYczYAqKiq0Z88ePfXUU0pPT1dLS4skKSMjQ2lpaWpqatKePXt07bXXyuv16quvvtKWLVt05ZVXKjc3dzDGDwBIUo4CaNeuXZJCXzY9V2lpqRYsWKDU1FQdOHBAO3bsUEdHhyZOnKh58+Zp6dKlcRswAGB4cBRAVVVVF9yfmZmpNWvWfKcBAQBGBlbDHiKxfLg9Ej9Atnb69GnHff7yl7847nPu8lUDVV9fH7W99zoJBoNcM0gqLEYKADBBAAEATBBAAAATBBAAwAQBBAAwQQABAEwQQAAAEwQQAMAEAQQAMEEAAQBMEEAAABMEEADABIuRDhEWiUwOPT09jvs0NzcPSR8gHlJSnM87YnlfDAQzIACACQIIAGCCAAIAmCCAAAAmCCAAgAkCCABgggACAJgggAAAJgggAIAJAggAYIIAAgCYSNi14AoLC/u0eTweSVJRUZECgcBQDylhUIcQ6hBCHUKoQ8jF6jAUa8FF+/93NK4gq2QCAAwk1S24M2fO6Je//KXOnDljPRRT1CGEOoRQhxDqEJJMdUiqAAoGgzp8+PCI/2kD6hBCHUKoQwh1CEmmOiRVAAEAhg8CCABgIqkCyO126+6775bb7bYeiinqEEIdQqhDCHUISaY68BQcAMBEUs2AAADDBwEEADBBAAEATBBAAAATCbsW3Pl27typt956Sy0tLcrNzdVDDz2k/Px862ENqaqqKlVXV0e0ZWdna9OmTTYDGiJ1dXXavn27Dh8+rObmZq1cuVJz584N7w8Gg6qqqtL777+vQCCgwsJCFRcXa/LkyYajjr+L1WHz5s3avXt3RJ+ioiKtWrVqqIc6aLZt26ba2lo1NjYqLS1NBQUFuv/++5WdnR0+5uzZs6qsrNTevXvV2dmpoqIiFRcXy+/32w08zgZSh2effVZ1dXUR/W699VY98sgjQz3cfiVFAO3du1eVlZUqKSnR9OnT9c4776i8vFybNm3SuHHjrIc3pC677DI988wz4e1YFhZMNh0dHcrLy9PChQv1wgsv9Nn/5ptv6t1331VZWZmysrL0t7/9TeXl5dq4caPS0tIMRjw4LlYHSbrmmmtUWloa3k5NTYq3+IDV1dXptttu0+WXX67u7m5t3bpV69at08aNGzVmzBhJ0pYtW7Rv3z6tWLFCGRkZqqio0IYNG/Tcc88Zjz5+BlIHSVq0aJGWLVsW3k6090NSXJ1vv/22Fi1apFtuuUWSVFJSon379unDDz/UnXfeaTu4IZaSkjKs/iY3ELNnz9bs2bOj7gsGg9qxY4eWLl2qOXPmSJIee+wxlZSU6JNPPtGNN944lEMdVBeqQ6/U1NRhfX2cP5srKytTcXGxGhoadNVVV6mtrU0ffPCBfv7zn2vGjBmSpNLSUj355JOqr69XQUGBxbDj7mJ16DV69OiEvh4SPoC6urrU0NAQETQpKSmaOXOm6uvr7QZmpKmpSY8++qjcbrcKCgp03333KTMz03pYZo4fP66WlhbNmjUr3JaRkaH8/HzV19cPqwAaiLq6OhUXF8vj8WjGjBlavny5fD6f9bAGTVtbmyTJ6/VKkhoaGtTd3a2ZM2eGj8nJyVFmZuawCqDznV+HXjU1NaqpqZHf79d1112nu+66S6NHj7YYYlQJH0CnTp1ST09PnxT3+/06duyYzaCMTJ8+XaWlpcrOzlZzc7Oqq6u1evVqbdiwQenp6dbDM9HS0iJJfW7Fjhs3LrxvpLjmmms0b948ZWVlqampSVu3btXzzz+v8vLyYXmrtqenRy+//LKuuOIKTZkyRVLoekhNTQ3/Jk6v4Xw9RKuDJN10003KzMzUhAkT9OWXX+rVV1/VsWPHtHLlSsPRRkr4AML/O/f2S25ubjiQPv74Yy1cuNBwZEgE5872pkyZotzcXD3++OM6dOhQxIxguKioqNDRo0e1du1a66GY6q8Ot956a/jPU6ZM0fjx47V27Vo1NTVp0qRJQz3MqBL+r0Vjx45VSkpKn7+9tLS0JPS9zaHg8XiUnZ2tpqYm66GY6b0GTp48GdF+8uTJEX99XHLJJfL5fMPy+qioqNC+ffv061//WhMnTgy3+/1+dXV19fkl0OF6PfRXh2h6nxpOpOsh4QMoNTVV06ZN08GDB8NtPT09Onjw4LC9nztQ7e3tampqGpZvrIHKysqS3+/XgQMHwm1tbW36/PPPR/z18c033+j06dMaP3689VDiJhgMqqKiQrW1tVq9erWysrIi9k+bNk2jRo2KuB6OHTumEydODKvr4WJ1iObIkSOSlFDXQ1Lcglu8eLE2b96sadOmKT8/Xzt27FBHR4cWLFhgPbQhVVlZqeuvv16ZmZlqbm5WVVWVUlJSdNNNN1kPbVD1Bm2v48eP68iRI/J6vcrMzNQdd9yhN954Q5MnT1ZWVpZee+01jR8/PvxU3HBxoTp4vV69/vrrmjdvnvx+v77++mu98sormjRpkoqKigxHHV8VFRXas2ePnnrqKaWnp4fvjGRkZCgtLU0ZGRlauHChKisr5fV6lZGRoZdeekkFBQXDKoAuVoempibt2bNH1157rbxer7766itt2bJFV155pXJzc20Hf46kWQ17586d2r59u1paWpSXl6cHH3xQ06dPtx7WkNq0aZP+/e9/q7W1VWPHjlVhYaGWL1+eMPdzB8uhQ4e0Zs2aPu0333yzysrKwl9Efe+999TW1qbCwkI9/PDDEV/KGw4uVIeSkhL99re/1eHDhxUIBDRhwgTNmjVLy5YtG1Yz5HvvvTdqe2lpafgvpL1fRP3oo4/U1dU1LL+IerE6nDhxQn/4wx909OhRdXR0aOLEiZo7d66WLl2qjIyMIR5t/5ImgAAAw0vCfwYEABieCCAAgAkCCABgggACAJgggAAAJgggAIAJAggAYIIAAgCYIIAAACYIIACACQIIAGCCAAIAmPg/9LpTBg/wyG0AAAAASUVORK5CYII=\n"
          },
          "metadata": {}
        },
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 640x480 with 1 Axes>"
            ],
            "image/png": "iVBORw0KGgoAAAANSUhEUgAAAaAAAAGdCAYAAABU0qcqAAAAOnRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjEwLjAsIGh0dHBzOi8vbWF0cGxvdGxpYi5vcmcvlHJYcgAAAAlwSFlzAAAPYQAAD2EBqD+naQAAJPVJREFUeJzt3X1wU1X+x/FP2qalTVICdCq0QkuFCgrUIsLuKiOCMzgOOwo6yjI6zipVp+i6MozOiKKgdZef4rI6/LG7U0c6umCtsj7hw/qwTpF1UUEGqC6rgLhlKoINLSktfcjvj0yzhKQP95r0JO379Y/m3PvNPZzc5pN7c3OuIxAIBAQAwABLMd0BAMDQRAABAIwggAAARhBAAAAjCCAAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIxIM92BnixZskRfffVVWJvL5VJtba1mz54tv99vqGfmMQ5BjEMQ4xDEOATFYxyGDx9uaf2JEyfqz3/+c5/rxS2A3n77bb3++uvy+XwqKCjQrbfeqgkTJvS7/quvvtKuXbvC2jwejyRp9+7dam5ujml/kwnjEMQ4BDEOQYxDUDzGYeTIkTF5nrPF5RTc9u3bVVVVpeuvv15r165VQUGBKioqdOLEiXhsDgCQhOISQG+88YbmzZunK664Queee67KysqUnp6uDz/8MB6bAwAkoZifguvo6NCBAwd07bXXhtpSUlI0depU7d+/P2L99vZ2tbe3hx47HA5lZmbK5XKFDiW7dT8+u32oYRyCGIcgxiGIcQiKxzhYfa6srKx+rRfzAGpqalJXV5e8Xm9Yu9fr1ZEjRyLW37Jli2pqakKPx48fr7Vr16q2trbHbdTX18esv8mMcQhiHIIYhyDGISgZxsH4VXALFy7UggULQo8dDockafbs2dq9e3fYuh6PR/X19crPzx/yXzIyDoxDN8YhiHEIisc4WL0I4cILL9Sbb77Z53oxD6Ds7GylpKTI5/OFtft8voijIklyOp1yOp0R7X6/v8fBa25uHtI7WDfGIYhxCGIcghiHoFiOQ7T36N60tLT0a72YX4SQlpamoqIi7d27N9TW1dWlvXv3qri4ONabAwAkqbicgluwYIE2bNigoqIiTZgwQVu3blVbW5vmzJkTj80BAJJQXALoF7/4hZqamlRdXS2fz6fCwkI98MADUU/BAQCGprhdhHDVVVfpqquuitfTA71KTU21VdfZ2RnjnkTXfbGNFYFAIA49Afr2448/Wlq/v5MOMBkpAMAIAggAYAQBBAAwggACABhBAAEAjCCAAABGEEAAACMIIACAEQQQAMAIAggAYAQBBAAwggACABhh/I6oMC/RJ+60I5H7Jklz5861XPP+++/HoSeAORwBAQCMIIAAAEYQQAAAIwggAIARBBAAwAgCCABgBAEEADCCAAIAGEEAAQCMIIAAAEYQQAAAIwggAIARBBAAwAhmw0bCzxw9GL3xxhuWa9LT03td7vP5+tXWH+Xl5ZZrXnzxRVvbwtDFERAAwAgCCABgBAEEADCCAAIAGEEAAQCMIIAAAEYQQAAAIwggAIARBBAAwAgCCABgBAEEADCCAAIAGMFkpMAZpk+fbrnmb3/7m+WaYcOGWa7pS0pK5OfJkSNH2nque++913LNE088Yblm2rRplmtSU1OjtrvdbknBf/PZE7ceP37c8nYQfxwBAQCMIIAAAEYQQAAAIwggAIARBBAAwAgCCABgBAEEADCCAAIAGEEAAQCMIIAAAEYQQAAAIwggAIARTEYKnOHFF1+0XDN27Ng49MSsWbNmWa7p6OiwXDN//nzLNX//+9/7XCcQCFh+Xgw8joAAAEYQQAAAI2J+Cq66ulo1NTVhbXl5eVq/fn2sNwUASGJx+Q5o7Nixeuihh0KPo90oCwAwtMUlgFJSUuT1euPx1ACAQSIuAdTQ0KA77rhDTqdTxcXFWrJkiXJycqKu297ervb29tBjh8OhzMxMuVwueTyesHW7H5/dPtQwDkHxGAeHwxGz50LfMjMzLdf09Hp335K7+79nOvM9ZrBLhPcHl8vVr/UcgRhfr7hr1y61trYqLy9PjY2Nqqmp0Y8//qh169ZF3dnO/s5o/PjxWrt2bSy7BABIQDEPoLP5/X6Vl5frlltu0dy5cyOW93QENHv2bO3evTtsXY/Ho/r6euXn56u5uTme3U5ojENQPMZh165dlmvOO++8mGw72dn5HVBZWZnlmg8//DBqu9vt1t69ezVlyhSdPHkybNmPP/5oeTvJKhHeH0pKSlRbW9vnenH/IarL5VJeXp4aGhqiLnc6nXI6nRHtfr+/x8Frbm4e0m+83RiHoFiOAz9gHFinTp2yXNPXa33y5MmIdYbi34nJ9we/39+v9eJ+eVpra6saGhq4KAEAECbmR0BVVVWaMWOGcnJy1NjYqOrqaqWkpOiyyy6L9aYAAEks5gH0448/6o9//KOam5uVnZ2tSZMmqaKiQtnZ2bHeFAAgicU8gH7729/G+ikBy3744QdbdT39XCAR9PYlf1paWtTlaWkDN9+wnW1t3rzZcs3Pf/7zqO1ZWVmSgheFtLS0hC375JNPLG8H8ccUBQAAIwggAIARBBAAwAgCCABgBAEEADCCAAIAGEEAAQCMIIAAAEYQQAAAIwggAIARBBAAwAgCCABgxMDNVAjYtHDhwqjt3bd4/+UvfxlxY7NEnlRUkj7//HPLNZs2bYranpGRoYqKCj388MNqa2sLW3bbbbfZ6t/kyZNt1Q2Ed955p9flW7ZsiWgbPXq0rW3ZuWEe+o8jIACAEQQQAMAIAggAYAQBBAAwggACABhBAAEAjCCAAABGEEAAACMIIACAEQQQAMAIAggAYAQBBAAwggACABjhCAQCAdOdiGb69OnatWtXWJvH41FTU5Oys7PV3Nz8k7fhcDhs1ZkesliPw0BKTU21XNPU1NTjsqysLLW0tERtHyhffPGF5ZrS0lLLNSkp0T8vejwe+Xw+eb3eiP0hPz/f8nYkad++fZZrPB6PrW1Z1dvfn8PhiLrc7v7Q2tpqq86kRHh/KC0t1c6dO/tcjyMgAIARBBAAwAgCCABgBAEEADCCAAIAGEEAAQCMIIAAAEYQQAAAIwggAIARBBAAwAgCCABgBAEEADAizXQHTDI9qWiyszOZa0FBgeWazZs3R213Op26+eab9fLLL6u9vT1s2a233mp5O5JUVVVluWb58uW2tmVVT/trd3sgEIhYp6ioyNa27Ews2tHRYbkmLc36W1BnZ2evzxdt+TnnnGN5O5L07bff2qpD/3AEBAAwggACABhBAAEAjCCAAABGEEAAACMIIACAEQQQAMAIAggAYAQBBAAwggACABhBAAEAjCCAAABGDOnJSPHTZGdnW645cOCA5ZrVq1dHbXe73br55pv1+9//XidPngxbduedd1rejqSISU37IzU1dUBqxowZE7Xd7XZLkvLy8iLGYeHChZa3I9l7ncaPH29rW1b1NYlwtOVTpkyxtS0mI40vjoAAAEYQQAAAIyyfgqurq9Nrr72mgwcPqrGxUStWrNDMmTNDywOBgKqrq/X+++/L7/dr0qRJWrp0aY+nDwAAQ5PlI6C2tjYVFhbqtttui7r81Vdf1VtvvaWysjI9/vjjysjIUEVFhU6fPv2TOwsAGDwsB1BpaakWL14cdtTTLRAIaOvWrVq0aJEuueQSFRQU6K677lJjY6M+/fTTmHQYADA4xPQquKNHj8rn82natGmhtqysLE2YMEH79+/XpZdeGlHT3t4eduWRw+FQZmamXC5XxG2Bux/buV3wYJIo42Bn+11dXZZruq/yOpvL5Qr775nsjs1AXQVnh51xSE9Pt7UtO7dbT2SZmZm26kz/jdmRCO8P0fbFaGIaQD6fT5I0fPjwsPbhw4eHlp1ty5YtqqmpCT0eP3681q5dq9ra2h63U19f/5P7OhgwDkE7duww3YWE8Nlnn5nuwoBwOp2Wl7/00kvx6k7CSob3B+O/A1q4cKEWLFgQetz9yWv27NnavXt32Loej0f19fXKz89Xc3PzgPYzkSTKOJz9QaM/Tpw4Yblm7NixUdtdLpd27NihmTNnyu/3hy1raGiwvB0psY+ARo8eHbXd5XLps88+04wZMyLG4Te/+Y2tbV111VWWawoLCy3X2DnS6u01cjqdUZcvWbLE8nYk6Z133rFVZ1IivD+UlJT0ehDRLaYB5PV6JQXfZEaMGBFqP3HiRI87p9PpjPqJxe/39zh4zc3NQzqAupkeh5QU61fx2+nv2T+uPJvf749Yx+64JHIA9XQKrlu0cbB78U9fP/ZMNqdOnbJVl8zvMybfH87+INSTmP4OKDc3V16vV3v27Am1tbS06Ouvv1ZxcXEsNwUASHKWj4BaW1vDTm8cPXpUhw4dktvtVk5Ojq6++mq98sorGjNmjHJzc7V582aNGDFCl1xySUw7DgBIbpYD6Jtvvgmbm6uqqkqSdPnll2vZsmW65ppr1NbWpj/96U9qaWnRpEmT9MADD9i+GgcAMDg5Agl6snf69OnatWtXWJvH41FTU5Oys7OT7txsVlaW5Zqezlt7PB6dOHFCw4cPjxgHOy9nT1/y98XOVTZ2LsPuSaLsD31dlRWNne+aetqOx+PR8ePHNWrUqJiNQ2trq+UaO98JDpTnn3/eVl1ZWZnlGjtjF0uJ8HdRWlqqnTt39rle4u4xAIBBjQACABhBAAEAjCCAAABGEEAAACMIIACAEQQQAMAIAggAYAQBBAAwggACABhBAAEAjCCAAABGEEAAACOM35J7qGhpaYnZc3XPeB0IBCJmv+7rrpnRHDlyxFY/0tKs7z5279CZyOzMbG1HZ2dnr+2dnZ0R69i9FfXhw4ct19i5Jbcdvc347nA4oi4vKiqyta2exrw3dm4znqA3JYg7joAAAEYQQAAAIwggAIARBBAAwAgCCABgBAEEADCCAAIAGEEAAQCMIIAAAEYQQAAAIwggAIARBBAAwAgmIx0gdiZqPHTokOWaxYsXW655+eWXLddIUmNjo6062NPV1dVre1dXV8Q6dl/b9evX26qz6tixY5Zrevq7SElJ0fTp07Vr166IcXjmmWfsdE+XXHKJ5Zrt27fb2tZQxBEQAMAIAggAYAQBBAAwggACABhBAAEAjCCAAABGEEAAACMIIACAEQQQAMAIAggAYAQBBAAwggACABgxaCYjzczMtFxz6tSpOPQkOjsTi6ampvbanpqaGrHO008/bXk7M2bMsFwjSXfeeaetOgwcu/v4zTffbLlm9erVlmvsTEZaXFwctd3hcEiSvF6vAoFA2LK3337b8nYkqampyXJNdz+sOLu/QwVHQAAAIwggAIARBBAAwAgCCABgBAEEADCCAAIAGEEAAQCMIIAAAEYQQAAAIwggAIARBBAAwAgCCABgxKCZjHQgJxa1o6eJRXvT10SNBw8ejGizMynrLbfcYrlGYjLSweydd96xXPP+++9brpk6darlmp4mzx02bJjWr1+vp59+Wq2trWHLFi5caHk7kvTee+9ZrrEzwWqiv3/FC0dAAAAjCCAAgBGWT8HV1dXptdde08GDB9XY2KgVK1Zo5syZoeUbNmzQRx99FFZTUlKilStX/vTeAgAGDcsB1NbWpsLCQs2dO1dPPvlk1HUuuugilZeX/28jaYPmqyYAQIxYTobS0lKVlpb2/qRpafJ6vXb7BAAYAuJyaFJXV6elS5fK5XJpypQpWrx4sTweT9R129vb1d7eHnrscDiUmZkpl8sVUdP9uKfnSmR2roJLdKZfh2TeH2IpHuOQkmL962E7ZzpcLpflmmHDhkVtz8jICPvvmez8eyTJ7XZbrmlra7NcE8uzRInwd9Hf19UR+Ak3I7/hhhsivgP6+OOPlZGRodzcXDU0NGjTpk0aNmyYKioqou4E1dXVqqmpCT0eP3681q5da7dLAIAkEfMjoEsvvTT0/+PGjVNBQYHuvvtu7du3L+o1/wsXLtSCBQtCjx0OhyRp9uzZ2r17d9i6Ho9H9fX1ys/PV3Nzc6y7Hld2joCi/c6nm9frlc/ni9pu1dm/meiv3NxcW3Wxksz7QyzFYxwG6gjowgsvtFzT01cAGRkZWrt2re6///6Io5DTp09b3o4k/eMf/7Bcc/z4ccs1sfwdUCL8XZSUlKi2trbP9eJ+dcA555wjj8ejhoaGqAHkdDrldDoj2v1+f4+D19zcnHRvOIPxFFyivAbJuD/EQyzHYaACyO/3W67p6wNTW1tbxDp2A+jkyZOWa+y8BvH4IarJv4v+vq5x/x3Q8ePHdfLkSY0YMSLemwIAJBHLH1laW1vV0NAQenz06FEdOnRIbrdbbrdbL730kmbNmiWv16vvv/9ezz//vEaPHq2SkpKYdhwAkNwsB9A333yj1atXhx5XVVVJki6//HKVlZXp8OHD+uijj+T3+zVy5EhNmzZNN954Y9TTbACAoctyAF144YWqrq7ucTkzHkR36NAhyzV9XVAQq99a9XRZa19+9atfWa7ZtGmTrW1hYNm5ODY9Pd1yjZ0LWR599NGo7d0XMD344IMR/V++fLnl7Zz5nFbYOdvzySefWK4ZDJgLDgBgBAEEADCCAAIAGEEAAQCMIIAAAEYQQAAAIwggAIARBBAAwAgCCABgBAEEADCCAAIAGEEAAQCMIIAAAEbE/Y6oCHrqqacGpGYgvfDCC5ZrmA3bvp7uqtvdnpqaGrFORkaGrW3ZmaV62bJllmvuueceyzX//e9/o7Y7HA7l5OTo5MmTEbNh252l/+DBg5ZrPv74Y1vbGoo4AgIAGEEAAQCMIIAAAEYQQAAAIwggAIARBBAAwAgCCABgBAEEADCCAAIAGEEAAQCMIIAAAEYQQAAAI5iM1IaUFOu5/dxzz1muSfTJSLu6uizXpKVZ3+U6Ojos19h5jezW2elfenq65ZrPPvssant3n7dt2xbxmpx77rmWtyNJw4cPt1VnlcPhsFxTWFjY6/KCggKbvYm0fft2yzV2/k1nT546VHAEBAAwggACABhBAAEAjCCAAABGEEAAACMIIACAEQQQAMAIAggAYAQBBAAwggACABhBAAEAjCCAAABGMBmpDXYm4fzrX/8ah56YlZqaarlm5MiRlms6Ozujtrvd7tBznj2554EDByxvR5I+//xzyzV2Jp+0M2Hm+PHje11+4YUXWn7OWLIzoaadsbPznHYnV7UzOW1P+ysicQQEADCCAAIAGEEAAQCMIIAAAEYQQAAAIwggAIARBBAAwAgCCABgBAEEADCCAAIAGEEAAQCMIIAAAEYwGekAueaaayzXtLa29rjM4XBEnfwxHpM7xtL3339vuaavyV+jTTxqZxJJSbriiiss19iZnNZu/xLZQO17HR0dPS5LS0uLurypqSmeXUoq9957r+WaP/zhD3HoCUdAAABDCCAAgBGWTsFt2bJFO3bsUH19vdLT01VcXKybbrpJeXl5oXVOnz6tqqoqbd++Xe3t7SopKdHSpUvl9Xpj3XcAQBKzdARUV1en+fPnq6KiQg8++KA6Ozv12GOPhX1XsXHjRn3++edavny5Vq9ercbGRq1bty7mHQcAJDdLAbRy5UrNmTNHY8eOVWFhoZYtW6Zjx46FvgRuaWnRBx98oFtuuUVTpkxRUVGRysvL9e9//1v79++Pyz8AAJCcftJVcC0tLZL+d2vkAwcOqLOzU1OnTg2tk5+fr5ycHO3fv1/FxcURz9He3q729vbQY4fDoczMTLlcLnk8nrB1ux+f3Z4Mzr5lNICBk4zvGXb19T6ZkZFh+zn7y+Vy9Ws92wHU1dWl5557Tueff77GjRsnSfL5fEpLS4vY+PDhw+Xz+aI+z5YtW1RTUxN6PH78eK1du1a1tbU9bru+vt5utweVRL/kOlb6umTZ9CXNprc/1KSl9f62FW35ULwMO5bvk7/73e9i9lxnsh1AlZWV+u6777RmzZqf1IGFCxdqwYIFocfdb6qzZ8/W7t27w9b1eDyqr69Xfn6+mpubf9J2B5qdI6Affvihx2XJ+jsgO3r7nU1KSkrU5QMZCvwOaGDZ+R3QyJEj49mlhNLX++SyZcssP+eGDRssrV9SUtLrQUQ3WwFUWVmpnTt3avXq1Ro1alSo3ev1qqOjQ36/P+wo6MSJEz1eBed0OuV0OiPa/X5/jyHT3Nw8JAIIQGwk2/tFLPT0PtnW1mbruazw+/39Ws/Sx7BAIKDKykrt2LFDq1atUm5ubtjyoqIipaamas+ePaG2I0eO6NixY1G//wEADF2WjoAqKyu1bds23XfffcrMzAx9r5OVlaX09HRlZWVp7ty5qqqqktvtVlZWlp599lkVFxcTQACAMJYC6N1335UkPfLII2Ht5eXlmjNnjiTplltukcPh0Lp169TR0RH6ISoAAGdyBKJ9k50Apk+frl27doW1eTweNTU1KTs7O+nO6dq5OOD888+P2u5yufTZZ59pxowZEedav/zyS1v9w+Bk5wIJyd7+evjwYcs1kydPtlxz6tSpqO3J/P4QS4kwDqWlpdq5c2ef63EpDgDACAIIAGAEAQQAMIIAAgAYQQABAIwggAAARhBAAAAjCCAAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIywfUtuWGNn0vGvvvoqarvH45Ek7d+/P2K224G8JXdvt0buSWpqahx6EsnuJO+dnZ2Wa+z8m6qqqizXPPnkk1HbXS6XPvnkE/3sZz+LmB194sSJlrcjSa+++qqtOqvsztaNwYEjIACAEQQQAMAIAggAYAQBBAAwggACABhBAAEAjCCAAABGEEAAACMIIACAEQQQAMAIAggAYAQBBAAwgslIYVtamtndx+PxqKmpSdnZ2RGTstplZzJXO5ORxnLS2O7Jaf/zn/9EjMPevXtjth0g1jgCAgAYQQABAIwggAAARhBAAAAjCCAAgBEEEADACAIIAGAEAQQAMIIAAgAYQQABAIwggAAARhBAAAAjmIwUOEMgELBc09HREYee9F97e3vov93/j8Gvp0lwu9tTU1OjrtPZ2RnXflnBERAAwAgCCABgBAEEADCCAAIAGEEAAQCMIIAAAEYQQAAAIwggAIARBBAAwAgCCABgBAEEADCCAAIAGMFkpACQhHqaVLS7vbOzM6EmHo2GIyAAgBEEEADACEun4LZs2aIdO3aovr5e6enpKi4u1k033aS8vLzQOo888ojq6urC6q688krdfvvtsekxAGBQsBRAdXV1mj9/vs477zx1dnZq06ZNeuyxx/TUU09p2LBhofXmzZunG2+8MfQ4PT09dj0GAAwKlgJo5cqVYY+XLVumpUuX6sCBA7rgggtC7RkZGfJ6vTHpIABgcPpJV8G1tLRIktxud1h7bW2tamtr5fV6dfHFF+u6665TRkZG1Oc4+zbCDodDmZmZcrlc8ng8Yet2Pz67fahhHIIYhyDGIYhxCEqEcXC5XP1azxEIBAJ2NtDV1aX/+7//k9/v16OPPhpqf++995STk6ORI0fq22+/1QsvvKAJEyZoxYoVUZ+nurpaNTU1ocfjx4/X2rVr7XQJAJBEbAfQX/7yF33xxRdas2aNRo0a1eN6e/fu1Zo1a/T0009r9OjREct7OgKaPXu2du/eHbaux+NRfX298vPz1dzcbKfbgwLjEMQ4BDEOQYxDUCKMQ0lJiWpra/tcz9YpuMrKSu3cuVOrV6/uNXwkacKECZKkhoaGqAHkdDrldDoj2v1+f4+D19zcPKR3sG6MQxDjEMQ4BDEOQSbHwe/392s9S78DCgQCqqys1I4dO7Rq1Srl5ub2WXPo0CFJ0ogRI6xsCgAwyFk6AqqsrNS2bdt03333KTMzUz6fT5KUlZWl9PR0NTQ0aNu2bZo+fbrcbrcOHz6sjRs3avLkySooKIhH/wEAScpSAL377ruSgj82PVN5ebnmzJmjtLQ07dmzR1u3blVbW5tGjRqlWbNmadGiRTHrMABgcLAUQNXV1b0uz8nJ0erVq39ShwAAQwNzwQEAjCCAAABGEEAAACMIIACAEQQQAMAIAggAYAQBBAAwggACABhBAAEAjCCAAABGEEAAACMIIACAEQQQAMAIW3dEReJyOByWa2bOnGlrW//6179s1cGe9PT0XtvT09Mj1jl9+nTc+5UMXC6Xrbr+3tkT9nAEBAAwggACABhBAAEAjCCAAABGEEAAACMIIACAEQQQAMAIAggAYAQBBAAwggACABhBAAEAjEjYueAmTZoU0dY9n1NJScmQnqOpt3GwMxfc+eefb6sfpucZG2r7g9PpjNrePQ7Tpk2LGIf29va49ytR9LY/ZGZm2nrOU6dO/eR+DbRE+LuI9v4djSMQCATi3BcAACIk1Sm4U6dO6f7770/KTyWxxDgEMQ5BjEMQ4xCUTOOQVAEUCAR08OBBDfWDNsYhiHEIYhyCGIegZBqHpAogAMDgQQABAIxIqgByOp26/vrre7waaKhgHIIYhyDGIYhxCEqmceAqOACAEUl1BAQAGDwIIACAEQQQAMAIAggAYETCzgV3trfffluvv/66fD6fCgoKdOutt2rChAmmuzWgqqurVVNTE9aWl5en9evXm+nQAKmrq9Nrr72mgwcPqrGxUStWrNDMmTNDywOBgKqrq/X+++/L7/dr0qRJWrp0qcaMGWOw17HX1zhs2LBBH330UVhNSUmJVq5cOdBdjZstW7Zox44dqq+vV3p6uoqLi3XTTTcpLy8vtM7p06dVVVWl7du3q729XSUlJVq6dKm8Xq+5jsdYf8bhkUceUV1dXVjdlVdeqdtvv32gu9ujpAig7du3q6qqSmVlZZo4caLefPNNVVRUaP369Ro+fLjp7g2osWPH6qGHHgo9TkkZ/AexbW1tKiws1Ny5c/Xkk09GLH/11Vf11ltvadmyZcrNzdWLL76oiooKPfXUU0pPTzfQ4/joaxwk6aKLLlJ5eXnocVpaUvyJ91tdXZ3mz5+v8847T52dndq0aZMee+wxPfXUUxo2bJgkaePGjdq5c6eWL1+urKwsVVZWat26dXr00UcN9z52+jMOkjRv3jzdeOONoceJ9veQFHvnG2+8oXnz5umKK66QJJWVlWnnzp368MMPde2115rt3ABLSUkZVJ/k+qO0tFSlpaVRlwUCAW3dulWLFi3SJZdcIkm66667VFZWpk8//VSXXnrpQHY1rnobh25paWmDev84+2hu2bJlWrp0qQ4cOKALLrhALS0t+uCDD3TPPfdoypQpkqTy8nLde++92r9/v4qLi010O+b6GoduGRkZCb0/JHwAdXR06MCBA2FBk5KSoqlTp2r//v3mOmZIQ0OD7rjjDjmdThUXF2vJkiXKyckx3S1jjh49Kp/Pp2nTpoXasrKyNGHCBO3fv39QBVB/1NXVaenSpXK5XJoyZYoWL14sj8djultx09LSIklyu92SpAMHDqizs1NTp04NrZOfn6+cnJxBFUBnO3scutXW1qq2tlZer1cXX3yxrrvuOmVkZJjoYlQJH0BNTU3q6uqKSHGv16sjR46Y6ZQhEydOVHl5ufLy8tTY2KiamhqtWrVK69ats32/k2Tn8/kkKeJU7PDhw0PLhoqLLrpIs2bNUm5urhoaGrRp0yY9/vjjqqioGJSnaru6uvTcc8/p/PPP17hx4yQF94e0tLTQPXG6Deb9Ido4SNJll12mnJwcjRw5Ut9++61eeOEFHTlyRCtWrDDY23AJH0D4nzNPvxQUFIQC6Z///Kfmzp1rsGdIBGce7Y0bN04FBQW6++67tW/fvrAjgsGisrJS3333ndasWWO6K0b1NA5XXnll6P/HjRunESNGaM2aNWpoaNDo0aMHuptRJfzHouzsbKWkpER8evH5fAl9bnMguFwu5eXlqaGhwXRXjOneB06cOBHWfuLEiSG/f5xzzjnyeDyDcv+orKzUzp079fDDD2vUqFGhdq/Xq46Ojog7gQ7W/aGncYim+6rhRNofEj6A0tLSVFRUpL1794baurq6tHfv3kF7Pre/Wltb1dDQMCj/sPorNzdXXq9Xe/bsCbW1tLTo66+/HvL7x/Hjx3Xy5EmNGDHCdFdiJhAIqLKyUjt27NCqVauUm5sbtryoqEipqalh+8ORI0d07NixQbU/9DUO0Rw6dEiSEmp/SIpTcAsWLNCGDRtUVFSkCRMmaOvWrWpra9OcOXNMd21AVVVVacaMGcrJyVFjY6Oqq6uVkpKiyy67zHTX4qo7aLsdPXpUhw4dktvtVk5Ojq6++mq98sorGjNmjHJzc7V582aNGDEidFXcYNHbOLjdbr300kuaNWuWvF6vvv/+ez3//PMaPXq0SkpKDPY6tiorK7Vt2zbdd999yszMDJ0ZycrKUnp6urKysjR37lxVVVXJ7XYrKytLzz77rIqLiwdVAPU1Dg0NDdq2bZumT58ut9utw4cPa+PGjZo8ebIKCgrMdv4MSTMb9ttvv63XXntNPp9PhYWF+vWvf62JEyea7taAWr9+vb788ks1NzcrOztbkyZN0uLFixPmfG687Nu3T6tXr45ov/zyy7Vs2bLQD1Hfe+89tbS0aNKkSbrtttvCfpQ3GPQ2DmVlZXriiSd08OBB+f1+jRw5UtOmTdONN944qI6Qb7jhhqjt5eXloQ+k3T9E/fjjj9XR0TEof4ja1zgcO3ZMzzzzjL777ju1tbVp1KhRmjlzphYtWqSsrKwB7m3PkiaAAACDS8J/BwQAGJwIIACAEQQQAMAIAggAYAQBBAAwggACABhBAAEAjCCAAABGEEAAACMIIACAEQQQAMAIAggAYMT/A53lrLF5W9z/AAAAAElFTkSuQmCC\n"
          },
          "metadata": {}
        },
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 640x480 with 1 Axes>"
            ],
            "image/png": "iVBORw0KGgoAAAANSUhEUgAAAaAAAAGdCAYAAABU0qcqAAAAOnRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjEwLjAsIGh0dHBzOi8vbWF0cGxvdGxpYi5vcmcvlHJYcgAAAAlwSFlzAAAPYQAAD2EBqD+naQAAJzFJREFUeJzt3X9w1PWdx/HX5seGZHdDgFyEcJJAY4xFiKCGnsKJyJyOZUahnUIdb5zT0M6EOm0tZ+e09YRCb7DF0To40+nEqzk9bMyVUZE6jtXRAHdDjwAFcjaDgHjByCD5xYaE/Nj7Y2f32Oxuks+XTT67m+fjH9jP9/vez4fPfndffHe/+1lXIBAICACACZZhewAAgMmJAAIAWEEAAQCsIIAAAFYQQAAAKwggAIAVBBAAwAoCCABgBQEEALCCAAIAWJFlewDxPPDAA/r4448j2jwejxobG7Vs2TL5/X5LI7OPeQhiHoKYhyDmISgZ5qGiokL//u//Pup+rvFaC+6dd97RW2+9pY6ODpWUlOjhhx9WWVnZmOsXL16sQ4cORbT5fD51dXUpPz9f3d3diR5yymAegpiHIOYhiHkISoZ5WLRokZqamkbdb1zegtu/f7/q6ur0zW9+U9u2bVNJSYm2bt2qzs7O8egOAJCCxiWAdu/erbvuukt33nmn/vqv/1rr16+X2+3WBx98MB7dAQBSUMI/AxoYGNDJkyd1//33h9syMjK0YMECtbS0RO3f39+v/v7+8G2Xy6Xc3Fx5PB75fL6IfUO3h7dPNsxDEPMQxDwEMQ9ByTAPHo9nTPslPIC6uro0NDSkgoKCiPaCggKdPXs2av9du3apoaEhfHvu3Lnatm2bGhsb4/bR2tqasPGmMuYhiHkIYh6CmIegVJgH61fBrV69WqtWrQrfdrlckqRly5bpyJEjEfv6fD61trZq9uzZk/5DRuaBeQhhHoKYh6BkmIfKysoRTyJCEh5A+fn5ysjIUEdHR0R7R0dH1FmRJGVnZys7Ozuq3e/3x5287u7uSX2AhTAPQcxDEPMQxDwE2ZyHsV7+nfCLELKysjRv3jwdO3Ys3DY0NKRjx46pvLw80d0BAFLUuLwFt2rVKu3YsUPz5s1TWVmZ9uzZo76+Pi1fvnw8ugMApKBxCaDbbrtNXV1dqq+vV0dHh0pLS/XEE0/EfAsOADA5jdtFCPfcc4/uuecex/WzZs3ShQsXItq8Xq8kac6cObp48WLEtk8//dRxX8BEO3r0qHHNggULxmEkgD0sRgoAsIIAAgBYQQABAKwggAAAVhBAAAArCCAAgBUEEADACgIIAGAFAQQAsIIAAgBYQQABAKwggAAAVlj/RdR4Pv/886gFRkO/cX7mzBl+cAopzcnCorF+uPHK9lg/7tjf328+OGCCcAYEALCCAAIAWEEAAQCsIIAAAFYQQAAAKwggAIAVBBAAwAoCCABgBQEEALCCAAIAWEEAAQCsIIAAAFYQQAAAK5J2NeyJkJHhLH+HhoYSPBK7pkyZ4qiut7c3wSPBSAYGBkZsHxgYiNqnvLzcUV8tLS2O6gATnAEBAKwggAAAVhBAAAArCCAAgBUEEADACgIIAGAFAQQAsIIAAgBYQQABAKwggAAAVhBAAAArCCAAgBWTejHSdFtU1CkWFZ14LpfLuGbv3r0x2zMzMyVJ7777rgYHByO23XLLLeaDk/Twww8b17S1tRnXfPnll8Y1//Zv/xazPbS48H/9139FPbedzkNfX5+jOowNZ0AAACsIIACAFQQQAMAKAggAYAUBBACwggACAFhBAAEArCCAAABWEEAAACsIIACAFQQQAMAKAggAYMWkXowUSAQnC10eOHDAuGa0BUyXLFkS1TYwMGDcjyT19PQY13z96183rvnRj35kXHPPPffEbPd6vfr444+1Zs0aXbx4MWIbi4omJ86AAABWEEAAACsS/hZcfX29GhoaItqKi4v13HPPJborAEAKG5fPgK699lr99Kc/Dd8O/VAUAAAh4xJAGRkZKigoGI+7BgCkiXEJoLa2Nn33u99Vdna2ysvL9cADD6iwsDDmvv39/erv7w/fdrlcys3Nlcfjkc/ni9g3dHt4+2TDPAQlyzx4PB6r/Y+H3Nxc4xq3221c4+Sx83q9MdtDj0Osx8P2MTKRkuF5MdbnhCsQCAQS2fGhQ4fU29ur4uJitbe3q6GhQRcuXND27dtjHtTDPzOaO3eutm3blsghAQCSUMIDaDi/36+amho99NBDWrFiRdT2eGdAy5Yt05EjRyL29fl8am1t1ezZs9Xd3T2ew05qzENQsszD4sWLjWs++OAD45rRvgcUi9PvAT300EPGNbfddptxzRNPPGFcM2vWrJjtHo9HBw8e1M033yy/3x+x7ezZs8b9pKpkeF5UVlaqsbFx1P3G/YuoHo9HxcXFamtri7k9Oztb2dnZUe1+vz/u5HV3d0/qF94Q5iHI9jwMf7FLB5cuXTKuuXz5snGNk8dttLeW/H5/1BdRJ+PzxObzYqzPiXG/PK23t1dtbW1clAAAiJDwM6C6ujrdcsstKiwsVHt7u+rr65WRkaGlS5cmuisAQApLeABduHBBzz//vLq7u5Wfn6+Kigpt3bpV+fn5ie4KAJDCEh5AP/jBDxJ9l0hS06dPN665cOFCwvoJfRYwffr0qM8Rr7ywxcSLL75oXLNy5UrjGicXFDjx9ttvO6o7ePCgcc2uXbsc9WWqtbU1ZnvoeDh79uyk/MwnFbFEAQDACgIIAGAFAQQAsIIAAgBYQQABAKwggAAAVhBAAAArCCAAgBUEEADACgIIAGAFAQQAsIIAAgBYMe4/SOdUU1NT3G1dXV1RbRO1uKNTmZmZxjVZWbEfnpycnPCfw38ErK+vz7ifwsJC4xpJOn/+vKM6U/fdd1/M9ilTpkiS7r33XvX29kZs+9WvfuWor7H+lv2VJurYG+mxzcnJibk9Ly/PUV9VVVXGNf/7v/9rXDM0NGRcg/TBGRAAwAoCCABgBQEEALCCAAIAWEEAAQCsIIAAAFYQQAAAKwggAIAVBBAAwAoCCABgBQEEALCCAAIAWEEAAQCsSNrVsBcvXqxDhw5FtPl8PnV1dSk/P1/d3d2WRuZMIBAwriktLY3ZHlqxec6cOfL7/RHb/vKXvxj343RVaycrfM+fP9+45qWXXhpx+4svvmh8n7adO3fOuKasrCxmu8/nU2trq+bNmxf1vHC73Y7Gd/jwYeOa/fv3G9ecPXvWuAbpgzMgAIAVBBAAwAoCCABgBQEEALCCAAIAWEEAAQCsIIAAAFYQQAAAKwggAIAVBBAAwAoCCABgBQEEALAiaRcjTTdf+9rXjGuOHz8es93n80kKLmiZiEVZL1y44Kiurq7OuKampsZRX+nm17/+tXFNb29vzPbs7Ozw9uH79PT0mA9O0syZM41rWltbjWucLGB6++23G9cgOXEGBACwggACAFhBAAEArCCAAABWEEAAACsIIACAFQQQAMAKAggAYAUBBACwggACAFhBAAEArCCAAABWsBipAx6Px7jGyaKL8QwNDUmSOjs7oxYjLSwsNL6/G264wdE4Tpw4YVxTUVFhXPPJJ58Y10ykjz76yLhmy5YtxjX9/f0jtvf390ft43K5jPuRpIyMifm/aVVVVcJqQs/LW265RX6/P2Lbf//3f5sPTlJBQYFxTWdnp3HN4OCgcU064AwIAGAFAQQAsML4Lbjm5ma9+eabOnXqlNrb27Vx48aIU+JAIKD6+nr98Y9/lN/vV0VFhaqrqzVr1qyEDhwAkNqMz4D6+vpUWlqqRx55JOb2N954Q3/4wx+0fv16/fznP1dOTo62bt2qy5cvX/VgAQDpwziAFi1apHXr1sX8IDAQCGjPnj1as2aNbr31VpWUlOh73/ue2tvb9ac//SkhAwYApIeEXgV37tw5dXR0aOHCheG2vLw8lZWVqaWlJeZP6Q6/csflcik3N1cejyf809MhodvD2yeak6vgEnlV0Ujz4PV6je8vMzPzqsc0Vk7Gl+yczJ+TYzjeuwgjHQ9Or4JLZvGef3l5eRF/Xsnpa4aTutBVqiYSeRVcMrxOjvU1MqEB1NHRIUmaOnVqRPvUqVPD24bbtWuXGhoawrfnzp2rbdu2qbGxMW4/Tn57Ph2l4jwcOXLE9hASLtZ/rEZz/vz5hI8jFY+HrCzzl6D3339/xO27d+92Opy0kgrHg/XvAa1evVqrVq0K3w79j23ZsmVRL1Y+n0+tra2aPXt21PdfJpKTM6Dh30u4GiPNw4wZM4zvz+kZ0OHDh41rnLxYJ3to7du3z7jmvvvuM64Z6Qwo3vHg9Ayovb3duMbJWf7AwIBxzd/93d/FbM/Ly9Pu3bu1atUq9fT0RGxramoy7kdKze8BJcPrZGVl5YgnESEJDaDQg9XZ2alp06aF2zs7O1VaWhqzJjs7W9nZ2VHtfr8/7uR1d3dbDSAnp9iJDKCQWPOQk5NjfD8T+RbcxYsXJ6yvieLkxcPJ8TvahTyxjod0fAtutOdST09P1D5OXy+cPDec9DUeX0S1+To51te7hH4PqKioSAUFBTp69Gi4raenRydOnFB5eXkiuwIApDjjM6De3l61tbWFb587d06nT5+W1+tVYWGh7r33Xv3+97/XrFmzVFRUpNdee03Tpk3TrbfemtCBAwBSm3EAffLJJ9q0aVP4dl1dnSTpjjvu0IYNG3Tfffepr69Pv/71r9XT06OKigo98cQTcrvdiRs1ACDlGQfQ/PnzVV9fH3e7y+XS2rVrtXbt2qsaWDJz8nmOk8+NnHyo6+TqqlifwY1FUVGRcc1YPpi06e///u+Na1599VXjmkAgYFzjRLJ/BuTkM5Y33ngjZnvo37pz586o+d27d6/54OTseJisC4s6wVpwAAArCCAAgBUEEADACgIIAGAFAQQAsIIAAgBYQQABAKwggAAAVhBAAAArCCAAgBUEEADACgIIAGAFAQQAsML6T3JPFk5WtnbCyQrV586dc9TXzp07jWvmz59vXDPSytEulyvm9o6ODuN+JOmVV15xVJesnKzCLjn72XknP+PtZCX2a665ZsTtsZ4Dq1evNu5Hkvr7+x3VYWw4AwIAWEEAAQCsIIAAAFYQQAAAKwggAIAVBBAAwAoCCABgBQEEALCCAAIAWEEAAQCsIIAAAFYQQAAAK5J2MVKXyyWXyxXVFm/bSAtWxuN0gVAnCzz+4he/MK75x3/8R+MaJwuLlpaWGtdIUlVVlaM6U9///vdjtk+ZMkXPPPOMfvzjH6u3tzdi2wsvvDARQ0tbw+dzLI4ePWpcc+uttxrXjGb4a4MkDQwMOLqvpUuXGtd8+OGHjvqajDgDAgBYQQABAKwggAAAVhBAAAArCCAAgBUEEADACgIIAGAFAQQAsIIAAgBYQQABAKwggAAAVhBAAAArknYx0kAgELXAaOh2rG1Tpkwx7sPJgotOOVlY1AknC6x2dXU56svJ4pN79+41rnnxxRdjtvt8Pj3zzDP6zW9+o+7ubuP7RWLdcMMNtocQV09Pj6O6EydOJHgk9sVarHU0ThZ7HgvOgAAAVhBAAAArCCAAgBUEEADACgIIAGAFAQQAsIIAAgBYQQABAKwggAAAVhBAAAArCCAAgBUEEADAiqRdjNTURC4smsy8Xq9xzVtvveWor5tuusm4pqioyLjmX/7lX2K2Dw4Ohv8M/R32eDwe20OI6+2333ZU19ramuCR2DdeC4s6wRkQAMAKAggAYIXxW3DNzc168803derUKbW3t2vjxo2qqqoKb9+xY4c+/PDDiJrKyko9+eSTVz9aAEDaMA6gvr4+lZaWasWKFfrlL38Zc5+bbrpJNTU1/99JVtp81AQASBDjZFi0aJEWLVo08p1mZamgoMDpmAAAk8C4nJo0NzerurpaHo9HN954o9atWyefzxdz3/7+fvX394dvu1wu5ebmyuPxRNWEbse7r8lipHlwMjeZmZlXPaaxcvKT4fH+TRwPQczD6LKzsx3VpeKcJsPxMNYrIl2Bq7gm71vf+lbUZ0D79u1TTk6OioqK1NbWpp07d2rKlCnaunVrzBef+vp6NTQ0hG/PnTtX27ZtczokAECKSPgZ0O233x7++5w5c1RSUqJHH31Ux48f14IFC6L2X716tVatWhW+7XK5JEnLli3TkSNHIvb1+XxqbW3V7Nmz1d3dneihp4yR5iE/P9/4/v7jP/7D0ThiPZ6j+fjjj41rli9fHrOd4yEoWeahs7PTuCb0fB9vv/vd7xzVrV+/PsEjGX/JcDxUVlaqsbFx1P3G/eqAa665Rj6fT21tbTFfsLKzs2OeHvv9/riT193dPalfcEJizYOTJ/REfolzaGjIuGa0x5rjIYh5iO/Kt/lNpPJ82jwe/H7/mPYb9+8Bffnll7p48aKmTZs23l0BAFKI8RlQb2+v2trawrfPnTun06dPy+v1yuv16vXXX9eSJUtUUFCgL774Qq+88opmzpypysrKhA4cAJDajAPok08+0aZNm8K36+rqJEl33HGH1q9frzNnzujDDz+U3+/X9OnTtXDhQq1du9bxVSgAgPRkHEDz589XfX193O2seGBXaWmpcc2dd97pqK/Dhw8b13z961931Bcm1vPPP29cM1EXFDjx8MMP2x4CYmAtOACAFQQQAMAKAggAYAUBBACwggACAFhBAAEArCCAAABWEEAAACsIIACAFQQQAMAKAggAYAUBBACwggACAFgx7r+Iiol19OhR45re3l5HfbndbuOaQ4cOGdd8+9vfjtmel5cnKfgz8D09PRHbPvroI+N+0tHXvvY1R3WPPvpogkeSOA888EDM9tzcXNXW1uqRRx7RpUuXIrY5/UVUjC/OgAAAVhBAAAArCCAAgBUEEADACgIIAGAFAQQAsIIAAgBYQQABAKwggAAAVhBAAAArCCAAgBUEEADAChYjTTMul8u4JiPD2f9D/vznPxvXzJ8/37jm/fffH3H77t27o9oqKiqM+5GkEydOOKqbCPEWfw21u93uqH1ef/31cR/X1Whvbzeuee2112K2+3w+1dbWqqGhQd3d3Vc7NEwAzoAAAFYQQAAAKwggAIAVBBAAwAoCCABgBQEEALCCAAIAWEEAAQCsIIAAAFYQQAAAKwggAIAVBBAAwAoWI00zWVnmD2lJSYmjvn7wgx8Y1xw8eNC4Jt7CopmZmaqqqtLBgwc1ODgYse3y5cvG/UjOFnMNBALGNZmZmcY18RaNDbVnZGRE7fP5558b9yNJxcXFjupMOZm7eI9RqN3lckXt46QfjD/OgAAAVhBAAAArCCAAgBUEEADACgIIAGAFAQQAsIIAAgBYQQABAKwggAAAVhBAAAArCCAAgBUEEADAChYjTTNOFuEcGBhw1NeWLVuMa/72b//WuGbp0qUx291ut6qqqrRv376of/fFixeN+5lIThaN9Xq9Mds9Hk/4z+GLbs6fP998cHK2eKeT42jOnDnGNU4WI0128RaaHcnQ0NA4jGRicQYEALCCAAIAWGH0PsCuXbt04MABtba2yu12q7y8XA8++GDEb4dcvnxZdXV12r9/v/r7+1VZWanq6moVFBQkeuwAgBRmdAbU3Nysu+++W1u3btVPfvITDQ4OasuWLert7Q3v8/LLL+vgwYN67LHHtGnTJrW3t2v79u0JHzgAILUZBdCTTz6p5cuX69prr1Vpaak2bNig8+fP6+TJk5Kknp4evf/++3rooYd04403at68eaqpqdFf/vIXtbS0jMs/AACQmq7qKrienh5J/391zsmTJzU4OKgFCxaE95k9e7YKCwvV0tKi8vLyqPvo7+9Xf39/+LbL5VJubq48Ho98Pl/EvqHbw9snm0TPQ7yrq0aTnZ1tXJObm2tc43a7R2yPtd3p3Di5ksvJFWM5OTnGNfEep1C708fRJiePU7zjbqTnxfCfbE82ibwKLhleJ0NXZo7GcQANDQ3pt7/9ra6//vrwpZQdHR3KysqK6nzq1Knq6OiIeT+7du1SQ0ND+PbcuXO1bds2NTY2xu27tbXV6bDTCvMQtGHDhqi2H/7whxZGYtfRo0et9p+ZmWlc8/nnnyd8HGfOnEn4faaiVHh9cBxAtbW1+uyzz7R58+arGsDq1au1atWq8O3Q9fvLli3TkSNHIvb1+XxqbW3V7Nmz1d3dfVX9prJEz8Nf/dVfOarr6+szrrntttuMa/7mb/4mZrvb7daGDRu0Y8eOqO8BPf/888b9SFJ7e7txTTKcAR09elQLFiyI+v7T8ePHjftxOj4nZxklJSXGNVd+5nwln8+nM2fOaM6cOVHPi8l2BmT7dbKysnLEk4gQRwFUW1urpqYmbdq0STNmzAi3FxQUaGBgQH6/P+IsqLOzM+5VcNnZ2TFPqf1+f9zJ6+7untQBFJKoeXDytpgU/4VgJJcuXTKuGe3LtZcvX47ax+m8OKlzEkBOvjA8mosXLyb9F3CHczLfox13sZ4XkymAQmy+Tvr9/jHtZ/SvDgQCqq2t1YEDB/TUU0+pqKgoYvu8efOUmZkZ8VbA2bNndf78+Zif/wAAJi+jM6Da2lrt3btXjz/+uHJzc8Of6+Tl5cntdisvL08rVqxQXV2dvF6v8vLy9NJLL6m8vJwAAgBEMAqgd999V5L09NNPR7TX1NRo+fLlkqSHHnpILpdL27dv18DAQPiLqAAAXMkogOrr60fdx+12q7q6mtCxxMmVSE45eX/51VdfNa45f/58zPbQBSv3339/1Ocwjz32mHE/kvTRRx8Z1/zrv/6rcc2VXz0Yq3iXo0+ZMkWStHLlyqjPR5weD07qnFyU4uTzs3if54TaBwcHk/4zn+HSYWFRJ1gLDgBgBQEEALCCAAIAWEEAAQCsIIAAAFYQQAAAKwggAIAVBBAAwAoCCABgBQEEALCCAAIAWEEAAQCsIIAAAFY4/kluJKfQCtEm4q02PZof/ehHxjX5+fnGNVf+um4sc+bMiWpzurqwk58MHxgYMK758ssvjWvi/Ypq6BeFly9fHrXKtpN+JKm4uNi4JrQqtwknvxPW0tISsz0vLy/85/DVsHt6eoz7wfjjDAgAYAUBBACwggACAFhBAAEArCCAAABWEEAAACsIIACAFQQQAMAKAggAYAUBBACwggACAFhBAAEArEjaxUivv/56BQKBiLbQopQLFy6U3++P2Hb48OGJGlpSc7IwppOFJyXphRdeMK75yU9+Ylzz4IMPxmzPzc1VfX29vv3tb+vSpUsR2x577DHjfiTpn/7pn4xrDh48aFyTmZlpXPP000/HbHe73ZKkL774ImrBUqePrRNOFoD97LPPjGviLSwamtOenh4WH00RnAEBAKwggAAAVhBAAAArCCAAgBUEEADACgIIAGAFAQQAsIIAAgBYQQABAKwggAAAVhBAAAArCCAAgBWuwPAVP5NEVVWVDh06FNHm8/l04cIFTZ8+Xd3d3RHbnCzCmap8Pp+6urqUn58fNQ+TSSrPw7Rp04xrQouODuf1enXixAmVlZXp4sWLEds2bdrkaHzr1q0zrpk1a5ZxTbx/00g6Oztjtqfy8ZBIyTAPixYtUlNT06j7cQYEALCCAAIAWEEAAQCsIIAAAFYQQAAAKwggAIAVBBAAwAoCCABgBQEEALCCAAIAWEEAAQCsIIAAAFZk2R5APAMDA1ELjIZux9oGpJL29vaE3VdPT48k6dy5c1GLTy5cuNDRfU6dOtW4Zt++fcY1ixcvNq5B+uAMCABgBQEEALDC6C24Xbt26cCBA2ptbZXb7VZ5ebkefPBBFRcXh/d5+umn1dzcHFG3cuVKfec730nMiAEAacEogJqbm3X33XfrK1/5igYHB7Vz505t2bJFzz77rKZMmRLe76677tLatWvDt5386BQAIL0ZBdCTTz4ZcXvDhg2qrq7WyZMn9dWvfjXcnpOTo4KCgoQMEACQnq7qKrjQ1TderzeivbGxUY2NjSooKNDNN9+sb3zjG8rJyYl5H/39/erv7w/fdrlcys3Nlcfjkc/ni9g3dHt4+2TDPAQxD0EjzUNmZuaEjSMjw/wj5UQ+dhwPQckwDx6PZ0z7uQKBQMBJB0NDQ3rmmWfk9/v1s5/9LNz+3nvvqbCwUNOnT9enn36qV199VWVlZdq4cWPM+6mvr1dDQ0P49ty5c7Vt2zYnQwIApBDHAfSb3/xGhw8f1ubNmzVjxoy4+x07dkybN2/Wr371K82cOTNqe7wzoGXLlunIkSMR+/p8PrW2tmr27NlR33eYTJiHIOYhaKR5eO+99xzdZ1VVlXHN8OfrWCxbtsy4Jh6Oh6BkmIfKyko1NjaOup+jt+Bqa2vV1NSkTZs2jRg+klRWViZJamtrixlA2dnZys7Ojmr3+/1xJ6+7u3tSH2AhzEMQ8xAUax4GBwcnrP+hoSHjmvF43DgegmzOg9/vH9N+Rm/aBgIB1dbW6sCBA3rqqadUVFQ0as3p06clSdOmTTPpCgCQ5ozOgGpra7V37149/vjjys3NVUdHhyQpLy9PbrdbbW1t2rt3rxYvXiyv16szZ87o5Zdf1g033KCSkpLxGD8AIEUZBdC7774rKfhl0yvV1NRo+fLlysrK0tGjR7Vnzx719fVpxowZWrJkidasWZOwAQMA0oNRANXX14+4vbCwUJs2bbqqAQEAJoekXQ0bzrhcLuMahxdC4io4+X5OvO/ZhC7iiXVBzz333GM+OAW/DmHqz3/+s6O+JoKT7yhJzi6swNixGCkAwAoCCABgBQEEALCCAAIAWEEAAQCsIIAAAFYQQAAAKwggAIAVBBAAwAoCCABgBQEEALCCAAIAWMFipGmGhUUnXkVFhXFNS0uLcU28hTFD7UNDQ1H7dHV1GfcjSceOHTOuSeZjL9kXFc3NzTWuuXTp0jiMZGJxBgQAsIIAAgBYQQABAKwggAAAVhBAAAArCCAAgBUEEADACgIIAGAFAQQAsIIAAgBYQQABAKxI2rXgYq2v5fF4JEmVlZXy+/0TPaSkwTwEJcs8zJ0717gmNHYTLpcrZnteXp4kaeHCherp6YnYNjg4aNyPJGVmZhrXOO0rUZLleHBiypQpxjW9vb0x25NhHsa6PqIrkMwrCAIA0lZKvQV36dIl/fjHP06LVWCvBvMQxDwEMQ9BzENQKs1DSgVQIBDQqVOnknrZ94nAPAQxD0HMQxDzEJRK85BSAQQASB8EEADAipQKoOzsbH3zm99Udna27aFYxTwEMQ9BzEMQ8xCUSvPAVXAAACtS6gwIAJA+CCAAgBUEEADACgIIAGBF0q4FN9w777yjt956Sx0dHSopKdHDDz+ssrIy28OaUPX19WpoaIhoKy4u1nPPPWdnQBOkublZb775pk6dOqX29nZt3LhRVVVV4e2BQED19fX64x//KL/fr4qKClVXV2vWrFkWR514o83Djh079OGHH0bUVFZW6sknn5zooY6bXbt26cCBA2ptbZXb7VZ5ebkefPBBFRcXh/e5fPmy6urqtH//fvX396uyslLV1dUqKCiwN/AEG8s8PP3002pubo6oW7lypb7zne9M9HDjSokA2r9/v+rq6rR+/Xpdd911evvtt7V161Y999xzmjp1qu3hTahrr71WP/3pT8O3MzLS/yS2r69PpaWlWrFihX75y19GbX/jjTf0hz/8QRs2bFBRUZF+97vfaevWrXr22WfldrstjHh8jDYPknTTTTeppqYmfDsrKyWe4mPW3Nysu+++W1/5ylc0ODionTt3asuWLXr22WfDC3q+/PLLampq0mOPPaa8vDzV1tZq+/bt+tnPfmZ59IkzlnmQpLvuuktr164N306250NKHJ27d+/WXXfdpTvvvFOStH79ejU1NemDDz7Q/fffb3dwEywjIyOt/ic3FosWLdKiRYtibgsEAtqzZ4/WrFmjW2+9VZL0ve99T+vXr9ef/vQn3X777RM51HE10jyEZGVlpfXxMfxsbsOGDaqurtbJkyf11a9+VT09PXr//ff1/e9/XzfeeKMkqaamRj/84Q/V0tKi8vJyG8NOuNHmISQnJyepj4ekD6CBgQGdPHkyImgyMjK0YMECtbS02BuYJW1tbfrud7+r7OxslZeX64EHHlBhYaHtYVlz7tw5dXR0aOHCheG2vLw8lZWVqaWlJa0CaCyam5tVXV0tj8ejG2+8UevWrZPP57M9rHET+vkJr9crSTp58qQGBwe1YMGC8D6zZ89WYWFhWgXQcMPnIaSxsVGNjY0qKCjQzTffrG984xvKycmxMcSYkj6Aurq6NDQ0FJXiBQUFOnv2rJ1BWXLdddeppqZGxcXFam9vV0NDg5566ilt375dubm5todnRUdHhyRFvRU7derU8LbJ4qabbtKSJUtUVFSktrY27dy5Uz//+c+1devWtHyrdmhoSL/97W91/fXXa86cOZKCx0NWVlbU7y2l8/EQax4kaenSpSosLNT06dP16aef6tVXX9XZs2e1ceNGi6ONlPQBhP935dsvJSUl4UD6z//8T61YscLiyJAMrjzbmzNnjkpKSvToo4/q+PHjEWcE6aK2tlafffaZNm/ebHsoVsWbh5UrV4b/PmfOHE2bNk2bN29WW1ubZs6cOdHDjCnp/1uUn5+vjIyMqP+9dHR0JPV7mxPB4/GouLhYbW1ttodiTegY6OzsjGjv7Oyc9MfHNddcI5/Pl5bHR21trZqamvTP//zPmjFjRri9oKBAAwMDUb8Emq7HQ7x5iCV01XAyHQ9JH0BZWVmaN2+ejh07Fm4bGhrSsWPH0vb93LHq7e1VW1tbWj6xxqqoqEgFBQU6evRouK2np0cnTpyY9MfHl19+qYsXL2ratGm2h5IwgUBAtbW1OnDggJ566ikVFRVFbJ83b54yMzMjjoezZ8/q/PnzaXU8jDYPsZw+fVqSkup4SIm34FatWqUdO3Zo3rx5Kisr0549e9TX16fly5fbHtqEqqur0y233KLCwkK1t7ervr5eGRkZWrp0qe2hjatQ0IacO3dOp0+fltfrVWFhoe699179/ve/16xZs1RUVKTXXntN06ZNC18Vly5Gmgev16vXX39dS5YsUUFBgb744gu98sormjlzpiorKy2OOrFqa2u1d+9ePf7448rNzQ2/M5KXlye32628vDytWLFCdXV18nq9ysvL00svvaTy8vK0CqDR5qGtrU179+7V4sWL5fV6debMGb388su64YYbVFJSYnfwV0iZ1bDfeecdvfnmm+ro6FBpaan+4R/+Qdddd53tYU2o5557Tv/zP/+j7u5u5efnq6KiQuvWrUua93PHy/Hjx7Vp06ao9jvuuEMbNmwIfxH1vffeU09PjyoqKvTII49EfCkvHYw0D+vXr9cvfvELnTp1Sn6/X9OnT9fChQu1du3atDpD/ta3vhWzvaamJvwf0tAXUfft26eBgYG0/CLqaPNw/vx5vfDCC/rss8/U19enGTNmqKqqSmvWrFFeXt4Ejza+lAkgAEB6SfrPgAAA6YkAAgBYQQABAKwggAAAVhBAAAArCCAAgBUEEADACgIIAGAFAQQAsIIAAgBYQQABAKwggAAAVvwfI4ShwH1bdmMAAAAASUVORK5CYII=\n"
          },
          "metadata": {}
        },
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 640x480 with 1 Axes>"
            ],
            "image/png": "iVBORw0KGgoAAAANSUhEUgAAAaAAAAGdCAYAAABU0qcqAAAAOnRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjEwLjAsIGh0dHBzOi8vbWF0cGxvdGxpYi5vcmcvlHJYcgAAAAlwSFlzAAAPYQAAD2EBqD+naQAAHjtJREFUeJzt3X9s1PUdx/HX1V5Le3flgK5Cq7RgqVWBij8gmXYiuGgMyQSNMGLi1FaTotlmiEtkoqB16SaEbSGLWWqkUXG1kYgOjfFHXBEX3HAErK7Tgpqyg4A9OK60Xuntj0tvHu1Br9z1fdc+H3/5/Xw/3/u+ffdbXv1+73vfc4TD4bAAABhlWdYFAADGJwIIAGCCAAIAmCCAAAAmCCAAgAkCCABgggACAJgggAAAJgggAIAJAggAYCLbuoB4Vq5cqc8//zxmzOVyqbW1VdXV1QoGg0aV2aMPEfQhgj5E0IeIdOhDZWWlXnrppXPOc6TqWXBvvfWWXn/9dfn9fpWWluree+9VeXn5sLe/6qqr9Mknn8SMeTwenThxQgUFBQoEAskuOWPQhwj6EEEfIuhDRDr0Yd68edqzZ88556XkEtyuXbvU1NSkO+64Qw0NDSotLVV9fb2OHz+eit0BADJQSgLojTfe0OLFi3XjjTfqoosuUm1trXJycvT++++nYncAgAyU9PeA+vr61NHRodtuuy06lpWVpTlz5qi9vX3Q/FAopFAoFF12OBzKy8uTy+WSx+OJmTuwfOb4eEMfIuhDBH2IoA8R6dAHl8s1rHlJD6ATJ06ov79fXq83Ztzr9erQoUOD5m/btk0tLS3R5RkzZqihoUGtra1x99HZ2Zm0ejMZfYigDxH0IYI+RGRCH8zvglu6dKmWLFkSXXY4HJKk6upq7d27N2aux+NRZ2enSkpKxv2bjPSBPgygDxH0ISId+lBVVXXWk4gBSQ+ggoICZWVlye/3x4z7/f5BZ0WS5HQ65XQ6B40Hg8G4zQsEAuP6ABtAHyLoQwR9iKAPEZZ9GO7t30m/CSE7O1szZ87U/v37o2P9/f3av3+/Kioqkr07AECGSskluCVLlmjz5s2aOXOmysvLtWPHDvX29mrhwoWp2B0AIAOlJIB++MMf6sSJE2pubpbf71dZWZkeffTRIS/BAQDGp5TdhHDLLbfolltuSdXLAwAyHA8jBQCYIIAAACYIIACACQIIAGCCAAIAmCCAAAAmCCAAgAkCCABgggACAJgggAAAJgggAIAJAggAYIIAAgCYIIAAACYIIACACQIIAGCCAAIAmCCAAAAmCCAAgAkCCABgggACAJgggAAAJgggAIAJAggAYIIAAgCYIIAAACYIIACACQIIAGCCAAIAmCCAAAAmCCAAgAkCCABgggACAJgggAAAJgggAIAJAggAYIIAAgCYIIAAACYIIACAiWzrAgAMT0FBwZDjHo8nut7hcMSs8/v9I9pXOBxOeJv29vaEt7nssssS3gZjB2dAAAATBBAAwAQBBAAwQQABAEwQQAAAEwQQAMAEAQQAMEEAAQBMEEAAABMEEADABAEEADBBAAEATPAwUiBD/OY3vxly3Ol0SpIef/xxhUKhmHVnPpx0uEayndfrHdG+MH5xBgQAMEEAAQBMJP0SXHNzs1paWmLGiouLtWnTpmTvCgCQwVLyHtDFF1+sxx57LLqclcWJFgAgVkoCKCsrizckAQBnlZIA8vl8euCBB+R0OlVRUaGVK1eqsLBwyLmhUCjmzh2Hw6G8vDy5XK7oVw0PGFg+c3y8oQ8R460PA3e7xRuPt360jOTOuWT+7Mbb8RBPOvTB5XINa54jPJIvfz+LTz75RD09PSouLlZXV5daWlr07bffasOGDcrLyxs0/8z3jGbMmKGGhoZklgQASENJD6AzBYNB1dXV6e6779aiRYsGrY93BlRdXa29e/fGzPV4POrs7FRJSYkCgUAqy05r9CFivPVhw4YNQ447nU797Gc/0/PPPz/oc0C1tbWjUZok6fDhwwlvM2vWrKTtf7wdD/GkQx+qqqrU2tp6znkp/yCqy+VScXGxfD7fkOudTueQlw6CwWDc5gUCgXF9gA2gDxHjpQ9nhstQ6881J5VG8rdsKn5u4+V4OBfLPgSDwWHNS/ntaT09PfL5fNyUAACIkfQzoKamJl1zzTUqLCxUV1eXmpublZWVpeuvvz7ZuwIAZLCkB9C3336r3//+9woEAiooKFBlZaXq6+tVUFCQ7F0BADJY0gPoF7/4RbJfEoDiP4zU7XartrZWGzdu1MmTJ2PW1dXVjWhf/f39CW/z/vvvj2hfGL94RAEAwAQBBAAwQQABAEwQQAAAEwQQAMAEAQQAMEEAAQBMEEAAABMEEADABAEEADBBAAEATBBAAAATKf9COgDJEe/L5gbGk/mFdCN5GOnKlSuTsm+MH5wBAQBMEEAAABMEEADABAEEADBBAAEATBBAAAATBBAAwAQBBAAwQQABAEwQQAAAEwQQAMAEAQQAMEEAAQBM8DRsIEP897//Pev6//znP0nb1z/+8Y+kvRYQD2dAAAATBBAAwAQBBAAwQQABAEwQQAAAEwQQAMAEAQQAMEEAAQBMEEAAABMEEADABAEEADBBAAEATPAwUsBASUlJwts4HI7zWp+In/70p0l7LSAezoAAACYIIACACQIIAGCCAAIAmCCAAAAmCCAAgAkCCABgggACAJgggAAAJgggAIAJAggAYIIAAgCY4GGkgIHPPvvMuoSzOnXqlHUJGAc4AwIAmCCAAAAmEr4E19bWpu3bt+vAgQPq6urS6tWrNX/+/Oj6cDis5uZmvfvuuwoGg6qsrFRNTY2mTZuW1MIBAJkt4TOg3t5elZWV6b777hty/WuvvaY333xTtbW1evrpp5Wbm6v6+np99913510sAGDsSDiA5s2bpxUrVsSc9QwIh8PasWOHli1bpmuvvValpaV68MEH1dXVpY8//jgpBQMAxoak3gV35MgR+f1+zZ07NzqWn5+v8vJytbe367rrrhu0TSgUUigUii47HA7l5eXJ5XLJ4/HEzB1YPnN8vKEPEfQhddxud8LbdHd3p6CS4eN4iEiHPrhcrmHNS2oA+f1+SdLEiRNjxidOnBhdd6Zt27appaUlujxjxgw1NDSotbU17n46OzvPu9axgD5E0Ifk++KLL6xLGDGOh4hM6IP554CWLl2qJUuWRJcdDockqbq6Wnv37o2Z6/F41NnZqZKSEgUCgVGtM53Qh4hM7sNI/nEYzb9oy8vLE97myJEjKahk+DL5eEimdOhDVVXVWU8iBiQ1gLxeryTp+PHjmjRpUnT8+PHjKisrG3Ibp9Mpp9M5aDwYDMZtXiAQGNcH2AD6EEEfku/kyZMJb5MuPwOOhwjLPgSDwWHNS+rngIqKiuT1erVv377oWHd3t7744gtVVFQkc1cAgAyX8BlQT0+PfD5fdPnIkSM6ePCg3G63CgsLdeutt+rVV1/VtGnTVFRUpJdfflmTJk3Stddem9TCAQCZLeEA+vLLL7Vu3broclNTkyTphhtu0KpVq/STn/xEvb29evbZZ9Xd3a3Kyko9+uijysnJSV7VAICMl3AAXXHFFWpubo673uFwaPny5Vq+fPl5FQaMZel+q/Dhw4etS8A4wLPgAAAmCCAAgAkCCABgggACAJgggAAAJgggAIAJAggAYIIAAgCYIIAAACYIIACACQIIAGCCAAIAmCCAAAAmzL+SG8h0999/v3UJcXV0dFiXAMTFGRAAwAQBBAAwQQABAEwQQAAAEwQQAMAEAQQAMEEAAQBMEEAAABMEEADABAEEADBBAAEATBBAAAATPIwUOE+bN2+2LiGuBQsWWJcAxMUZEADABAEEADBBAAEATBBAAAATBBAAwAQBBAAwQQABAEwQQAAAEwQQAMAEAQQAMEEAAQBMEEAAABM8jBT4HofDkfA2WVnp+3fcsWPHrEsA4krf3xwAwJhGAAEATBBAAAATBBAAwAQBBAAwQQABAEwQQAAAEwQQAMAEAQQAMEEAAQBMEEAAABMEEADABA8jBb7nkksuSXib0XoYaW9vb9x1ubm5Q64Ph8OpLAk4L5wBAQBMEEAAABMJX4Jra2vT9u3bdeDAAXV1dWn16tWaP39+dP3mzZv1wQcfxGxTVVWlNWvWnH+1AIAxI+EA6u3tVVlZmRYtWqRnnnlmyDlXXnml6urq/r+TbN5qAgDESjgZ5s2bp3nz5p39RbOz5fV6R1oTAGAcSMmpSVtbm2pqauRyuTR79mytWLFCHo9nyLmhUEihUCi67HA4lJeXJ5fLNWibgeV4rzVe0IeIVPTB7XYn7bXSwXg6Rvi9iEiHPrhcrmHNc4TP4z7NO++8c9B7QB9++KFyc3NVVFQkn8+nrVu3asKECaqvrx/ydtXm5ma1tLREl2fMmKGGhoaRlgQAyBBJPwO67rrrov89ffp0lZaW6qGHHtKnn36qOXPmDJq/dOlSLVmyJLrscDgkSdXV1dq7d2/MXI/Ho87OTpWUlCgQCCS79IxBHyJS0YeRfA7ok08+Scq+z2UknwP6wQ9+kMqS0gq/FxHp0Ieqqiq1traec17K7w648MIL5fF45PP5hgwgp9Mpp9M5aDwYDMZtXiAQGNcH2AD6EJHMPpw8eTIpr5MuxuPxwe9FhGUfgsHgsOal/HNAx44d08mTJzVp0qRU7woAkEESPgPq6emRz+eLLh85ckQHDx6U2+2W2+3WK6+8ogULFsjr9erw4cN64YUXNHXqVFVVVSW1cABAZks4gL788kutW7cuutzU1CRJuuGGG1RbW6uvv/5aH3zwgYLBoCZPnqy5c+dq+fLlQ15mAwCMXwkH0BVXXKHm5ua463niATLZn/70J+sS4tqyZcuQ406nU/fcc49eeumlmI80AOmOZ8EBAEwQQAAAEwQQAMAEAQQAMEEAAQBMEEAAABMEEADABAEEADBBAAEATBBAAAATBBAAwAQBBAAwQQABAEyk/BtRgUzyox/9yLqEuOrq6oYc93g8uueee/Twww/zTaDIKJwBAQBMEEAAABMEEADABAEEADBBAAEATBBAAAATBBAAwAQBBAAwQQABAEwQQAAAEwQQAMAEAQQAMMHDSDEmORyOEW337bffJrzN1KlTE94mFAolvE1OTs5Zx3NycgbNOXXqVML7AUYLZ0AAABMEEADABAEEADBBAAEATBBAAAATBBAAwAQBBAAwQQABAEwQQAAAEwQQAMAEAQQAMEEAAQBMjJmHkT7wwAMJb/Pss8+moBKkg6uvvnpE202cODHJlQzt2LFjCW/T09Mz5LjT6YyujzcHSEecAQEATBBAAAATBBAAwAQBBAAwQQABAEwQQAAAEwQQAMAEAQQAMEEAAQBMEEAAABMEEADABAEEADAxZh5GyoNF8X3bt28f0XZ5eXlJrmRoK1euTHibcDh81vFwOBx3DpCOOAMCAJgggAAAJhK6BLdt2zbt3r1bnZ2dysnJUUVFhe666y4VFxdH53z33XdqamrSrl27FAqFVFVVpZqaGnm93mTXDgDIYAmdAbW1tenmm29WfX29fv3rX+v06dN66qmnYr4Ea8uWLfrnP/+phx9+WOvWrVNXV5c2bNiQ9MIBAJktoQBas2aNFi5cqIsvvlhlZWVatWqVjh49qo6ODklSd3e33nvvPd19992aPXu2Zs6cqbq6Ov373/9We3t7Sv4HAACZ6bzuguvu7pYkud1uSVJHR4dOnz6tOXPmROeUlJSosLBQ7e3tqqioGPQaoVBIoVAouuxwOJSXlyeXyyWPxxMzd2D5zPHxhj5EnK0PDodjtMtJSH5+fsLbxPt5czxE0IeIdOiDy+Ua1rwRB1B/f7+ef/55XXrppZo+fbokye/3Kzs7e9DOJ06cKL/fP+TrbNu2TS0tLdHlGTNmqKGhQa2trXH33dnZOdKyxxT6EJGJfXjjjTeS/pqZ2IdUoA8RmdCHEQdQY2OjvvnmG61fv/68Cli6dKmWLFkSXR74y7W6ulp79+6NmevxeNTZ2amSkhIFAoHz2m8mow8RZ+vDSC/5Tp06NRmlndP3j/nh+tvf/jbkOMdDBH2ISIc+VFVVnfUkYsCIAqixsVF79uzRunXrNGXKlOi41+tVX1+fgsFgzFnQ8ePH494F53Q65XQ6B40Hg8G4zQsEAuP6ABtAHyKG6kO6fyBz4PJ1Is71s+Z4iKAPEZZ9CAaDw5qX0E0I4XBYjY2N2r17t9auXauioqKY9TNnztQFF1ygffv2RccOHTqko0ePDvn+DwBg/EroDKixsVE7d+7UI488ory8vOj7Ovn5+crJyVF+fr4WLVqkpqYmud1u5efn67nnnlNFRQUBBACIkVAAvf3225KkJ554Ima8rq5OCxculCTdfffdcjgc2rBhg/r6+qIfRAUA4PsSCqDm5uZzzsnJyVFNTQ2hg6SZPHnykOMDt5lOnjx50PuIfX19Ka9rwMmTJxPe5qOPPkpBJUBm4VlwAAATBBAAwAQBBAAwQQABAEwQQAAAEwQQAMAEAQQAMEEAAQBMEEAAABMEEADABAEEADBBAAEATBBAAAATI/5KbmC03HrrrUOOT5gwQZL04x//WD09PTHrLrroopTXdWYdibjmmmsS3mbnzp0JbwOkM86AAAAmCCAAgAkCCABgggACAJgggAAAJgggAIAJAggAYIIAAgCYIIAAACYIIACACQIIAGCCAAIAmOBhpEh78R7C6Xa7JUl///vfdfLkyZh17e3tI9rXpZdemvA2n3/+ecLb8GBRgDMgAIARAggAYIIAAgCYIIAAACYIIACACQIIAGCCAAIAmCCAAAAmCCAAgAkCCABgggACAJgggAAAJngYKdLewYMHhxz3eDySpK+++kqBQCBmXWVlZarLAnCeOAMCAJgggAAAJgggAIAJAggAYIIAAgCYIIAAACYIIACACQIIAGCCAAIAmCCAAAAmCCAAgAkCCABgggACAJgggAAAJgggAICJhL4PaNu2bdq9e7c6OzuVk5OjiooK3XXXXSouLo7OeeKJJ9TW1haz3U033aT7778/ORUDAMaEhAKora1NN998sy655BKdPn1aW7du1VNPPaWNGzdqwoQJ0XmLFy/W8uXLo8s5OTnJqxgAMCYkFEBr1qyJWV61apVqamrU0dGhyy+/PDqem5srr9eblAIBAGPTeX0ld3d3tyTJ7XbHjLe2tqq1tVVer1dXX321br/9duXm5g75GqFQSKFQKLrscDiUl5cnl8sV/crlAQPLZ46PN/Qhgj5E0IcI+hCRDn1wuVzDmucIh8Phkeygv79fv/3tbxUMBvXkk09Gx9955x0VFhZq8uTJ+uqrr/Tiiy+qvLxcq1evHvJ1mpub1dLSEl2eMWOGGhoaRlISACCDjDiA/vznP+tf//qX1q9frylTpsSdt3//fq1fv15/+MMfNHXq1EHr450BVVdXa+/evTFzPR6POjs7VVJSokAgMJKyxwT6EEEfIuhDBH2ISIc+VFVVqbW19ZzzRnQJrrGxUXv27NG6devOGj6SVF5eLkny+XxDBpDT6ZTT6Rw0HgwG4zYvEAiM6wNsAH2IoA8R9CGCPkRY9iEYDA5rXkKfAwqHw2psbNTu3bu1du1aFRUVnXObgwcPSpImTZqUyK4AAGNcQmdAjY2N2rlzpx555BHl5eXJ7/dLkvLz85WTkyOfz6edO3fqqquuktvt1tdff60tW7bosssuU2lpaSrqBwBkqIQC6O2335YU+bDp99XV1WnhwoXKzs7Wvn37tGPHDvX29mrKlClasGCBli1blrSCAQBjQ0IB1NzcfNb1hYWFWrdu3XkVBAAYH3gWHADABAEEADBBAAEATBBAAAATBBAAwAQBBAAwQQABAEwQQAAAEwQQAMAEAQQAMEEAAQBMEEAAABMEEADABAEEADBBAAEATBBAAAATBBAAwAQBBAAwQQABAEwQQAAAEwQQAMAEAQQAMEEAAQBMEEAAABMEEADARLZ1AfFUVlYOGnO5XJKkqqoqBYPB0S4pbdCHCPoQQR8i6ENEOvRhqH+/h+IIh8PhFNcCAMAgGXUJ7tSpU/rVr36lU6dOWZdiij5E0IcI+hBBHyIyqQ8ZFUDhcFgHDhzQeD9pow8R9CGCPkTQh4hM6kNGBRAAYOwggAAAJjIqgJxOp+644w45nU7rUkzRhwj6EEEfIuhDRCb1gbvgAAAmMuoMCAAwdhBAAAATBBAAwAQBBAAwkbbPgjvTW2+9pddff11+v1+lpaW69957VV5ebl3WqGpublZLS0vMWHFxsTZt2mRT0Chpa2vT9u3bdeDAAXV1dWn16tWaP39+dH04HFZzc7PeffddBYNBVVZWqqamRtOmTTOsOvnO1YfNmzfrgw8+iNmmqqpKa9asGe1SU2bbtm3avXu3Ojs7lZOTo4qKCt11110qLi6Ozvnuu+/U1NSkXbt2KRQKqaqqSjU1NfJ6vXaFJ9lw+vDEE0+ora0tZrubbrpJ999//2iXG1dGBNCuXbvU1NSk2tpazZo1S3/9619VX1+vTZs2aeLEidbljaqLL75Yjz32WHQ5K2vsn8T29vaqrKxMixYt0jPPPDNo/WuvvaY333xTq1atUlFRkf7yl7+ovr5eGzduVE5OjkHFqXGuPkjSlVdeqbq6uuhydnZG/IoPW1tbm26++WZdcsklOn36tLZu3aqnnnpKGzdu1IQJEyRJW7Zs0Z49e/Twww8rPz9fjY2N2rBhg5588knj6pNnOH2QpMWLF2v58uXR5XT7fciIo/ONN97Q4sWLdeONN0qSamtrtWfPHr3//vu67bbbbIsbZVlZWWPqL7nhmDdvnubNmzfkunA4rB07dmjZsmW69tprJUkPPvigamtr9fHHH+u6664bzVJT6mx9GJCdnT2mj48zz+ZWrVqlmpoadXR06PLLL1d3d7fee+89/fznP9fs2bMlSXV1dfrlL3+p9vZ2VVRUWJSddOfqw4Dc3Ny0Ph7SPoD6+vrU0dEREzRZWVmaM2eO2tvb7Qoz4vP59MADD8jpdKqiokIrV65UYWGhdVlmjhw5Ir/fr7lz50bH8vPzVV5ervb29jEVQMPR1tammpoauVwuzZ49WytWrJDH47EuK2W6u7slSW63W5LU0dGh06dPa86cOdE5JSUlKiwsHFMBdKYz+zCgtbVVra2t8nq9uvrqq3X77bcrNzfXosQhpX0AnThxQv39/YNS3Ov16tChQzZFGZk1a5bq6upUXFysrq4utbS0aO3atdqwYYPy8vKsyzPh9/sladCl2IkTJ0bXjRdXXnmlFixYoKKiIvl8Pm3dulVPP/206uvrx+Sl2v7+fj3//PO69NJLNX36dEmR4yE7Ozv6nTgDxvLxMFQfJOn6669XYWGhJk+erK+++kovvviiDh06pNWrVxtWGyvtAwj/9/3LL6WlpdFA+uijj7Ro0SLDypAOvn+2N336dJWWluqhhx7Sp59+GnNGMFY0Njbqm2++0fr1661LMRWvDzfddFP0v6dPn65JkyZp/fr18vl8mjp16miXOaS0/7OooKBAWVlZg/568fv9aX1tczS4XC4VFxfL5/NZl2Jm4Bg4fvx4zPjx48fH/fFx4YUXyuPxjMnjo7GxUXv27NHjjz+uKVOmRMe9Xq/6+voGfRPoWD0e4vVhKAN3DafT8ZD2AZSdna2ZM2dq//790bH+/n7t379/zF7PHa6enh75fL4x+Ys1XEVFRfJ6vdq3b190rLu7W1988cW4Pz6OHTumkydPatKkSdalJE04HFZjY6N2796ttWvXqqioKGb9zJkzdcEFF8QcD4cOHdLRo0fH1PFwrj4M5eDBg5KUVsdDRlyCW7JkiTZv3qyZM2eqvLxcO3bsUG9vrxYuXGhd2qhqamrSNddco8LCQnV1dam5uVlZWVm6/vrrrUtLqYGgHXDkyBEdPHhQbrdbhYWFuvXWW/Xqq69q2rRpKioq0ssvv6xJkyZF74obK87WB7fbrVdeeUULFiyQ1+vV4cOH9cILL2jq1KmqqqoyrDq5GhsbtXPnTj3yyCPKy8uLXhnJz89XTk6O8vPztWjRIjU1Ncntdis/P1/PPfecKioqxlQAnasPPp9PO3fu1FVXXSW3262vv/5aW7Zs0WWXXabS0lLb4r8nY56G/dZbb2n79u3y+/0qKyvTPffco1mzZlmXNao2bdqkzz77TIFAQAUFBaqsrNSKFSvS5npuqnz66adat27doPEbbrhBq1atin4Q9Z133lF3d7cqKyt13333xXwobyw4Wx9qa2v1u9/9TgcOHFAwGNTkyZM1d+5cLV++fEydId95551DjtfV1UX/IB34IOqHH36ovr6+MflB1HP14ejRo/rjH/+ob775Rr29vZoyZYrmz5+vZcuWKT8/f5SrjS9jAggAMLak/XtAAICxiQACAJgggAAAJgggAIAJAggAYIIAAgCYIIAAACYIIACACQIIAGCCAAIAmCCAAAAmCCAAgIn/AX3xkq24Q2NqAAAAAElFTkSuQmCC\n"
          },
          "metadata": {}
        }
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "# 1 라이브러리 임포트\n",
        "import torch\n",
        "import torch.nn as nn\n",
        "import torch.optim as optim\n",
        "from torch.utils.data import DataLoader\n",
        "import torchvision.datasets as datasets\n",
        "import torchvision.transforms as transforms\n",
        "import matplotlib.pyplot as plt\n",
        "from torchvision.utils import make_grid, save_image\n",
        "from tqdm import tqdm\n",
        "import numpy as np\n",
        "\n",
        "plt.style.use(\"ggplot\")\n",
        "device = torch.device(\"cuda\" if torch.cuda.is_available() else \"cpu\")"
      ],
      "metadata": {
        "id": "8Ngs0_w194Ek"
      },
      "execution_count": 15,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "# 2 변수 설정\n",
        "batch_size = 512\n",
        "epochs = 200\n",
        "sample_size = 64\n",
        "nz = 128  # 노이즈 벡터 크기\n",
        "num_classes = 10  # MNIST 숫자 클래스 (0~9)\n",
        "k = 1"
      ],
      "metadata": {
        "id": "ivCBE5eR-Q7Q"
      },
      "execution_count": 16,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "# 3 MNIST 데이터셋 다운로드\n",
        "transform = transforms.Compose(\n",
        "    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]\n",
        ")\n",
        "train_dataset = datasets.MNIST(\n",
        "    root=\"pytorch/data2\", train=True, transform=transform, download=True\n",
        ")\n",
        "train_loader = DataLoader(\n",
        "    train_dataset, batch_size=batch_size, shuffle=True, num_workers=4\n",
        ")"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "9_4ouWJf-Uqe",
        "outputId": "ce708c15-c4a6-4a54-8b00-5f1a5cd03ac7"
      },
      "execution_count": 17,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stderr",
          "text": [
            "100%|██████████| 9.91M/9.91M [00:00<00:00, 54.4MB/s]\n",
            "100%|██████████| 28.9k/28.9k [00:00<00:00, 1.69MB/s]\n",
            "100%|██████████| 1.65M/1.65M [00:00<00:00, 14.8MB/s]\n",
            "100%|██████████| 4.54k/4.54k [00:00<00:00, 7.84MB/s]\n",
            "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.\n",
            "  warnings.warn(\n"
          ]
        }
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "# 4 생성자 (Conditional GAN)\n",
        "class Generator(nn.Module):\n",
        "    def __init__(self, nz, num_classes):\n",
        "        super().__init__()\n",
        "        self.nz = nz\n",
        "        self.num_classes = num_classes\n",
        "        # 레이블 임베딩\n",
        "        self.label_emb = nn.Embedding(num_classes, num_classes)\n",
        "        # 생성자 입력: 노이즈 + 레이블\n",
        "        self.main = nn.Sequential(\n",
        "            nn.Linear(self.nz + num_classes, 256),\n",
        "            nn.LeakyReLU(0.2),\n",
        "            nn.Linear(256, 512),\n",
        "            nn.LeakyReLU(0.2),\n",
        "            nn.Linear(512, 1024),\n",
        "            nn.LeakyReLU(0.2),\n",
        "            nn.Linear(1024, 784),\n",
        "            nn.Tanh(),\n",
        "        )\n",
        "\n",
        "    def forward(self, x, labels):\n",
        "        # 레이블을 임베딩하고 노이즈와 결합\n",
        "        label_emb = self.label_emb(labels)\n",
        "        x = torch.cat([x, label_emb], dim=1)\n",
        "        return self.main(x).view(-1, 1, 28, 28)"
      ],
      "metadata": {
        "id": "msIA3gyx-fji"
      },
      "execution_count": 18,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "# 5 판별자 (Conditional GAN)\n",
        "class Discriminator(nn.Module):\n",
        "    def __init__(self, num_classes):\n",
        "        super().__init__()\n",
        "        self.n_input = 784\n",
        "        self.label_emb = nn.Embedding(num_classes, num_classes)\n",
        "        self.main = nn.Sequential(\n",
        "            nn.Linear(self.n_input + num_classes, 1024),\n",
        "            nn.LeakyReLU(0.2),\n",
        "            nn.Dropout(0.3),\n",
        "            nn.Linear(1024, 512),\n",
        "            nn.LeakyReLU(0.2),\n",
        "            nn.Dropout(0.3),\n",
        "            nn.Linear(512, 256),\n",
        "            nn.LeakyReLU(0.2),\n",
        "            nn.Dropout(0.3),\n",
        "            nn.Linear(256, 1),\n",
        "            nn.Sigmoid(),\n",
        "        )\n",
        "\n",
        "    def forward(self, x, labels):\n",
        "        x = x.view(-1, 784)\n",
        "        label_emb = self.label_emb(labels)\n",
        "        x = torch.cat([x, label_emb], dim=1)\n",
        "        return self.main(x)"
      ],
      "metadata": {
        "id": "Ejqk5lNF-uzx"
      },
      "execution_count": 19,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "# 6 객체 설정\n",
        "generator = Generator(nz, num_classes).to(device)\n",
        "discriminator = Discriminator(num_classes).to(device)\n",
        "print(generator)\n",
        "print(discriminator)"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "Psucfstg-2_5",
        "outputId": "d398dcf0-0b0d-4006-abed-713e2a3a7758"
      },
      "execution_count": 20,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Generator(\n",
            "  (label_emb): Embedding(10, 10)\n",
            "  (main): Sequential(\n",
            "    (0): Linear(in_features=138, out_features=256, bias=True)\n",
            "    (1): LeakyReLU(negative_slope=0.2)\n",
            "    (2): Linear(in_features=256, out_features=512, bias=True)\n",
            "    (3): LeakyReLU(negative_slope=0.2)\n",
            "    (4): Linear(in_features=512, out_features=1024, bias=True)\n",
            "    (5): LeakyReLU(negative_slope=0.2)\n",
            "    (6): Linear(in_features=1024, out_features=784, bias=True)\n",
            "    (7): Tanh()\n",
            "  )\n",
            ")\n",
            "Discriminator(\n",
            "  (label_emb): Embedding(10, 10)\n",
            "  (main): Sequential(\n",
            "    (0): Linear(in_features=794, out_features=1024, bias=True)\n",
            "    (1): LeakyReLU(negative_slope=0.2)\n",
            "    (2): Dropout(p=0.3, inplace=False)\n",
            "    (3): Linear(in_features=1024, out_features=512, bias=True)\n",
            "    (4): LeakyReLU(negative_slope=0.2)\n",
            "    (5): Dropout(p=0.3, inplace=False)\n",
            "    (6): Linear(in_features=512, out_features=256, bias=True)\n",
            "    (7): LeakyReLU(negative_slope=0.2)\n",
            "    (8): Dropout(p=0.3, inplace=False)\n",
            "    (9): Linear(in_features=256, out_features=1, bias=True)\n",
            "    (10): Sigmoid()\n",
            "  )\n",
            ")\n"
          ]
        }
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "# 7 손실 함수 및 옵티마이저 설정\n",
        "optim_g = optim.Adam(generator.parameters(), lr=0.0002)\n",
        "optim_d = optim.Adam(discriminator.parameters(), lr=0.0002)\n",
        "criterion = nn.BCELoss()\n",
        "\n",
        "losses_g = []\n",
        "losses_d = []\n",
        "images = []"
      ],
      "metadata": {
        "id": "wBtpWJ52-3CX"
      },
      "execution_count": 21,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "# 8 이미지 저장 함수\n",
        "def save_generator_image(image, path):\n",
        "    save_image(image, path)"
      ],
      "metadata": {
        "id": "524tBM1m-7pZ"
      },
      "execution_count": 22,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "# 9 판별자 학습 함수\n",
        "def train_discriminator(optimizer, data_real, labels, data_fake):\n",
        "    b_size = data_real.size(0)\n",
        "    real_label = torch.ones(b_size, 1).to(device)\n",
        "    fake_label = torch.zeros(b_size, 1).to(device)\n",
        "    optimizer.zero_grad()\n",
        "    output_real = discriminator(data_real, labels)\n",
        "    loss_real = criterion(output_real, real_label)\n",
        "    output_fake = discriminator(data_fake, labels)\n",
        "    loss_fake = criterion(output_fake, fake_label)\n",
        "    loss_real.backward()\n",
        "    loss_fake.backward()\n",
        "    optimizer.step()\n",
        "    return loss_real + loss_fake"
      ],
      "metadata": {
        "id": "OfNEYIxj-9IQ"
      },
      "execution_count": 23,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "# 10 생성자 학습 함수\n",
        "def train_generator(optimizer, data_fake, labels):\n",
        "    b_size = data_fake.size(0)\n",
        "    real_label = torch.ones(b_size, 1).to(device)\n",
        "    optimizer.zero_grad()\n",
        "    output = discriminator(data_fake, labels)\n",
        "    loss = criterion(output, real_label)\n",
        "    loss.backward()\n",
        "    optimizer.step()\n",
        "    return loss"
      ],
      "metadata": {
        "id": "X0dd-HiO--zH"
      },
      "execution_count": 24,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "# 11 모델 학습\n",
        "generator.train()\n",
        "discriminator.train()\n",
        "\n",
        "for epoch in range(epochs):\n",
        "    loss_g = 0.0\n",
        "    loss_d = 0.0\n",
        "    idxh = 0\n",
        "    for idx, (image, labels) in enumerate(train_loader):\n",
        "        image = image.to(device)\n",
        "        labels = labels.to(device)\n",
        "        b_size = len(image)\n",
        "        for step in range(k):\n",
        "            # 랜덤 노이즈와 동일한 레이블 사용\n",
        "            data_fake = generator(torch.randn(b_size, nz).to(device), labels).detach()\n",
        "            data_real = image\n",
        "            loss_d += train_discriminator(optim_d, data_real, labels, data_fake)\n",
        "        data_fake = generator(torch.randn(b_size, nz).to(device), labels).detach()\n",
        "        loss_g += train_generator(optim_g, data_fake, labels)\n",
        "        idxh = idx\n",
        "\n",
        "    # 특정 숫자(예: 0~9 중 랜덤)로 샘플 이미지 생성\n",
        "    sample_labels = torch.randint(0, num_classes, (sample_size,)).to(device)\n",
        "    generated_image = generator(torch.randn(sample_size, nz).to(device), sample_labels).cpu().detach()\n",
        "    generated_image = make_grid(generated_image)\n",
        "    save_generator_image(generated_image, f\"pytorch/data2/generated_images_{epoch}.png\")\n",
        "    images.append(generated_image)\n",
        "    epoch_loss_g = loss_g / idxh\n",
        "    epoch_loss_d = loss_d / idxh\n",
        "    losses_g.append(epoch_loss_g)\n",
        "    losses_d.append(epoch_loss_d)\n",
        "\n",
        "    print(f\"Epoch {epoch} of {epochs}\")\n",
        "    print(f\"Generator loss: {epoch_loss_g:.8f}, Discriminator loss: {epoch_loss_d:.8f}\")"
      ],
      "metadata": {
        "id": "7ujd-YEt_AZ7"
      },
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [],
      "metadata": {
        "id": "x_hPXXUh_Cxf"
      },
      "execution_count": null,
      "outputs": []
    }
  ]
}